In [ ]:
!pip install emmet-core

In [ ]:
!pip install datasets

In [ ]:
pip install --upgrade pyarrow


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade pandas datasets


   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.5 MB 1.1 MB/s eta 0:00:11
    --------------------------------------- 0.2/11.5 MB 1.7 MB/s eta 0:00:07
    --------------------------------------- 0.2/11.5 MB 1.4 MB/s eta 0:00:08
   - -------------------------------------- 0.3/11.5 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.4/11.5 MB 1.6 MB/s eta 0:00:08
   - -------------------------------------- 0.5/11.5 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.6/11.5 MB 1.7 MB/s eta 0:00:07
   -- ------------------------------------- 0.6/11.5 MB 1.6 MB/s eta 0:00:07
   -- ------------------------------------- 0.6/11.5 MB 1.5 MB/s eta 0:00:08
   -- ------------------------------------- 0.6/11.5 MB 1.4 MB/s eta 0:00:08
   -- ------------------------------------- 0.7/11.5 MB 1.4 MB/s eta 0:00:08
   -- -------

In [ ]:
import pickle
import pandas as pd
file_path = r"C:\Users\User\Downloads\Elasticity_Data.pkl"
with open(file_path, "rb") as f:
    data = pickle.load(f,encoding = "latin1")
print(data)

Dataset({
    features: ['text', 'label'],
    num_rows: 1870
})


In [ ]:
!pip install transformers

     ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
     -------------------------- ----------- 30.7/44.0 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 44.0/44.0 kB 544.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.5/10.0 MB 31.4 MB/s eta 0:00:01
   ------------------- -------------------- 4.8/10.0 MB 50.4 MB/s eta 0:00:01
   -------------------------------- ------- 8.1/10.0 MB 64.9 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 63.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/308.9 kB ? eta -:--:--
   ---------------------------------------- 308.9/308.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 151.6 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision torchaudio


   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 1.5/204.1 MB 31.8 MB/s eta 0:00:07
    --------------------------------------- 4.3/204.1 MB 54.6 MB/s eta 0:00:04
   - -------------------------------------- 5.8/204.1 MB 46.0 MB/s eta 0:00:05
   - -------------------------------------- 6.3/204.1 MB 36.4 MB/s eta 0:00:06
   - -------------------------------------- 6.7/204.1 MB 32.7 MB/s eta 0:00:07
   - -------------------------------------- 7.1/204.1 MB 28.2 MB/s eta 0:00:07
   - -------------------------------------- 7.3/204.1 MB 26.0 MB/s eta 0:00:08
   - -------------------------------------- 7.7/204.1 MB 23.4 MB/s eta 0:00:09
   - -------------------------------------- 8.0/204.1 MB 22.3 MB/s eta 0:00:09
   - -------------------------------------- 8.6/204.1 MB 21.1 MB/s eta 0:00:10
   - -------------------------------------- 9.3/204.1 MB 20.5 MB/s eta 0:00:10
   - -------------------------------------- 9.7/204.1 MB 19.

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding
)
model_id = "m3rg-iitd/matscibert"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels = 2)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding = "max_length", max_length = 512)
tokenized_data = data.map(tokenize_function, batched = True)
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
print(tokenized_data)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at m3rg-iitd/matscibert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1870 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1870
})


In [ ]:
from datasets import Dataset, DatasetDict

# First, split into train (70%) and temp (30%)
train_test_split = tokenized_data.train_test_split(test_size=0.3, seed=42)

# Then, split temp into validation (50%) and test (50%)
val_test_split = train_test_split["test"].train_test_split(test_size=0.5, seed=42)

# Combine into a DatasetDict

dataset_splits = DatasetDict({"train": train_test_split["train"],
    "val": val_test_split["train"],
    "test": val_test_split["test"]})

print(dataset_splits)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1309
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 280
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 281
    })
})


In [ ]:
train_data = dataset_splits["train"]
test_data = dataset_splits["test"]
val_data = dataset_splits["val"]

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)  # Get predicted labels

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')  # Use 'weighted' for class imbalance handling

    return {"accuracy": accuracy, "f1_score": f1}

In [ ]:
!pip install accelerate


   ---------------------------------------- 0.0/345.1 kB ? eta -:--:--
   - -------------------------------------- 10.2/345.1 kB ? eta -:--:--
   ---- ---------------------------------- 41.0/345.1 kB 495.5 kB/s eta 0:00:01
   ----------------- ---------------------- 153.6/345.1 kB 1.3 MB/s eta 0:00:01
   --------------------------- ------------ 235.5/345.1 kB 1.6 MB/s eta 0:00:01
   ----------------------------------- ---- 307.2/345.1 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 345.1/345.1 kB 1.5 MB/s eta 0:00:00


In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir='./logs',
    evaluation_strategy="steps",  # Validates at the end of each epoch
    save_strategy="steps",# see what
    logging_steps = 50,
    lr_scheduler_type="cosine",
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    max_grad_norm = 0.3,
    do_eval = True,
    save_steps = 50,
    eval_steps = 50,
    learning_rate = 1e-4 ,#(is better why?),
    weight_decay=0.01,#
    warmup_ratio = 0.1, #
    load_best_model_at_end=True,# Loads the best model based on validation loss
    optim = "adafactor",
    report_to="none"
)

KeyboardInterrupt: 

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)

C:\Users\User\AppData\Local\Temp\ipykernel_12924\1335924893.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1 Score
50,0.732200,0.825077,0.667857,0.637402
100,0.620000,0.568315,0.757143,0.756147
150,0.597000,0.504118,0.792857,0.790772
200,0.447200,0.506984,0.807143,0.806163
250,0.442700,0.435574,0.835714,0.835420
300,0.421500,0.516767,0.796429,0.794228
350,0.368700,0.459959,0.853571,0.853577
400,0.219600,0.745098,0.839286,0.839045
450,0.371000,0.681121,0.842857,0.842793


TrainOutput(global_step=492, training_loss=0.4627682251658866, metrics={'train_runtime': 7771.4284, 'train_samples_per_second': 0.505, 'train_steps_per_second': 0.063, 'total_flos': 1033237114398720.0, 'train_loss': 0.4627682251658866, 'epoch': 3.0})

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# After training is complete
model.save_pretrained("results")
tokenizer.save_pretrained("results")


('results\\tokenizer_config.json',
 'results\\special_tokens_map.json',
 'results\\vocab.txt',
 'results\\added_tokens.json',
 'results\\tokenizer.json')

In [ ]:
val_results = trainer.evaluate(val_data)
print(val_results)

{'eval_loss': 0.43557441234588623, 'eval_accuracy': 0.8357142857142857, 'eval_f1_score': 0.8354201831108383, 'eval_runtime': 84.1237, 'eval_samples_per_second': 3.328, 'eval_steps_per_second': 0.832, 'epoch': 3.0}


In [ ]:
test_predictions = trainer.predict(test_data)
print(test_predictions)


PredictionOutput(predictions=array([[ 1.1162564 , -1.566244  ],
       [ 1.3003608 , -2.0040238 ],
       [-1.9760388 ,  2.487948  ],
       [ 1.1435437 , -1.8600467 ],
       [ 1.1125146 , -1.7311445 ],
       [-1.738739  ,  2.138801  ],
       [ 0.8006565 , -1.2366127 ],
       [-1.9764359 ,  2.411737  ],
       [-1.9096619 ,  2.3432622 ],
       [ 0.96628386, -1.5958945 ],
       [-1.7466165 ,  2.262409  ],
       [ 1.0657753 , -1.4769865 ],
       [ 0.27427012, -0.58915716],
       [ 0.1433538 , -0.32276028],
       [ 1.2921957 , -1.9993584 ],
       [-1.9018886 ,  2.4592893 ],
       [-1.9127948 ,  2.3831549 ],
       [-1.7457223 ,  2.2498527 ],
       [ 1.0866753 , -1.7062414 ],
       [-1.9255317 ,  2.4802673 ],
       [ 0.6353973 , -0.9149554 ],
       [ 0.18689895, -0.2921076 ],
       [-1.9787118 ,  2.42311   ],
       [-0.7926605 ,  0.98790526],
       [ 1.112393  , -1.5376352 ],
       [-0.67663175,  0.74680907],
       [ 1.3252019 , -1.9291643 ],
       [-1.8644506 ,  2.32

In [ ]:
import shutil
output_dir = "results"
# Zip the output directory to make it easier to download
shutil.make_archive('RESULTS_PROJECT', 'zip', output_dir)


'C:\\Users\\User\\RESULTS_PROJECT.zip'

In [ ]:
test_data

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 281
})

In [ ]:
test_data['text']

['UNTe is Matlockite structured and crystallizes in the tetragonal P4/nmm space group. The structure is two-dimensional and consists of one UNTe sheet oriented in the (0, 0, 1) direction. U⁵⁺ is bonded in a 4-coordinate geometry to four equivalent N³⁻ and four equivalent Te²⁻ atoms. All U-N bond lengths are 2.26 Å. All U-Te bond lengths are 3.24 Å. N³⁻ is bonded to four equivalent U⁵⁺ atoms to form a mixture of corner and edge-sharing NU₄ tetrahedra. Te²⁻ is bonded in a 4-coordinate geometry to four equivalent U⁵⁺ atoms.',
 'ScZrB₂₄ crystallizes in the trigonal R̅3m space group. Sc²⁺ is bonded in a 24-coordinate geometry to twenty-four B+0.25- atoms. All Sc-B bond lengths are 2.75 Å. Zr⁴⁺ is bonded in a 24-coordinate geometry to twenty-four B+0.25- atoms. All Zr-B bond lengths are 2.75 Å. There are three inequivalent B+0.25- sites. In the first B+0.25- site, B+0.25- is bonded in a 7-coordinate geometry to one Sc²⁺, one Zr⁴⁺, and five B+0.25- atoms. There are a spread of B-B bond distan

In [ ]:
test_data['label']

[0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,


In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from sklearn.metrics import f1_score, matthews_corrcoef

# Load fine-tuned model and tokenizer from the saved path

tokenizer = AutoTokenizer.from_pretrained("results")
model = AutoModelForSequenceClassification.from_pretrained("results")
# Move model to GPU (if available)
model.eval()  # Set model to evaluation mode

# Function to make predictions
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=1).cpu().numpy()[0]  # Convert logits to probabilities
    predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class, probabilities

# Explicit text input (update with actual text samples)
texts = ['UNTe is Matlockite structured and crystallizes in the tetragonal P4/nmm space group. The structure is two-dimensional and consists of one UNTe sheet oriented in the (0, 0, 1) direction. U⁵⁺ is bonded in a 4-coordinate geometry to four equivalent N³⁻ and four equivalent Te²⁻ atoms. All U-N bond lengths are 2.26 Å. All U-Te bond lengths are 3.24 Å. N³⁻ is bonded to four equivalent U⁵⁺ atoms to form a mixture of corner and edge-sharing NU₄ tetrahedra. Te²⁻ is bonded in a 4-coordinate geometry to four equivalent U⁵⁺ atoms.',
 'ScZrB₂₄ crystallizes in the trigonal R̅3m space group. Sc²⁺ is bonded in a 24-coordinate geometry to twenty-four B+0.25- atoms. All Sc-B bond lengths are 2.75 Å. Zr⁴⁺ is bonded in a 24-coordinate geometry to twenty-four B+0.25- atoms. All Zr-B bond lengths are 2.75 Å. There are three inequivalent B+0.25- sites. In the first B+0.25- site, B+0.25- is bonded in a 7-coordinate geometry to one Sc²⁺, one Zr⁴⁺, and five B+0.25- atoms. There are a spread of B-B bond distances ranging from 1.69-1.78 Å. In the second B+0.25- site, B+0.25- is bonded in a 7-coordinate geometry to two equivalent Sc²⁺ and five B+0.25- atoms. There is one shorter (1.69 Å) and two longer (1.77 Å) B-B bond lengths. In the third B+0.25- site, B+0.25- is bonded in a 7-coordinate geometry to two equivalent Zr⁴⁺ and five B+0.25- atoms. There is one shorter (1.69 Å) and two longer (1.78 Å) B-B bond lengths.',
 'MgPd is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Mg is bonded in a body-centered cubic geometry to eight equivalent Pd atoms. All Mg-Pd bond lengths are 2.73 Å. Pd is bonded in a body-centered cubic geometry to eight equivalent Mg atoms.',
 'Ni₃N is Upper Bainite structured and crystallizes in the hexagonal P6_322 space group. Ni¹⁺ is bonded in a bent 120 degrees geometry to two equivalent N³⁻ atoms. Both Ni-N bond lengths are 1.87 Å. N³⁻ is bonded to six equivalent Ni¹⁺ atoms to form corner-sharing NNi₆ octahedra. The corner-sharing octahedral tilt angles are 50°.',
 'CdSnF₆ is Upper Bainite-derived structured and crystallizes in the trigonal R̅3 space group. Cd²⁺ is bonded to six equivalent F¹⁻ atoms to form CdF₆ octahedra that share corners with six equivalent SnF₆ octahedra. The corner-sharing octahedral tilt angles are 43°. All Cd-F bond lengths are 2.23 Å. Sn⁴⁺ is bonded to six equivalent F¹⁻ atoms to form SnF₆ octahedra that share corners with six equivalent CdF₆ octahedra. The corner-sharing octahedral tilt angles are 43°. All Sn-F bond lengths are 1.98 Å. F¹⁻ is bonded in a distorted bent 150 degrees geometry to one Cd²⁺ and one Sn⁴⁺ atom.',
 'Ag₂Te crystallizes in the monoclinic P2₁/c space group. There are two inequivalent Ag¹⁺ sites. In the first Ag¹⁺ site, Ag¹⁺ is bonded in a 10-coordinate geometry to six Ag¹⁺ and four equivalent Te²⁻ atoms. There are a spread of Ag-Ag bond distances ranging from 2.84-3.16 Å. There are a spread of Ag-Te bond distances ranging from 2.86-3.21 Å. In the second Ag¹⁺ site, Ag¹⁺ is bonded in a 8-coordinate geometry to four Ag¹⁺ and four equivalent Te²⁻ atoms. The Ag-Ag bond length is 2.92 Å. There are a spread of Ag-Te bond distances ranging from 2.88-3.02 Å. Te²⁻ is bonded in a 8-coordinate geometry to eight Ag¹⁺ atoms.',
 'EuCo₂Si₂ crystallizes in the tetragonal I4/mmm space group. Eu²⁺ is bonded in a body-centered cubic geometry to eight equivalent Si⁴⁻ atoms. All Eu-Si bond lengths are 3.09 Å. Co³⁺ is bonded to four equivalent Si⁴⁻ atoms to form a mixture of corner and edge-sharing CoSi₄ tetrahedra. All Co-Si bond lengths are 2.26 Å. Si⁴⁻ is bonded in a 9-coordinate geometry to four equivalent Eu²⁺, four equivalent Co³⁺, and one Si⁴⁻ atom. The Si-Si bond length is 2.79 Å.',
 'Pt₃Ga is Uranium Silicide structured and crystallizes in the cubic Pm̅3m space group. Pt is bonded to eight equivalent Pt and four equivalent Ga atoms to form distorted PtGa₄Pt₈ cuboctahedra that share corners with twelve equivalent PtGa₄Pt₈ cuboctahedra, edges with eight equivalent GaPt₁₂ cuboctahedra, edges with sixteen equivalent PtGa₄Pt₈ cuboctahedra, faces with four equivalent GaPt₁₂ cuboctahedra, and faces with fourteen equivalent PtGa₄Pt₈ cuboctahedra. All Pt-Pt bond lengths are 2.76 Å. All Pt-Ga bond lengths are 2.76 Å. Ga is bonded to twelve equivalent Pt atoms to form GaPt₁₂ cuboctahedra that share corners with twelve equivalent GaPt₁₂ cuboctahedra, edges with twenty-four equivalent PtGa₄Pt₈ cuboctahedra, faces with six equivalent GaPt₁₂ cuboctahedra, and faces with twelve equivalent PtGa₄Pt₈ cuboctahedra.',
 'LaAu₃ crystallizes in the hexagonal P6₃/mmc space group. La is bonded to twelve equivalent Au atoms to form a mixture of distorted corner and face-sharing LaAu₁₂ cuboctahedra. There are six shorter (3.16 Å) and six longer (3.35 Å) La-Au bond lengths. Au is bonded in a 10-coordinate geometry to four equivalent La and six equivalent Au atoms. There are four shorter (2.93 Å) and two longer (3.02 Å) Au-Au bond lengths.',
 'Ca₂As crystallizes in the tetragonal I4/mmm space group. There are two inequivalent Ca sites. In the first Ca site, Ca is bonded in a distorted square co-planar geometry to four equivalent As atoms. All Ca-As bond lengths are 3.12 Å. In the second Ca site, Ca is bonded in a 5-coordinate geometry to five equivalent As atoms. There is one shorter (2.95 Å) and four longer (3.30 Å) Ca-As bond lengths. As is bonded in a 9-coordinate geometry to nine Ca atoms.',
 'Re₂Y is Hexagonal Laves structured and crystallizes in the hexagonal P6₃/mmc space group. Y is bonded in a 12-coordinate geometry to twelve Re atoms. There are three shorter (3.15 Å) and nine longer (3.17 Å) Y-Re bond lengths. There are two inequivalent Re sites. In the first Re site, Re is bonded to six equivalent Y and six equivalent Re atoms to form a mixture of corner, edge, and face-sharing ReY₆Re₆ cuboctahedra. All Re-Re bond lengths are 2.73 Å. In the second Re site, Re is bonded to six equivalent Y and six Re atoms to form a mixture of corner, edge, and face-sharing ReY₆Re₆ cuboctahedra. There are two shorter (2.64 Å) and two longer (2.77 Å) Re-Re bond lengths.',
 'C is diamond-like structured and crystallizes in the trigonal R̅3m space group. There are four inequivalent C sites. In the first C site, C is bonded to four C atoms to form corner-sharing CC₄ tetrahedra. There are three shorter (1.54 Å) and one longer (1.57 Å) C-C bond length. In the second C site, C is bonded to four C atoms to form corner-sharing CC₄ tetrahedra. All C-C bond lengths are 1.54 Å. In the third C site, C is bonded to four C atoms to form corner-sharing CC₄ tetrahedra. The C-C bond length is 1.54 Å. In the fourth C site, C is bonded to four C atoms to form corner-sharing CC₄ tetrahedra.',
 'N₂ is gamma nitrogen structured and crystallizes in the tetragonal P4₂/mnm space group. The structure is zero-dimensional and consists of two N₂ clusters. N is bonded in a single-bond geometry to one N atom. The N-N bond length is 1.11 Å.',
 'N₂ is Magnesium structured and crystallizes in the hexagonal P6₃/mmc space group. The structure is zero-dimensional and consists of two N₂ clusters. N is bonded in a 1-coordinate geometry to  atoms.',
 'SbNSr₃ is (Cubic) Perovskite structured and crystallizes in the cubic Pm̅3m space group. Sr²⁺ is bonded in a linear geometry to four equivalent Sb³⁻ and two equivalent N³⁻ atoms. All Sr-Sb bond lengths are 3.67 Å. Both Sr-N bond lengths are 2.60 Å. Sb³⁻ is bonded to twelve equivalent Sr²⁺ atoms to form SbSr₁₂ cuboctahedra that share corners with twelve equivalent SbSr₁₂ cuboctahedra, faces with six equivalent SbSr₁₂ cuboctahedra, and faces with eight equivalent NSr₆ octahedra. N³⁻ is bonded to six equivalent Sr²⁺ atoms to form NSr₆ octahedra that share corners with six equivalent NSr₆ octahedra and faces with eight equivalent SbSr₁₂ cuboctahedra. The corner-sharing octahedra are not tilted.',
 'PrN is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Pr³⁺ is bonded to four equivalent N³⁻ atoms to form corner-sharing PrN₄ tetrahedra. All Pr-N bond lengths are 2.41 Å. N³⁻ is bonded to four equivalent Pr³⁺ atoms to form corner-sharing NPr₄ tetrahedra.',
 'Pd₃Mn is Uranium Silicide-like structured and crystallizes in the tetragonal I4/mmm space group. Mn is bonded to twelve Pd atoms to form MnPd₁₂ cuboctahedra that share corners with four equivalent PdMn₄Pd₈ cuboctahedra, corners with eight equivalent MnPd₁₂ cuboctahedra, edges with four equivalent MnPd₁₂ cuboctahedra, edges with twenty PdMn₄Pd₈ cuboctahedra, faces with five equivalent MnPd₁₂ cuboctahedra, and faces with thirteen PdMn₄Pd₈ cuboctahedra. There are a spread of Mn-Pd bond distances ranging from 2.75-2.78 Å. There are five inequivalent Pd sites. In the first Pd site, Pd is bonded to four equivalent Mn and eight Pd atoms to form distorted PdMn₄Pd₈ cuboctahedra that share corners with four equivalent MnPd₁₂ cuboctahedra, corners with eight equivalent PdMn₄Pd₈ cuboctahedra, edges with four equivalent MnPd₁₂ cuboctahedra, edges with twenty PdMn₄Pd₈ cuboctahedra, faces with five equivalent MnPd₁₂ cuboctahedra, and faces with thirteen PdMn₄Pd₈ cuboctahedra. There are four shorter (2.76 Å) and four longer (2.77 Å) Pd-Pd bond lengths. In the second Pd site, Pd is bonded to four equivalent Mn and eight Pd atoms to form distorted PdMn₄Pd₈ cuboctahedra that share corners with twelve PdMn₄Pd₈ cuboctahedra, edges with eight equivalent MnPd₁₂ cuboctahedra, edges with sixteen PdMn₄Pd₈ cuboctahedra, faces with four equivalent MnPd₁₂ cuboctahedra, and faces with fourteen PdMn₄Pd₈ cuboctahedra. All Pd-Mn bond lengths are 2.75 Å. All Pd-Pd bond lengths are 2.77 Å. In the third Pd site, Pd is bonded to four equivalent Mn and eight Pd atoms to form distorted PdMn₄Pd₈ cuboctahedra that share corners with twelve PdMn₄Pd₈ cuboctahedra, edges with eight equivalent MnPd₁₂ cuboctahedra, edges with sixteen PdMn₄Pd₈ cuboctahedra, faces with four equivalent MnPd₁₂ cuboctahedra, and faces with fourteen PdMn₄Pd₈ cuboctahedra. All Pd-Pd bond lengths are 2.77 Å. In the fourth Pd site, Pd is bonded to four equivalent Mn and eight Pd atoms to form distorted PdMn₄Pd₈ cuboctahedra that share corners with twelve PdMn₄Pd₈ cuboctahedra, edges with eight equivalent MnPd₁₂ cuboctahedra, edges with sixteen PdMn₄Pd₈ cuboctahedra, faces with four equivalent MnPd₁₂ cuboctahedra, and faces with fourteen PdMn₄Pd₈ cuboctahedra. All Pd-Pd bond lengths are 2.77 Å. In the fifth Pd site, Pd is bonded to four equivalent Mn and eight Pd atoms to form distorted PdMn₄Pd₈ cuboctahedra that share corners with twelve PdMn₄Pd₈ cuboctahedra, edges with eight equivalent MnPd₁₂ cuboctahedra, edges with sixteen PdMn₄Pd₈ cuboctahedra, faces with four equivalent MnPd₁₂ cuboctahedra, and faces with fourteen PdMn₄Pd₈ cuboctahedra. All Pd-Pd bond lengths are 2.77 Å.',
 'Pa is Protactinium structured and crystallizes in the tetragonal I4/mmm space group. Pa is bonded in a distorted q6 geometry to ten equivalent Pa atoms. There are two shorter (3.17 Å) and eight longer (3.20 Å) Pa-Pa bond lengths.',
 'GeO₂ is low (alpha) Cristobalite structured and crystallizes in the tetragonal P4_12_12 space group. Ge⁴⁺ is bonded to four equivalent O²⁻ atoms to form corner-sharing GeO₄ tetrahedra. All Ge-O bond lengths are 1.75 Å. O²⁻ is bonded in a bent 120 degrees geometry to two equivalent Ge⁴⁺ atoms.',
 'FeCl is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Fe¹⁺ is bonded in a body-centered cubic geometry to eight equivalent Cl¹⁻ atoms. All Fe-Cl bond lengths are 2.69 Å. Cl¹⁻ is bonded in a body-centered cubic geometry to eight equivalent Fe¹⁺ atoms.',
 'WSe₂ is Molybdenite-like structured and crystallizes in the trigonal P̅3m1 space group. The structure is two-dimensional and consists of four WSe₂ sheets oriented in the (0, 0, 1) direction. W⁴⁺ is bonded to six equivalent Se²⁻ atoms to form distorted edge-sharing WSe₆ pentagonal pyramids. All W-Se bond lengths are 2.54 Å. Se²⁻ is bonded in a 3-coordinate geometry to three equivalent W⁴⁺ atoms.',
 'CoI₂ is trigonal omega structured and crystallizes in the trigonal P̅3m1 space group. The structure is two-dimensional and consists of one CoI₂ sheet oriented in the (0, 0, 1) direction. Co²⁺ is bonded to six equivalent I¹⁻ atoms to form edge-sharing CoI₆ octahedra. All Co-I bond lengths are 2.69 Å. I¹⁻ is bonded in a distorted T-shaped geometry to three equivalent Co²⁺ atoms.',
 'Pt is Copper structured and crystallizes in the cubic Fm̅3m space group. Pt is bonded to twelve equivalent Pt atoms to form a mixture of corner, edge, and face-sharing PtPt₁₂ cuboctahedra. All Pt-Pt bond lengths are 2.79 Å.',
 'WN₂ is Fluorite structured and crystallizes in the trigonal R̅3m space group. W⁶⁺ is bonded in a body-centered cubic geometry to eight equivalent N³⁻ atoms. There are six shorter (2.11 Å) and two longer (2.26 Å) W-N bond lengths. N³⁻ is bonded to four equivalent W⁶⁺ atoms to form a mixture of distorted corner and edge-sharing NW₄ tetrahedra.',
 'As₂Ge₂Te₅ is MAX Phase-like structured and crystallizes in the trigonal P̅3m1 space group. The structure is two-dimensional and consists of one As₂Ge₂Te₅ sheet oriented in the (0, 0, 1) direction. Ge⁴⁺ is bonded to six Te²⁻ atoms to form GeTe₆ octahedra that share corners with three equivalent GeTe₆ octahedra, corners with three equivalent AsTe₆ octahedra, edges with three equivalent AsTe₆ octahedra, and edges with nine equivalent GeTe₆ octahedra. The corner-sharing octahedral tilt angles range from 0-2°. There are three shorter (2.94 Å) and three longer (2.98 Å) Ge-Te bond lengths. As¹⁺ is bonded to six Te²⁻ atoms to form AsTe₆ octahedra that share corners with three equivalent GeTe₆ octahedra, edges with three equivalent GeTe₆ octahedra, and edges with six equivalent AsTe₆ octahedra. The corner-sharing octahedral tilt angles are 2°. There are three shorter (2.84 Å) and three longer (3.03 Å) As-Te bond lengths. There are three inequivalent Te²⁻ sites. In the first Te²⁻ site, Te²⁻ is bonded to three equivalent Ge⁴⁺ and three equivalent As¹⁺ atoms to form TeGe₃As₃ octahedra that share corners with three equivalent TeGe₆ octahedra and edges with nine TeGe₃As₃ octahedra. The corner-sharing octahedral tilt angles are 1°. In the second Te²⁻ site, Te²⁻ is bonded in a 3-coordinate geometry to three equivalent As¹⁺ atoms. In the third Te²⁻ site, Te²⁻ is bonded to six equivalent Ge⁴⁺ atoms to form a mixture of corner and edge-sharing TeGe₆ octahedra. The corner-sharing octahedral tilt angles are 1°.',
 'TaN crystallizes in the hexagonal P6/mmm space group. Ta³⁺ is bonded to six N³⁻ atoms to form a mixture of corner, edge, and face-sharing TaN₆ octahedra. The corner-sharing octahedral tilt angles range from 0-60°. There are four shorter (2.20 Å) and two longer (2.68 Å) Ta-N bond lengths. There are two inequivalent N³⁻ sites. In the first N³⁻ site, N³⁻ is bonded to six equivalent Ta³⁺ atoms to form a mixture of distorted corner, edge, and face-sharing NTa₆ pentagonal pyramids. In the second N³⁻ site, N³⁻ is bonded in a hexagonal planar geometry to six equivalent Ta³⁺ atoms.',
 'YN is Halite, Rock Salt structured and crystallizes in the cubic Fm̅3m space group. Y³⁺ is bonded to six equivalent N³⁻ atoms to form a mixture of corner and edge-sharing YN₆ octahedra. The corner-sharing octahedra are not tilted. All Y-N bond lengths are 2.45 Å. N³⁻ is bonded to six equivalent Y³⁺ atoms to form a mixture of corner and edge-sharing NY₆ octahedra. The corner-sharing octahedra are not tilted.',
 'Zr is Magnesium structured and crystallizes in the hexagonal P6₃/mmc space group. Zr is bonded to twelve equivalent Zr atoms to form a mixture of corner, edge, and face-sharing ZrZr₁₂ cuboctahedra. There are six shorter (3.19 Å) and six longer (3.24 Å) Zr-Zr bond lengths.',
 'RbAu₅ crystallizes in the hexagonal P6/mmm space group. Rb is bonded in a 18-coordinate geometry to eighteen Au atoms. There are six shorter (3.36 Å) and twelve longer (3.68 Å) Rb-Au bond lengths. There are two inequivalent Au sites. In the first Au site, Au is bonded in a 9-coordinate geometry to three equivalent Rb and six equivalent Au atoms. All Au-Au bond lengths are 2.81 Å. In the second Au site, Au is bonded to four equivalent Rb and eight Au atoms to form a mixture of corner, edge, and face-sharing AuRb₄Au₈ cuboctahedra. All Au-Au bond lengths are 2.91 Å.',
 'O₂ is diamond structured and crystallizes in the cubic Fd̅3m space group. O is bonded to four equivalent O atoms to form corner-sharing OO₄ tetrahedra. All O-O bond lengths are 1.88 Å.',
 'ReC₂ crystallizes in the tetragonal I4/mmm space group. Re⁴⁺ is bonded in a distorted q6 geometry to ten equivalent C²⁻ atoms. There are two shorter (2.25 Å) and eight longer (2.26 Å) Re-C bond lengths. C²⁻ is bonded in a 5-coordinate geometry to five equivalent Re⁴⁺ atoms.',
 'IrAl₃ is Sodium arsenide structured and crystallizes in the hexagonal P6₃/mmc space group. Ir is bonded in a 11-coordinate geometry to eleven Al atoms. There are a spread of Ir-Al bond distances ranging from 2.47-2.79 Å. There are two inequivalent Al sites. In the first Al site, Al is bonded in a trigonal planar geometry to three equivalent Ir and six equivalent Al atoms. All Al-Al bond lengths are 2.79 Å. In the second Al site, Al is bonded in a 11-coordinate geometry to four equivalent Ir and seven Al atoms. There is one shorter (2.63 Å) and three longer (2.76 Å) Al-Al bond lengths.',
 'HfO₂ crystallizes in the tetragonal P4₂/nmc space group. Hf⁴⁺ is bonded in a distorted body-centered cubic geometry to eight equivalent O²⁻ atoms. There are four shorter (2.05 Å) and four longer (2.37 Å) Hf-O bond lengths. O²⁻ is bonded to four equivalent Hf⁴⁺ atoms to form a mixture of distorted corner and edge-sharing OHf₄ tetrahedra.',
 'NbRh is Tetraauricupride structured and crystallizes in the tetragonal P4/mmm space group. Nb is bonded to eight equivalent Rh atoms to form a mixture of distorted corner, edge, and face-sharing NbRh₈ cuboctahedra. All Nb-Rh bond lengths are 2.80 Å. Rh is bonded to eight equivalent Nb atoms to form a mixture of distorted corner, edge, and face-sharing RhNb₈ cuboctahedra.',
 'ErB₄ crystallizes in the tetragonal P4/mbm space group. Er is bonded in a 18-coordinate geometry to eighteen B atoms. There are a spread of Er-B bond distances ranging from 2.71-3.05 Å. There are three inequivalent B sites. In the first B site, B is bonded in a 9-coordinate geometry to four equivalent Er and five B atoms. There is one shorter (1.62 Å) and four longer (1.74 Å) B-B bond lengths. In the second B site, B is bonded in a 9-coordinate geometry to four equivalent Er and five B atoms. There is one shorter (1.71 Å) and two longer (1.80 Å) B-B bond lengths. In the third B site, B is bonded in a distorted trigonal planar geometry to six equivalent Er and three B atoms. The B-B bond length is 1.74 Å.',
 'OsN₂ is Corundum-like structured and crystallizes in the hexagonal P6₃/mmc space group. Os⁴⁺ is bonded in a 6-coordinate geometry to six equivalent N²⁻ atoms. All Os-N bond lengths are 2.13 Å. N²⁻ is bonded in a distorted rectangular see-saw-like geometry to three equivalent Os⁴⁺ and one N²⁻ atom. The N-N bond length is 1.31 Å.',
 'LiBC crystallizes in the hexagonal P6₃/mmc space group. Li¹⁺ is bonded in a 6-coordinate geometry to six equivalent C⁴⁻ atoms. All Li-C bond lengths are 2.35 Å. B³⁺ is bonded in a trigonal planar geometry to three equivalent C⁴⁻ atoms. All B-C bond lengths are 1.58 Å. C⁴⁻ is bonded in a distorted trigonal planar geometry to six equivalent Li¹⁺ and three equivalent B³⁺ atoms.',
 'SiSb₃ crystallizes in the hexagonal P6₃/mmc space group. Si⁴⁺ is bonded in a 12-coordinate geometry to twelve equivalent Sb+1.33- atoms. There are six shorter (3.23 Å) and six longer (3.30 Å) Si-Sb bond lengths. Sb+1.33- is bonded in a 12-coordinate geometry to four equivalent Si⁴⁺ and eight equivalent Sb+1.33- atoms. There are a spread of Sb-Sb bond distances ranging from 3.07-3.40 Å.',
 'LiYSi crystallizes in the hexagonal P̅62m space group. Li¹⁺ is bonded to four Si⁴⁻ atoms to form LiSi₄ tetrahedra that share corners with six equivalent YSi₅ square pyramids, corners with ten equivalent LiSi₄ tetrahedra, edges with six equivalent YSi₅ square pyramids, and edges with two equivalent LiSi₄ tetrahedra. There are two shorter (2.67 Å) and two longer (2.77 Å) Li-Si bond lengths. Y³⁺ is bonded to five Si⁴⁻ atoms to form YSi₅ square pyramids that share corners with ten equivalent YSi₅ square pyramids, corners with six equivalent LiSi₄ tetrahedra, edges with six equivalent YSi₅ square pyramids, and edges with six equivalent LiSi₄ tetrahedra. There are four shorter (2.97 Å) and one longer (2.99 Å) Y-Si bond length. There are two inequivalent Si⁴⁻ sites. In the first Si⁴⁻ site, Si⁴⁻ is bonded in a 9-coordinate geometry to six equivalent Li¹⁺ and three equivalent Y³⁺ atoms. In the second Si⁴⁻ site, Si⁴⁻ is bonded in a 9-coordinate geometry to three equivalent Li¹⁺ and six equivalent Y³⁺ atoms.',
 'YbSi₂ is hexagonal omega structure structured and crystallizes in the hexagonal P6/mmm space group. Yb is bonded to twelve equivalent Si atoms to form a mixture of edge and face-sharing YbSi₁₂ cuboctahedra. All Yb-Si bond lengths are 3.02 Å. Si is bonded in a 9-coordinate geometry to six equivalent Yb and three equivalent Si atoms. All Si-Si bond lengths are 2.35 Å.',
 'ThRh₂ crystallizes in the hexagonal P6₃/mmc space group. Th is bonded in a 11-coordinate geometry to eleven Rh atoms. There are a spread of Th-Rh bond distances ranging from 2.72-3.10 Å. There are two inequivalent Rh sites. In the first Rh site, Rh is bonded to five equivalent Th and six equivalent Rh atoms to form a mixture of corner and face-sharing RhTh₅Rh₆ trigonal bipyramids. All Rh-Rh bond lengths are 3.10 Å. In the second Rh site, Rh is bonded in a 14-coordinate geometry to six equivalent Th and eight Rh atoms. Both Rh-Rh bond lengths are 2.96 Å.',
 'CeB₂C₂ crystallizes in the tetragonal P4/mbm space group. Ce⁴⁺ is bonded in a body-centered cubic geometry to eight equivalent C⁴⁻ atoms. All Ce-C bond lengths are 2.77 Å. B²⁺ is bonded in a distorted trigonal planar geometry to three equivalent C⁴⁻ atoms. There is one shorter (1.53 Å) and two longer (1.61 Å) B-C bond lengths. C⁴⁻ is bonded in a 3-coordinate geometry to four equivalent Ce⁴⁺ and three equivalent B²⁺ atoms.',
 'Au₂Mn is Titanium Disilicide-like structured and crystallizes in the tetragonal I4/mmm space group. Mn is bonded in a distorted q6 geometry to ten equivalent Au atoms. There are eight shorter (2.78 Å) and two longer (2.97 Å) Mn-Au bond lengths. Au is bonded in a 5-coordinate geometry to five equivalent Mn atoms.',
 'ZrI₂ is Molybdenite-like structured and crystallizes in the monoclinic P2₁/m space group. The structure is two-dimensional and consists of two ZrI₂ sheets oriented in the (0, 0, 1) direction. In one of the ZrI₂ sheets, Zr²⁺ is bonded to six I¹⁻ atoms to form distorted edge-sharing ZrI₆ octahedra. There are a spread of Zr-I bond distances ranging from 2.91-3.15 Å. There are two inequivalent I¹⁻ sites. In the first I¹⁻ site, I¹⁻ is bonded in a 3-coordinate geometry to three equivalent Zr²⁺ atoms. In the second I¹⁻ site, I¹⁻ is bonded in a 3-coordinate geometry to three equivalent Zr²⁺ atoms. In one of the ZrI₂ sheets, Zr²⁺ is bonded to six I¹⁻ atoms to form distorted edge-sharing ZrI₆ octahedra. There are a spread of Zr-I bond distances ranging from 2.91-3.15 Å. There are two inequivalent I¹⁻ sites. In the first I¹⁻ site, I¹⁻ is bonded in a 3-coordinate geometry to three equivalent Zr²⁺ atoms. In the second I¹⁻ site, I¹⁻ is bonded in a 3-coordinate geometry to three equivalent Zr²⁺ atoms.',
 'Se is alpha Selenium structured and crystallizes in the trigonal P3_121 space group. The structure is one-dimensional and consists of one Se ribbon oriented in the (0, 0, 1) direction. Se is bonded in a water-like geometry to two equivalent Se atoms. Both Se-Se bond lengths are 2.38 Å.',
 'LaTl is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. La is bonded in a body-centered cubic geometry to eight equivalent Tl atoms. All La-Tl bond lengths are 3.44 Å. Tl is bonded in a body-centered cubic geometry to eight equivalent La atoms.',
 'PtAg₃ is Uranium Silicide structured and crystallizes in the cubic Pm̅3m space group. Pt is bonded to twelve equivalent Ag atoms to form PtAg₁₂ cuboctahedra that share corners with twelve equivalent PtAg₁₂ cuboctahedra, edges with twenty-four equivalent AgAg₈Pt₄ cuboctahedra, faces with six equivalent PtAg₁₂ cuboctahedra, and faces with twelve equivalent AgAg₈Pt₄ cuboctahedra. All Pt-Ag bond lengths are 2.87 Å. Ag is bonded to four equivalent Pt and eight equivalent Ag atoms to form AgAg₈Pt₄ cuboctahedra that share corners with twelve equivalent AgAg₈Pt₄ cuboctahedra, edges with eight equivalent PtAg₁₂ cuboctahedra, edges with sixteen equivalent AgAg₈Pt₄ cuboctahedra, faces with four equivalent PtAg₁₂ cuboctahedra, and faces with fourteen equivalent AgAg₈Pt₄ cuboctahedra. All Ag-Ag bond lengths are 2.87 Å.',
 'GdBr₃ is Aluminum trichloride structured and crystallizes in the monoclinic C2/m space group. The structure is two-dimensional and consists of one GdBr₃ sheet oriented in the (0, 0, 1) direction. Gd³⁺ is bonded to six Br¹⁻ atoms to form edge-sharing GdBr₆ octahedra. All Gd-Br bond lengths are 2.84 Å. There are two inequivalent Br¹⁻ sites. In the first Br¹⁻ site, Br¹⁻ is bonded in an L-shaped geometry to two equivalent Gd³⁺ atoms. In the second Br¹⁻ site, Br¹⁻ is bonded in an L-shaped geometry to two equivalent Gd³⁺ atoms.',
 'MnAu is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Mn is bonded in a body-centered cubic geometry to eight equivalent Au atoms. All Mn-Au bond lengths are 2.78 Å. Au is bonded in a body-centered cubic geometry to eight equivalent Mn atoms.',
 'Be₁₂W crystallizes in the monoclinic C2/m space group. There are six inequivalent Be sites. In the first Be site, Be is bonded in a 10-coordinate geometry to nine Be and one W atom. There are a spread of Be-Be bond distances ranging from 2.16-2.36 Å. The Be-W bond length is 2.52 Å. In the second Be site, Be is bonded to ten Be and two equivalent W atoms to form a mixture of corner, edge, and face-sharing BeBe₁₀W₂ cuboctahedra. There are a spread of Be-Be bond distances ranging from 2.10-2.20 Å. Both Be-W bond lengths are 2.76 Å. In the third Be site, Be is bonded in a 10-coordinate geometry to nine Be and one W atom. There are a spread of Be-Be bond distances ranging from 2.15-2.34 Å. The Be-W bond length is 2.53 Å. In the fourth Be site, Be is bonded in a 12-coordinate geometry to ten Be and two equivalent W atoms. There are two shorter (2.10 Å) and two longer (2.14 Å) Be-Be bond lengths. Both Be-W bond lengths are 2.60 Å. In the fifth Be site, Be is bonded to ten Be and two equivalent W atoms to form a mixture of corner, edge, and face-sharing BeBe₁₀W₂ cuboctahedra. Both Be-Be bond lengths are 2.11 Å. Both Be-W bond lengths are 2.75 Å. In the sixth Be site, Be is bonded in a 12-coordinate geometry to ten Be and two equivalent W atoms. Both Be-W bond lengths are 2.58 Å. W is bonded in a 20-coordinate geometry to twenty Be atoms.',
 'Cu₂O₃ is Hausmannite-like structured and crystallizes in the cubic Ia̅3 space group. There are two inequivalent Cu³⁺ sites. In the first Cu³⁺ site, Cu³⁺ is bonded to six equivalent O²⁻ atoms to form a mixture of corner and edge-sharing CuO₆ octahedra. The corner-sharing octahedral tilt angles are 48°. All Cu-O bond lengths are 1.95 Å. In the second Cu³⁺ site, Cu³⁺ is bonded to six equivalent O²⁻ atoms to form a mixture of distorted corner and edge-sharing CuO₆ octahedra. The corner-sharing octahedral tilt angles range from 48-61°. There are a spread of Cu-O bond distances ranging from 1.86-2.19 Å. O²⁻ is bonded in a distorted rectangular see-saw-like geometry to four Cu³⁺ atoms.',
 'LuCl₃ crystallizes in the hexagonal P6₃/mmc space group. The structure is one-dimensional and consists of two LuCl₃ ribbons oriented in the (0, 0, 1) direction. Lu³⁺ is bonded to six equivalent Cl¹⁻ atoms to form distorted face-sharing LuCl₆ pentagonal pyramids. All Lu-Cl bond lengths are 2.59 Å. Cl¹⁻ is bonded in an L-shaped geometry to two equivalent Lu³⁺ atoms.',
 'LaAg is Tetraauricupride structured and crystallizes in the tetragonal P4/mmm space group. La is bonded in a body-centered cubic geometry to eight equivalent Ag atoms. All La-Ag bond lengths are 3.30 Å. Ag is bonded in a body-centered cubic geometry to eight equivalent La atoms.',
 'YbHg₃ crystallizes in the hexagonal P6₃/mmc space group. Yb is bonded to twelve equivalent Hg atoms to form a mixture of corner and face-sharing YbHg₁₂ cuboctahedra. There are six shorter (3.09 Å) and six longer (3.27 Å) Yb-Hg bond lengths. Hg is bonded in a 4-coordinate geometry to four equivalent Yb atoms.',
 'PdSe₆(Br)₂ crystallizes in the triclinic P̅1 space group. The structure is one-dimensional and consists of two Br clusters and one PdSe₆ ribbon oriented in the (0, -1, 1) direction. In each Br cluster, Br⁵⁺ is bonded in a 1-coordinate geometry to  atoms. In the PdSe₆ ribbon, Pd²⁺ is bonded in a linear geometry to two equivalent Se²⁻ atoms. Both Pd-Se bond lengths are 2.44 Å. There are three inequivalent Se²⁻ sites. In the first Se²⁻ site, Se²⁻ is bonded in a distorted water-like geometry to two Se²⁻ atoms. There is one shorter (2.34 Å) and one longer (2.44 Å) Se-Se bond length. In the second Se²⁻ site, Se²⁻ is bonded in a 3-coordinate geometry to one Pd²⁺ and two Se²⁻ atoms. The Se-Se bond length is 2.43 Å. In the third Se²⁻ site, Se²⁻ is bonded in a distorted water-like geometry to two Se²⁻ atoms.',
 'Pt₃B crystallizes in the hexagonal P6₃/mmc space group. The structure is one-dimensional and consists of two Pt₃B ribbons oriented in the (0, 0, 1) direction. Pt is bonded in a distorted L-shaped geometry to two equivalent B atoms. Both Pt-B bond lengths are 2.18 Å. B is bonded to six equivalent Pt atoms to form distorted face-sharing BPt₆ pentagonal pyramids.',
 'TiCo₂ is Cubic Laves structured and crystallizes in the cubic Fd̅3m space group. Ti is bonded in a 16-coordinate geometry to four equivalent Ti and twelve equivalent Co atoms. All Ti-Ti bond lengths are 2.90 Å. All Ti-Co bond lengths are 2.77 Å. Co is bonded to six equivalent Ti and six equivalent Co atoms to form a mixture of corner, edge, and face-sharing CoTi₆Co₆ cuboctahedra. All Co-Co bond lengths are 2.36 Å.',
 'Ta₂Al is beta Plutonium-derived structured and crystallizes in the tetragonal P4₂/mnm space group. There are three inequivalent Ta sites. In the first Ta site, Ta is bonded in a 14-coordinate geometry to nine Ta and five Al atoms. There are a spread of Ta-Ta bond distances ranging from 2.69-3.30 Å. There are a spread of Ta-Al bond distances ranging from 2.97-3.13 Å. In the second Ta site, Ta is bonded in a 12-coordinate geometry to seven Ta and five Al atoms. There is one shorter (2.82 Å) and two longer (3.19 Å) Ta-Ta bond lengths. There are a spread of Ta-Al bond distances ranging from 2.68-2.90 Å. In the third Ta site, Ta is bonded in a 5-coordinate geometry to eleven Ta and four Al atoms. The Ta-Ta bond length is 2.72 Å. There are two shorter (2.72 Å) and two longer (3.03 Å) Ta-Al bond lengths. There are two inequivalent Al sites. In the first Al site, Al is bonded in a 10-coordinate geometry to nine Ta and one Al atom. The Al-Al bond length is 2.79 Å. In the second Al site, Al is bonded to twelve Ta atoms to form edge-sharing AlTa₁₂ cuboctahedra.',
 'NbCo₃ crystallizes in the hexagonal P6₃mc space group. There are three inequivalent Nb sites. In the first Nb site, Nb is bonded in a 16-coordinate geometry to three equivalent Nb and thirteen Co atoms. All Nb-Nb bond lengths are 2.99 Å. There are a spread of Nb-Co bond distances ranging from 2.67-2.97 Å. In the second Nb site, Nb is bonded in a 16-coordinate geometry to one Nb and fifteen Co atoms. The Nb-Nb bond length is 2.94 Å. There are a spread of Nb-Co bond distances ranging from 2.68-2.84 Å. In the third Nb site, Nb is bonded in a 16-coordinate geometry to four Nb and twelve Co atoms. There are a spread of Nb-Co bond distances ranging from 2.75-2.83 Å. There are five inequivalent Co sites. In the first Co site, Co is bonded to five Nb and seven Co atoms to form a mixture of corner, edge, and face-sharing CoNb₅Co₇ cuboctahedra. There are a spread of Co-Co bond distances ranging from 2.36-2.70 Å. In the second Co site, Co is bonded to four Nb and eight Co atoms to form a mixture of distorted corner, edge, and face-sharing CoNb₄Co₈ cuboctahedra. There are a spread of Co-Co bond distances ranging from 2.33-2.80 Å. In the third Co site, Co is bonded in a 16-coordinate geometry to four Nb and twelve Co atoms. All Co-Co bond lengths are 2.75 Å. In the fourth Co site, Co is bonded to three equivalent Nb and nine Co atoms to form CoNb₃Co₉ cuboctahedra that share corners with fifteen CoNb₅Co₇ cuboctahedra, edges with six equivalent CoNb₃Co₉ cuboctahedra, and faces with nineteen CoNb₅Co₇ cuboctahedra. In the fifth Co site, Co is bonded to six Nb and six Co atoms to form CoNb₆Co₆ cuboctahedra that share corners with fifteen CoNb₅Co₇ cuboctahedra, edges with six equivalent CoNb₆Co₆ cuboctahedra, and faces with nineteen CoNb₅Co₇ cuboctahedra.',
 'LaPt₃C is (Cubic) Perovskite structured and crystallizes in the cubic Pm̅3m space group. La is bonded to twelve equivalent Pt atoms to form LaPt₁₂ cuboctahedra that share corners with twelve equivalent LaPt₁₂ cuboctahedra, faces with six equivalent LaPt₁₂ cuboctahedra, and faces with eight equivalent CPt₆ octahedra. All La-Pt bond lengths are 3.12 Å. Pt is bonded to four equivalent La and two equivalent C atoms to form a mixture of distorted corner, edge, and face-sharing PtLa₄C₂ octahedra. The corner-sharing octahedral tilt angles range from 0-60°. Both Pt-C bond lengths are 2.20 Å. C is bonded to six equivalent Pt atoms to form CPt₆ octahedra that share corners with six equivalent CPt₆ octahedra and faces with eight equivalent LaPt₁₂ cuboctahedra. The corner-sharing octahedra are not tilted.',
 'RhCl₂ is Rutile structured and crystallizes in the tetragonal P4₂/mnm space group. Rh is bonded to six equivalent Cl atoms to form a mixture of corner and edge-sharing RhCl₆ octahedra. The corner-sharing octahedral tilt angles are 50°. There are two shorter (2.44 Å) and four longer (2.45 Å) Rh-Cl bond lengths. Cl is bonded in a distorted trigonal planar geometry to three equivalent Rh atoms.',
 'PtZn is Tetraauricupride structured and crystallizes in the tetragonal P4/mmm space group. Pt is bonded in a distorted body-centered cubic geometry to eight equivalent Zn atoms. All Pt-Zn bond lengths are 2.67 Å. Zn is bonded in a distorted body-centered cubic geometry to eight equivalent Pt atoms.',
 'Fe₃Ga is Uranium Silicide structured and crystallizes in the cubic Pm̅3m space group. Fe is bonded to eight equivalent Fe and four equivalent Ga atoms to form FeGa₄Fe₈ cuboctahedra that share corners with twelve equivalent FeGa₄Fe₈ cuboctahedra, edges with eight equivalent GaFe₁₂ cuboctahedra, edges with sixteen equivalent FeGa₄Fe₈ cuboctahedra, faces with four equivalent GaFe₁₂ cuboctahedra, and faces with fourteen equivalent FeGa₄Fe₈ cuboctahedra. All Fe-Fe bond lengths are 2.57 Å. All Fe-Ga bond lengths are 2.57 Å. Ga is bonded to twelve equivalent Fe atoms to form GaFe₁₂ cuboctahedra that share corners with twelve equivalent GaFe₁₂ cuboctahedra, edges with twenty-four equivalent FeGa₄Fe₈ cuboctahedra, faces with six equivalent GaFe₁₂ cuboctahedra, and faces with twelve equivalent FeGa₄Fe₈ cuboctahedra.',
 'Y₃PbC is (Cubic) Perovskite structured and crystallizes in the cubic Pm̅3m space group. Y is bonded in a distorted linear geometry to four equivalent Pb and two equivalent C atoms. All Y-Pb bond lengths are 3.47 Å. Both Y-C bond lengths are 2.45 Å. Pb is bonded to twelve equivalent Y atoms to form PbY₁₂ cuboctahedra that share corners with twelve equivalent PbY₁₂ cuboctahedra, faces with six equivalent PbY₁₂ cuboctahedra, and faces with eight equivalent CY₆ octahedra. C is bonded to six equivalent Y atoms to form CY₆ octahedra that share corners with six equivalent CY₆ octahedra and faces with eight equivalent PbY₁₂ cuboctahedra. The corner-sharing octahedra are not tilted.',
 'ScNb crystallizes in the hexagonal P̅6m2 space group. Sc is bonded to six equivalent Sc and six equivalent Nb atoms to form ScSc₆Nb₆ cuboctahedra that share corners with eighteen equivalent ScSc₆Nb₆ cuboctahedra, edges with six equivalent ScSc₆Nb₆ cuboctahedra, edges with twelve equivalent NbSc₆Nb₆ cuboctahedra, faces with eight equivalent ScSc₆Nb₆ cuboctahedra, and faces with twelve equivalent NbSc₆Nb₆ cuboctahedra. All Sc-Sc bond lengths are 3.07 Å. All Sc-Nb bond lengths are 3.09 Å. Nb is bonded to six equivalent Sc and six equivalent Nb atoms to form NbSc₆Nb₆ cuboctahedra that share corners with eighteen equivalent NbSc₆Nb₆ cuboctahedra, edges with six equivalent NbSc₆Nb₆ cuboctahedra, edges with twelve equivalent ScSc₆Nb₆ cuboctahedra, faces with eight equivalent NbSc₆Nb₆ cuboctahedra, and faces with twelve equivalent ScSc₆Nb₆ cuboctahedra. All Nb-Nb bond lengths are 3.07 Å.',
 'CoCl₂ is trigonal omega structured and crystallizes in the trigonal P̅3m1 space group. The structure is two-dimensional and consists of one CoCl₂ sheet oriented in the (0, 0, 1) direction. Co²⁺ is bonded to six equivalent Cl¹⁻ atoms to form edge-sharing CoCl₆ octahedra. All Co-Cl bond lengths are 2.44 Å. Cl¹⁻ is bonded in a distorted T-shaped geometry to three equivalent Co²⁺ atoms.',
 'Li₇Sn₂ crystallizes in the orthorhombic Cmmm space group. There are six inequivalent Li sites. In the first Li site, Li is bonded in a distorted body-centered cubic geometry to four equivalent Li and four equivalent Sn atoms. All Li-Li bond lengths are 2.84 Å. All Li-Sn bond lengths are 2.83 Å. In the second Li site, Li is bonded in a 10-coordinate geometry to eight Li and two equivalent Sn atoms. There are four shorter (2.68 Å) and four longer (2.80 Å) Li-Li bond lengths. Both Li-Sn bond lengths are 3.29 Å. In the third Li site, Li is bonded in a 4-coordinate geometry to four Li and four Sn atoms. Both Li-Li bond lengths are 2.65 Å. There are two shorter (2.93 Å) and two longer (2.94 Å) Li-Sn bond lengths. In the fourth Li site, Li is bonded in a 10-coordinate geometry to six Li and four Sn atoms. There are two shorter (2.76 Å) and four longer (2.99 Å) Li-Li bond lengths. There are two shorter (2.87 Å) and two longer (3.08 Å) Li-Sn bond lengths. In the fifth Li site, Li is bonded in a 11-coordinate geometry to seven Li and four Sn atoms. There are two shorter (2.80 Å) and one longer (2.83 Å) Li-Li bond length. There are a spread of Li-Sn bond distances ranging from 2.87-3.21 Å. In the sixth Li site, Li is bonded in a 3-coordinate geometry to four Li and three Sn atoms. There are two shorter (2.82 Å) and one longer (2.83 Å) Li-Sn bond length. There are two inequivalent Sn sites. In the first Sn site, Sn is bonded in a 8-coordinate geometry to thirteen Li atoms. In the second Sn site, Sn is bonded in a 12-coordinate geometry to twelve Li atoms.',
 'PbN is Wurtzite structured and crystallizes in the hexagonal P6₃mc space group. Pb²⁺ is bonded to four equivalent N²⁻ atoms to form corner-sharing PbN₄ tetrahedra. There are three shorter (2.36 Å) and one longer (2.40 Å) Pb-N bond length. N²⁻ is bonded to four equivalent Pb²⁺ atoms to form corner-sharing NPb₄ tetrahedra.',
 'BaSi₂ is hexagonal omega structure structured and crystallizes in the hexagonal P6/mmm space group. Ba²⁺ is bonded to twelve equivalent Si¹⁻ atoms to form a mixture of edge and face-sharing BaSi₁₂ cuboctahedra. All Ba-Si bond lengths are 3.50 Å. Si¹⁻ is bonded in a 9-coordinate geometry to six equivalent Ba²⁺ and three equivalent Si¹⁻ atoms. All Si-Si bond lengths are 2.38 Å.',
 'GaAs is High Pressure Cadmuum Telluride structured and crystallizes in the tetragonal P4/mmm space group. Ga¹⁺ is bonded in a linear geometry to two equivalent As¹⁻ atoms. Both Ga-As bond lengths are 2.73 Å. As¹⁻ is bonded to two equivalent Ga¹⁺ and four equivalent As¹⁻ atoms to form a mixture of corner and edge-sharing AsGa₂As₄ octahedra. The corner-sharing octahedra are not tilted. All As-As bond lengths are 2.72 Å.',
 'Au₃Cu is Uranium Silicide structured and crystallizes in the cubic Pm̅3m space group. Au is bonded to eight equivalent Au and four equivalent Cu atoms to form distorted AuCu₄Au₈ cuboctahedra that share corners with twelve equivalent AuCu₄Au₈ cuboctahedra, edges with eight equivalent CuAu₁₂ cuboctahedra, edges with sixteen equivalent AuCu₄Au₈ cuboctahedra, faces with four equivalent CuAu₁₂ cuboctahedra, and faces with fourteen equivalent AuCu₄Au₈ cuboctahedra. All Au-Au bond lengths are 2.84 Å. All Au-Cu bond lengths are 2.84 Å. Cu is bonded to twelve equivalent Au atoms to form CuAu₁₂ cuboctahedra that share corners with twelve equivalent CuAu₁₂ cuboctahedra, edges with twenty-four equivalent AuCu₄Au₈ cuboctahedra, faces with six equivalent CuAu₁₂ cuboctahedra, and faces with twelve equivalent AuCu₄Au₈ cuboctahedra.',
 'I₂O₅ crystallizes in the monoclinic P2₁/c space group. There are five inequivalent O²⁻ sites. In the first O²⁻ site, O²⁻ is bonded in a bent 150 degrees geometry to two I⁵⁺ atoms. There is one shorter (1.91 Å) and one longer (1.97 Å) O-I bond length. In the second O²⁻ site, O²⁻ is bonded in a distorted single-bond geometry to one I⁵⁺ atom. The O-I bond length is 1.78 Å. In the third O²⁻ site, O²⁻ is bonded in a 2-coordinate geometry to two equivalent I⁵⁺ atoms. There is one shorter (1.82 Å) and one longer (2.38 Å) O-I bond length. In the fourth O²⁻ site, O²⁻ is bonded in a distorted single-bond geometry to two I⁵⁺ atoms. There is one shorter (1.78 Å) and one longer (2.47 Å) O-I bond length. In the fifth O²⁻ site, O²⁻ is bonded in a distorted bent 120 degrees geometry to two equivalent I⁵⁺ atoms. There is one shorter (1.86 Å) and one longer (2.19 Å) O-I bond length. There are two inequivalent I⁵⁺ sites. In the first I⁵⁺ site, I⁵⁺ is bonded to five O²⁻ atoms to form distorted corner-sharing IO₅ square pyramids. In the second I⁵⁺ site, I⁵⁺ is bonded in a 3-coordinate geometry to four O²⁻ atoms.',
 'SiC is Moissanite-6H-like structured and crystallizes in the trigonal R3m space group. There are five inequivalent Si⁴⁺ sites. In the first Si⁴⁺ site, Si⁴⁺ is bonded to four C⁴⁻ atoms to form corner-sharing SiC₄ tetrahedra. There are three shorter (1.88 Å) and one longer (1.89 Å) Si-C bond length. In the second Si⁴⁺ site, Si⁴⁺ is bonded to four C⁴⁻ atoms to form corner-sharing SiC₄ tetrahedra. There are three shorter (1.88 Å) and one longer (1.89 Å) Si-C bond length. In the third Si⁴⁺ site, Si⁴⁺ is bonded to four C⁴⁻ atoms to form corner-sharing SiC₄ tetrahedra. All Si-C bond lengths are 1.89 Å. In the fourth Si⁴⁺ site, Si⁴⁺ is bonded to four C⁴⁻ atoms to form corner-sharing SiC₄ tetrahedra. All Si-C bond lengths are 1.89 Å. In the fifth Si⁴⁺ site, Si⁴⁺ is bonded to four C⁴⁻ atoms to form corner-sharing SiC₄ tetrahedra. There are three shorter (1.88 Å) and one longer (1.89 Å) Si-C bond length. There are five inequivalent C⁴⁻ sites. In the first C⁴⁻ site, C⁴⁻ is bonded to four Si⁴⁺ atoms to form corner-sharing CSi₄ tetrahedra. All C-Si bond lengths are 1.88 Å. In the second C⁴⁻ site, C⁴⁻ is bonded to four Si⁴⁺ atoms to form corner-sharing CSi₄ tetrahedra. In the third C⁴⁻ site, C⁴⁻ is bonded to four Si⁴⁺ atoms to form corner-sharing CSi₄ tetrahedra. All C-Si bond lengths are 1.88 Å. In the fourth C⁴⁻ site, C⁴⁻ is bonded to four Si⁴⁺ atoms to form corner-sharing CSi₄ tetrahedra. In the fifth C⁴⁻ site, C⁴⁻ is bonded to four Si⁴⁺ atoms to form corner-sharing CSi₄ tetrahedra.',
 'H₂ is Cubic alpha N2-like structured and crystallizes in the tetragonal I4/mmm space group. The structure is zero-dimensional and consists of two H₂ clusters. H is bonded in a single-bond geometry to one H atom. The H-H bond length is 0.74 Å.',
 'PmCu₃ crystallizes in the hexagonal P6₃/mmc space group. Pm is bonded in a 12-coordinate geometry to twelve equivalent Cu atoms. There are six shorter (3.02 Å) and six longer (3.10 Å) Pm-Cu bond lengths. Cu is bonded in a 10-coordinate geometry to four equivalent Pm and six equivalent Cu atoms. There are two shorter (2.46 Å) and four longer (2.56 Å) Cu-Cu bond lengths.',
 'Li₄SrP₂ crystallizes in the trigonal R̅3m space group. There are two inequivalent Li¹⁺ sites. In the first Li¹⁺ site, Li¹⁺ is bonded in a trigonal planar geometry to three equivalent P³⁻ atoms. All Li-P bond lengths are 2.48 Å. In the second Li¹⁺ site, Li¹⁺ is bonded to four equivalent P³⁻ atoms to form distorted LiP₄ tetrahedra that share corners with nine equivalent SrP₆ octahedra, corners with six equivalent LiP₄ tetrahedra, edges with three equivalent LiP₄ tetrahedra, and  a faceface with one SrP₆ octahedra. The corner-sharing octahedral tilt angles range from 50-51°. There is one shorter (2.53 Å) and three longer (2.73 Å) Li-P bond lengths. Sr²⁺ is bonded to six equivalent P³⁻ atoms to form SrP₆ octahedra that share corners with eighteen equivalent LiP₄ tetrahedra, edges with six equivalent SrP₆ octahedra, and faces with two equivalent LiP₄ tetrahedra. All Sr-P bond lengths are 3.22 Å. P³⁻ is bonded in a 10-coordinate geometry to seven Li¹⁺ and three equivalent Sr²⁺ atoms.',
 'Ti₃B₄ crystallizes in the orthorhombic Immm space group. There are two inequivalent Ti⁴⁺ sites. In the first Ti⁴⁺ site, Ti⁴⁺ is bonded in a 7-coordinate geometry to seven B³⁻ atoms. There are a spread of Ti-B bond distances ranging from 2.33-2.51 Å. In the second Ti⁴⁺ site, Ti⁴⁺ is bonded to twelve B³⁻ atoms to form a mixture of edge and face-sharing TiB₁₂ cuboctahedra. There are eight shorter (2.40 Å) and four longer (2.43 Å) Ti-B bond lengths. There are two inequivalent B³⁻ sites. In the first B³⁻ site, B³⁻ is bonded in a 9-coordinate geometry to six Ti⁴⁺ and three B³⁻ atoms. There is one shorter (1.77 Å) and two longer (1.78 Å) B-B bond lengths. In the second B³⁻ site, B³⁻ is bonded in a 9-coordinate geometry to seven Ti⁴⁺ and two equivalent B³⁻ atoms.',
 'NdCu₅ crystallizes in the hexagonal P6/mmm space group. Nd is bonded in a 18-coordinate geometry to eighteen Cu atoms. There are six shorter (2.93 Å) and twelve longer (3.25 Å) Nd-Cu bond lengths. There are two inequivalent Cu sites. In the first Cu site, Cu is bonded in a 9-coordinate geometry to three equivalent Nd and six equivalent Cu atoms. All Cu-Cu bond lengths are 2.51 Å. In the second Cu site, Cu is bonded to four equivalent Nd and eight Cu atoms to form a mixture of corner, edge, and face-sharing CuNd₄Cu₈ cuboctahedra. All Cu-Cu bond lengths are 2.53 Å.',
 'TlBi₂ crystallizes in the hexagonal P6/mmm space group. Tl is bonded in a 14-coordinate geometry to two equivalent Tl and twelve equivalent Bi atoms. Both Tl-Tl bond lengths are 3.39 Å. All Tl-Bi bond lengths are 3.71 Å. Bi is bonded to six equivalent Tl and five equivalent Bi atoms to form a mixture of distorted corner, edge, and face-sharing BiTl₆Bi₅ trigonal bipyramids. There are three shorter (3.30 Å) and two longer (3.39 Å) Bi-Bi bond lengths.',
 'AgI is lead oxide structured and crystallizes in the tetragonal P4/nmm space group. The structure is two-dimensional and consists of one AgI sheet oriented in the (0, 0, 1) direction. Ag¹⁺ is bonded to four equivalent I¹⁻ atoms to form a mixture of distorted corner and edge-sharing AgI₄ tetrahedra. All Ag-I bond lengths are 2.91 Å. I¹⁻ is bonded in a 4-coordinate geometry to four equivalent Ag¹⁺ atoms.',
 'MoC is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Mo⁴⁺ is bonded to four equivalent C⁴⁻ atoms to form corner-sharing MoC₄ tetrahedra. All Mo-C bond lengths are 2.03 Å. C⁴⁻ is bonded to four equivalent Mo⁴⁺ atoms to form corner-sharing CMo₄ tetrahedra.',
 'Nb₅Si₃ crystallizes in the tetragonal I4/mcm space group. There are two inequivalent Nb+2.40+ sites. In the first Nb+2.40+ site, Nb+2.40+ is bonded to five Si⁴⁻ atoms to form NbSi₅ trigonal bipyramids that share corners with four equivalent NbSi₆ octahedra, corners with twelve equivalent NbSi₅ trigonal bipyramids, edges with seven equivalent NbSi₅ trigonal bipyramids, faces with two equivalent NbSi₆ octahedra, and  a faceface with one NbSi₅ trigonal bipyramid. The corner-sharing octahedral tilt angles range from 43-64°. There are a spread of Nb-Si bond distances ranging from 2.61-2.74 Å. In the second Nb+2.40+ site, Nb+2.40+ is bonded to six Si⁴⁻ atoms to form NbSi₆ octahedra that share corners with six equivalent NbSi₆ octahedra, corners with sixteen equivalent NbSi₅ trigonal bipyramids, and faces with eight equivalent NbSi₅ trigonal bipyramids. The corner-sharing octahedral tilt angles range from 0-52°. There are four shorter (2.60 Å) and two longer (2.98 Å) Nb-Si bond lengths. There are two inequivalent Si⁴⁻ sites. In the first Si⁴⁻ site, Si⁴⁻ is bonded in a 9-coordinate geometry to eight Nb+2.40+ and one Si⁴⁻ atom. The Si-Si bond length is 2.40 Å. In the second Si⁴⁻ site, Si⁴⁻ is bonded in a 10-coordinate geometry to ten Nb+2.40+ atoms.',
 'OsTeCl₂ crystallizes in the cubic F̅43m space group. Os⁴⁺ is bonded in a body-centered cubic geometry to four equivalent Te²⁻ and four equivalent Cl¹⁻ atoms. All Os-Te bond lengths are 2.89 Å. All Os-Cl bond lengths are 2.89 Å. Te²⁻ is bonded in a 14-coordinate geometry to four equivalent Os⁴⁺ and ten Cl¹⁻ atoms. There are four shorter (2.89 Å) and six longer (3.33 Å) Te-Cl bond lengths. There are two inequivalent Cl¹⁻ sites. In the first Cl¹⁻ site, Cl¹⁻ is bonded to four equivalent Te²⁻ atoms to form distorted ClTe₄ tetrahedra that share corners with twenty-four ClTe₄ tetrahedra and faces with four equivalent ClTe₆Os₄ tetrahedra. In the second Cl¹⁻ site, Cl¹⁻ is bonded to four equivalent Os⁴⁺ and six equivalent Te²⁻ atoms to form a mixture of distorted corner and face-sharing ClTe₆Os₄ tetrahedra.',
 'PdIn is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Pd is bonded in a body-centered cubic geometry to eight equivalent In atoms. All Pd-In bond lengths are 2.83 Å. In is bonded in a body-centered cubic geometry to eight equivalent Pd atoms.',
 'Ca₃Hg₂ crystallizes in the tetragonal P4/mbm space group. There are two inequivalent Ca sites. In the first Ca site, Ca is bonded in a square co-planar geometry to four equivalent Hg atoms. All Ca-Hg bond lengths are 3.29 Å. In the second Ca site, Ca is bonded to six equivalent Hg atoms to form a mixture of distorted corner, edge, and face-sharing CaHg₆ pentagonal pyramids. There are two shorter (3.22 Å) and four longer (3.28 Å) Ca-Hg bond lengths. Hg is bonded in a 8-coordinate geometry to eight Ca atoms.',
 'Si is BC8 structured and crystallizes in the trigonal R̅3 space group. There are two inequivalent Si sites. In the first Si site, Si is bonded to four Si atoms to form corner-sharing SiSi₄ trigonal pyramids. There is one shorter (2.34 Å) and three longer (2.38 Å) Si-Si bond lengths. In the second Si site, Si is bonded to four Si atoms to form corner-sharing SiSi₄ trigonal pyramids. The Si-Si bond length is 2.41 Å.',
 'WCl₆ is Copper structured and crystallizes in the trigonal R̅3 space group. The structure is zero-dimensional and consists of three WCl₆ clusters. W⁶⁺ is bonded in an octahedral geometry to six equivalent Cl¹⁻ atoms. All W-Cl bond lengths are 2.29 Å. Cl¹⁻ is bonded in a single-bond geometry to one W⁶⁺ atom.',
 'BiS₂ crystallizes in the tetragonal I̅42d space group. Bi³⁺ is bonded to four equivalent S+1.50- atoms to form corner-sharing BiS₄ tetrahedra. All Bi-S bond lengths are 2.62 Å. S+1.50- is bonded in a water-like geometry to two equivalent Bi³⁺ atoms.',
 'CdSe is Wurtzite structured and crystallizes in the hexagonal P6₃mc space group. Cd²⁺ is bonded to four equivalent Se²⁻ atoms to form corner-sharing CdSe₄ tetrahedra. There are three shorter (2.66 Å) and one longer (2.67 Å) Cd-Se bond length. Se²⁻ is bonded to four equivalent Cd²⁺ atoms to form corner-sharing SeCd₄ tetrahedra.',
 'Be₅Zr crystallizes in the hexagonal P6/mmm space group. There are two inequivalent Be sites. In the first Be site, Be is bonded in a 9-coordinate geometry to six equivalent Be and three equivalent Zr atoms. All Be-Be bond lengths are 2.18 Å. All Be-Zr bond lengths are 2.64 Å. In the second Be site, Be is bonded to eight Be and four equivalent Zr atoms to form a mixture of corner, edge, and face-sharing BeZr₄Be₈ cuboctahedra. All Be-Be bond lengths are 2.28 Å. All Be-Zr bond lengths are 2.87 Å. Zr is bonded in a 18-coordinate geometry to eighteen Be atoms.',
 'C is rhombohedral graphite-like structured and crystallizes in the trigonal R̅3m space group. The structure is two-dimensional and consists of six C sheets oriented in the (0, 0, 1) direction. C is bonded in a trigonal planar geometry to three equivalent C atoms. All C-C bond lengths are 1.42 Å.',
 'NdPb₃ is Uranium Silicide structured and crystallizes in the cubic Pm̅3m space group. Nd is bonded to twelve equivalent Pb atoms to form a mixture of corner and face-sharing NdPb₁₂ cuboctahedra. All Nd-Pb bond lengths are 3.47 Å. Pb is bonded in a distorted square co-planar geometry to four equivalent Nd atoms.',
 'NaBi₃ is Uranium Silicide-like structured and crystallizes in the hexagonal P6₃/mmc space group. Na is bonded to twelve equivalent Bi atoms to form a mixture of corner and face-sharing NaBi₁₂ cuboctahedra. There are six shorter (3.47 Å) and six longer (3.53 Å) Na-Bi bond lengths. Bi is bonded in a distorted see-saw-like geometry to four equivalent Na atoms.',
 'Au₃Cd is Uranium Silicide structured and crystallizes in the tetragonal P4/mmm space group. There are two inequivalent Au sites. In the first Au site, Au is bonded in a distorted square co-planar geometry to four equivalent Cd atoms. All Au-Cd bond lengths are 2.95 Å. In the second Au site, Au is bonded in a distorted square co-planar geometry to four equivalent Cd atoms. All Au-Cd bond lengths are 2.94 Å. Cd is bonded to twelve Au atoms to form a mixture of corner and face-sharing CdAu₁₂ cuboctahedra.',
 'Cu₂S crystallizes in the tetragonal I4₁/amd space group. Cu¹⁺ is bonded in a linear geometry to two equivalent S²⁻ atoms. Both Cu-S bond lengths are 2.26 Å. S²⁻ is bonded to four equivalent Cu¹⁺ atoms to form distorted corner-sharing SCu₄ tetrahedra.',
 'AlCSc₃ is (Cubic) Perovskite structured and crystallizes in the cubic Pm̅3m space group. Sc is bonded in a linear geometry to four equivalent Al and two equivalent C atoms. All Sc-Al bond lengths are 3.19 Å. Both Sc-C bond lengths are 2.25 Å. Al is bonded to twelve equivalent Sc atoms to form AlSc₁₂ cuboctahedra that share corners with twelve equivalent AlSc₁₂ cuboctahedra, faces with six equivalent AlSc₁₂ cuboctahedra, and faces with eight equivalent CSc₆ octahedra. C is bonded to six equivalent Sc atoms to form CSc₆ octahedra that share corners with six equivalent CSc₆ octahedra and faces with eight equivalent AlSc₁₂ cuboctahedra. The corner-sharing octahedra are not tilted.',
 'Li crystallizes in the hexagonal P6/mmm space group. There are two inequivalent Li sites. In the first Li site, Li is bonded in a distorted linear geometry to fourteen Li atoms. There are two shorter (2.89 Å) and twelve longer (3.20 Å) Li-Li bond lengths. In the second Li site, Li is bonded to eleven Li atoms to form a mixture of distorted corner, edge, and face-sharing LiLi₁₁ trigonal bipyramids. There are three shorter (2.86 Å) and two longer (2.89 Å) Li-Li bond lengths.',
 'P is alpha As structured and crystallizes in the trigonal R̅3m space group. P is bonded in a 6-coordinate geometry to six equivalent P atoms. There are three shorter (2.29 Å) and three longer (2.82 Å) P-P bond lengths.',
 'SF₆ is Indium-like structured and crystallizes in the monoclinic C2/m space group. The structure is zero-dimensional and consists of six SF₆ clusters. In two of the SF₆ clusters, S⁶⁺ is bonded in an octahedral geometry to six equivalent F¹⁻ atoms. All S-F bond lengths are 1.58 Å. F¹⁻ is bonded in a single-bond geometry to one S⁶⁺ atom. In four of the SF₆ clusters, S⁶⁺ is bonded in an octahedral geometry to six equivalent F¹⁻ atoms. F¹⁻ is bonded in a single-bond geometry to one S⁶⁺ atom.',
 'Au₃Sc is Uranium Silicide-like structured and crystallizes in the hexagonal P6₃/mmc space group. Sc is bonded to twelve equivalent Au atoms to form a mixture of corner and face-sharing ScAu₁₂ cuboctahedra. There are six shorter (2.93 Å) and six longer (2.96 Å) Sc-Au bond lengths. Au is bonded in a distorted see-saw-like geometry to four equivalent Sc atoms.',
 'Pd₂₀Sb₇ crystallizes in the trigonal R̅3 space group. There are four inequivalent Pd sites. In the first Pd site, Pd is bonded in a 2-coordinate geometry to six Pd and four equivalent Sb atoms. There are a spread of Pd-Pd bond distances ranging from 2.83-3.05 Å. There are a spread of Pd-Sb bond distances ranging from 2.63-3.16 Å. In the second Pd site, Pd is bonded in a 10-coordinate geometry to seven Pd and three equivalent Sb atoms. There is one shorter (2.72 Å) and three longer (2.92 Å) Pd-Pd bond lengths. All Pd-Sb bond lengths are 2.63 Å. In the third Pd site, Pd is bonded in a 11-coordinate geometry to eight Pd and three Sb atoms. There are a spread of Pd-Pd bond distances ranging from 2.84-2.89 Å. There are a spread of Pd-Sb bond distances ranging from 2.67-2.94 Å. In the fourth Pd site, Pd is bonded in a 12-coordinate geometry to eight Pd and four Sb atoms. Both Pd-Pd bond lengths are 3.03 Å. There are a spread of Pd-Sb bond distances ranging from 2.67-2.83 Å. There are two inequivalent Sb sites. In the first Sb site, Sb is bonded in a cuboctahedral geometry to twelve Pd atoms. In the second Sb site, Sb is bonded in a 10-coordinate geometry to ten Pd atoms.',
 'MnSb is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Mn³⁺ is bonded to four equivalent Sb³⁻ atoms to form corner-sharing MnSb₄ tetrahedra. All Mn-Sb bond lengths are 2.68 Å. Sb³⁻ is bonded to four equivalent Mn³⁺ atoms to form corner-sharing SbMn₄ tetrahedra.',
 'MoP is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Mo³⁺ is bonded to four equivalent P³⁻ atoms to form corner-sharing MoP₄ tetrahedra. All Mo-P bond lengths are 2.35 Å. P³⁻ is bonded to four equivalent Mo³⁺ atoms to form corner-sharing PMo₄ tetrahedra.',
 'B is beta Boron structured and crystallizes in the trigonal R̅3m space group. There are fifteen inequivalent B sites. In the first B site, B is bonded in a distorted hexagonal planar geometry to six B atoms. There are four shorter (1.75 Å) and two longer (1.77 Å) B-B bond lengths. In the second B site, B is bonded in a 8-coordinate geometry to eight B atoms. There are a spread of B-B bond distances ranging from 1.76-1.87 Å. In the third B site, B is bonded in a 6-coordinate geometry to six B atoms. There are a spread of B-B bond distances ranging from 1.68-1.86 Å. In the fourth B site, B is bonded in a 6-coordinate geometry to six B atoms. There are a spread of B-B bond distances ranging from 1.70-1.86 Å. In the fifth B site, B is bonded in a 6-coordinate geometry to six B atoms. There is one shorter (1.63 Å) and two longer (1.75 Å) B-B bond lengths. In the sixth B site, B is bonded in a distorted q6 geometry to nine B atoms. There are three shorter (1.72 Å) and three longer (1.84 Å) B-B bond lengths. In the seventh B site, B is bonded in a 6-coordinate geometry to six equivalent B atoms. All B-B bond lengths are 1.73 Å. In the eighth B site, B is bonded in a distorted hexagonal planar geometry to six B atoms. There are two shorter (1.78 Å) and one longer (1.80 Å) B-B bond length. In the ninth B site, B is bonded in a 6-coordinate geometry to six B atoms. There is one shorter (1.70 Å) and one longer (1.82 Å) B-B bond length. In the tenth B site, B is bonded in a 6-coordinate geometry to six B atoms. Both B-B bond lengths are 1.78 Å. In the eleventh B site, B is bonded in a 6-coordinate geometry to six B atoms. Both B-B bond lengths are 1.83 Å. In the twelfth B site, B is bonded in a 8-coordinate geometry to eight B atoms. There are a spread of B-B bond distances ranging from 1.76-1.99 Å. In the thirteenth B site, B is bonded in a 6-coordinate geometry to six B atoms. There is one shorter (1.79 Å) and one longer (1.91 Å) B-B bond length. In the fourteenth B site, B is bonded in a 6-coordinate geometry to six B atoms. There is one shorter (1.87 Å) and one longer (1.88 Å) B-B bond length. In the fifteenth B site, B is bonded in a 8-coordinate geometry to eight B atoms.',
 'KBaSn is half-Heusler structured and crystallizes in the cubic F̅43m space group. K is bonded in a body-centered cubic geometry to four equivalent Ba and four equivalent Sn atoms. All K-Ba bond lengths are 3.50 Å. All K-Sn bond lengths are 3.50 Å. Ba is bonded in a distorted q6 geometry to four equivalent K and six equivalent Sn atoms. All Ba-Sn bond lengths are 4.04 Å. Sn is bonded to four equivalent K and six equivalent Ba atoms to form a mixture of distorted corner and face-sharing SnK₄Ba₆ tetrahedra.',
 'SmTl is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Sm is bonded in a body-centered cubic geometry to eight equivalent Tl atoms. All Sm-Tl bond lengths are 3.35 Å. Tl is bonded in a body-centered cubic geometry to eight equivalent Sm atoms.',
 'CuSe₂ is Pyrite-like structured and crystallizes in the cubic Pa̅3 space group. Cu²⁺ is bonded to six equivalent Se¹⁻ atoms to form CuSe₆ octahedra that share corners with twelve equivalent CuSe₆ octahedra and corners with six equivalent SeCu₃Se tetrahedra. The corner-sharing octahedral tilt angles are 65°. All Cu-Se bond lengths are 2.58 Å. Se¹⁻ is bonded to three equivalent Cu²⁺ and one Se¹⁻ atom to form distorted SeCu₃Se tetrahedra that share corners with three equivalent CuSe₆ octahedra and corners with fifteen equivalent SeCu₃Se tetrahedra. The corner-sharing octahedral tilt angles are 76°. The Se-Se bond length is 2.33 Å.',
 'Sn is Tungsten structured and crystallizes in the cubic Im̅3m space group. Sn is bonded in a body-centered cubic geometry to eight equivalent Sn atoms. All Sn-Sn bond lengths are 3.28 Å.',
 'SbNCa₃ is (Cubic) Perovskite structured and crystallizes in the cubic Pm̅3m space group. Ca²⁺ is bonded in a linear geometry to four equivalent Sb³⁻ and two equivalent N³⁻ atoms. All Ca-Sb bond lengths are 3.44 Å. Both Ca-N bond lengths are 2.43 Å. Sb³⁻ is bonded to twelve equivalent Ca²⁺ atoms to form SbCa₁₂ cuboctahedra that share corners with twelve equivalent SbCa₁₂ cuboctahedra, faces with six equivalent SbCa₁₂ cuboctahedra, and faces with eight equivalent NCa₆ octahedra. N³⁻ is bonded to six equivalent Ca²⁺ atoms to form NCa₆ octahedra that share corners with six equivalent NCa₆ octahedra and faces with eight equivalent SbCa₁₂ cuboctahedra. The corner-sharing octahedra are not tilted.',
 'ZnI₂ crystallizes in the hexagonal P6/mmm space group. The structure is two-dimensional and consists of one Zn cluster and one I sheet oriented in the (0, 0, 1) direction. In the Zn cluster, Zn²⁺ is bonded in a 12-coordinate geometry to  atoms. In the I sheet, I¹⁻ is bonded in a distorted trigonal planar geometry to three equivalent I¹⁻ atoms. All I-I bond lengths are 3.22 Å.',
 'Pd₂Hg₅ crystallizes in the tetragonal P4/mbm space group. Pd is bonded to two equivalent Pd and ten Hg atoms to form a mixture of distorted corner, edge, and face-sharing PdHg₁₀Pd₂ cuboctahedra. Both Pd-Pd bond lengths are 2.98 Å. There are a spread of Pd-Hg bond distances ranging from 2.91-3.04 Å. There are two inequivalent Hg sites. In the first Hg site, Hg is bonded in a 10-coordinate geometry to four equivalent Pd and six Hg atoms. There are a spread of Hg-Hg bond distances ranging from 2.96-3.22 Å. In the second Hg site, Hg is bonded in a distorted q6 geometry to four equivalent Pd and six Hg atoms. Both Hg-Hg bond lengths are 2.98 Å.',
 'HfBe₂ is hexagonal omega structure structured and crystallizes in the hexagonal P6/mmm space group. Be is bonded in a 9-coordinate geometry to three equivalent Be and six equivalent Hf atoms. All Be-Be bond lengths are 2.18 Å. All Be-Hf bond lengths are 2.70 Å. Hf is bonded to twelve equivalent Be atoms to form a mixture of edge and face-sharing HfBe₁₂ cuboctahedra.',
 'Li₃GaN₂ is alpha bismuth trifluoride-derived structured and crystallizes in the cubic Ia̅3 space group. Li¹⁺ is bonded to four N³⁻ atoms to form LiN₄ tetrahedra that share corners with four equivalent GaN₄ tetrahedra, corners with twelve equivalent LiN₄ tetrahedra, edges with two equivalent GaN₄ tetrahedra, and edges with four equivalent LiN₄ tetrahedra. There are a spread of Li-N bond distances ranging from 2.06-2.15 Å. Ga³⁺ is bonded to four N³⁻ atoms to form GaN₄ tetrahedra that share corners with four equivalent GaN₄ tetrahedra, corners with twelve equivalent LiN₄ tetrahedra, and edges with six equivalent LiN₄ tetrahedra. There is one shorter (1.96 Å) and three longer (1.99 Å) Ga-N bond lengths. There are two inequivalent N³⁻ sites. In the first N³⁻ site, N³⁻ is bonded in a body-centered cubic geometry to six equivalent Li¹⁺ and two equivalent Ga³⁺ atoms. In the second N³⁻ site, N³⁻ is bonded in a body-centered cubic geometry to six equivalent Li¹⁺ and two equivalent Ga³⁺ atoms.',
 'ThNI is Matlockite structured and crystallizes in the tetragonal P4/nmm space group. The structure is two-dimensional and consists of one ThNI sheet oriented in the (0, 0, 1) direction. Th⁴⁺ is bonded in a 4-coordinate geometry to four equivalent N³⁻ and four equivalent I¹⁻ atoms. All Th-N bond lengths are 2.35 Å. All Th-I bond lengths are 3.53 Å. N³⁻ is bonded to four equivalent Th⁴⁺ atoms to form a mixture of corner and edge-sharing NTh₄ tetrahedra. I¹⁻ is bonded in a 4-coordinate geometry to four equivalent Th⁴⁺ atoms.',
 'SiC is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Si⁴⁺ is bonded to four equivalent C⁴⁻ atoms to form corner-sharing SiC₄ tetrahedra. All Si-C bond lengths are 1.89 Å. C⁴⁻ is bonded to four equivalent Si⁴⁺ atoms to form corner-sharing CSi₄ tetrahedra.',
 'VTl₃ is alpha La-derived structured and crystallizes in the tetragonal I4/mmm space group. V is bonded to twelve Tl atoms to form VTl₁₂ cuboctahedra that share corners with four equivalent VTl₁₂ cuboctahedra, corners with eight equivalent TlTl₈V₄ cuboctahedra, edges with eight equivalent VTl₁₂ cuboctahedra, edges with sixteen equivalent TlTl₈V₄ cuboctahedra, faces with four equivalent VTl₁₂ cuboctahedra, and faces with fourteen TlTl₈V₄ cuboctahedra. There are four shorter (3.27 Å) and eight longer (3.32 Å) V-Tl bond lengths. There are two inequivalent Tl sites. In the first Tl site, Tl is bonded to four equivalent V and eight Tl atoms to form TlTl₈V₄ cuboctahedra that share corners with twelve equivalent TlTl₈V₄ cuboctahedra, edges with eight equivalent VTl₁₂ cuboctahedra, edges with sixteen TlTl₈V₄ cuboctahedra, faces with four equivalent VTl₁₂ cuboctahedra, and faces with fourteen TlTl₈V₄ cuboctahedra. There are four shorter (3.27 Å) and four longer (3.32 Å) Tl-Tl bond lengths. In the second Tl site, Tl is bonded to four equivalent V and eight equivalent Tl atoms to form TlTl₈V₄ cuboctahedra that share corners with four equivalent TlTl₈V₄ cuboctahedra, corners with eight equivalent VTl₁₂ cuboctahedra, edges with twenty-four TlTl₈V₄ cuboctahedra, faces with six equivalent VTl₁₂ cuboctahedra, and faces with twelve TlTl₈V₄ cuboctahedra.',
 'ScRu₂ is Hexagonal Laves structured and crystallizes in the hexagonal P6₃/mmc space group. Sc is bonded in a 12-coordinate geometry to twelve Ru atoms. There are nine shorter (3.00 Å) and three longer (3.09 Å) Sc-Ru bond lengths. There are two inequivalent Ru sites. In the first Ru site, Ru is bonded to six equivalent Sc and six equivalent Ru atoms to form a mixture of corner, edge, and face-sharing RuSc₆Ru₆ cuboctahedra. All Ru-Ru bond lengths are 2.64 Å. In the second Ru site, Ru is bonded to six equivalent Sc and six Ru atoms to form a mixture of corner, edge, and face-sharing RuSc₆Ru₆ cuboctahedra. There are two shorter (2.47 Å) and two longer (2.63 Å) Ru-Ru bond lengths.',
 'BSe₂Cl is Heusler structured and crystallizes in the cubic Fm̅3m space group. B³⁻ is bonded in a body-centered cubic geometry to eight equivalent Se²⁺ atoms. All B-Se bond lengths are 2.79 Å. Se²⁺ is bonded in a distorted body-centered cubic geometry to four equivalent B³⁻ and four equivalent Cl¹⁻ atoms. All Se-Cl bond lengths are 2.79 Å. Cl¹⁻ is bonded in a body-centered cubic geometry to eight equivalent Se²⁺ atoms.',
 'Mn₂Au crystallizes in the tetragonal I4/mmm space group. The structure is two-dimensional and consists of two Mn₂Au sheets oriented in the (0, 0, 1) direction. Mn is bonded in a 4-coordinate geometry to four equivalent Au atoms. All Mn-Au bond lengths are 2.82 Å. Au is bonded to eight equivalent Mn atoms to form a mixture of distorted edge and face-sharing AuMn₈ cuboctahedra.',
 'Ga₂TeS₂ crystallizes in the tetragonal I4₁md space group. Ga³⁺ is bonded to one Te²⁻ and three equivalent S²⁻ atoms to form a mixture of distorted corner and edge-sharing GaTeS₃ tetrahedra. The Ga-Te bond length is 2.56 Å. There is one shorter (2.30 Å) and two longer (2.35 Å) Ga-S bond lengths. Te²⁻ is bonded in a water-like geometry to two equivalent Ga³⁺ atoms. S²⁻ is bonded in a 3-coordinate geometry to three equivalent Ga³⁺ atoms.',
 'LiCaSn crystallizes in the hexagonal P̅6m2 space group. There are two inequivalent Li sites. In the first Li site, Li is bonded in a 3-coordinate geometry to three equivalent Ca and four Sn atoms. All Li-Ca bond lengths are 3.20 Å. There are three shorter (2.89 Å) and one longer (3.29 Å) Li-Sn bond length. In the second Li site, Li is bonded in a trigonal planar geometry to six equivalent Ca and three equivalent Sn atoms. All Li-Ca bond lengths are 3.37 Å. All Li-Sn bond lengths are 2.84 Å. There are two inequivalent Ca sites. In the first Ca site, Ca is bonded in a 14-coordinate geometry to six Li, two Ca, and six Sn atoms. Both Ca-Ca bond lengths are 3.61 Å. There are three shorter (3.37 Å) and three longer (3.47 Å) Ca-Sn bond lengths. In the second Ca site, Ca is bonded in a 6-coordinate geometry to two equivalent Ca and six equivalent Sn atoms. All Ca-Sn bond lengths are 3.27 Å. There are two inequivalent Sn sites. In the first Sn site, Sn is bonded in a 10-coordinate geometry to three equivalent Li, six Ca, and one Sn atom. The Sn-Sn bond length is 3.23 Å. In the second Sn site, Sn is bonded in a 11-coordinate geometry to five Li and six equivalent Ca atoms.',
 'Ba₂NbO is Heusler structured and crystallizes in the cubic Fm̅3m space group. Ba is bonded in a body-centered cubic geometry to four equivalent Nb and four equivalent O atoms. All Ba-Nb bond lengths are 3.41 Å. All Ba-O bond lengths are 3.41 Å. Nb is bonded in a body-centered cubic geometry to eight equivalent Ba atoms. O is bonded in a body-centered cubic geometry to eight equivalent Ba atoms.',
 'LaF₃ is Sodium arsenide structured and crystallizes in the hexagonal P6₃/mmc space group. La³⁺ is bonded in a 11-coordinate geometry to eleven F¹⁻ atoms. There are a spread of La-F bond distances ranging from 2.40-2.70 Å. There are two inequivalent F¹⁻ sites. In the first F¹⁻ site, F¹⁻ is bonded in a 4-coordinate geometry to four equivalent La³⁺ atoms. In the second F¹⁻ site, F¹⁻ is bonded in a trigonal planar geometry to three equivalent La³⁺ atoms.',
 'AuTl₂ crystallizes in the tetragonal I4/mcm space group. Au is bonded to two equivalent Au and four equivalent Tl atoms to form distorted corner-sharing AuTl₄Au₂ octahedra. The corner-sharing octahedral tilt angles range from 0-43°. Both Au-Au bond lengths are 2.71 Å. All Au-Tl bond lengths are 2.92 Å. Tl is bonded in a distorted bent 150 degrees geometry to two equivalent Au atoms.',
 'CdS is Boron Nitride-like structured and crystallizes in the trigonal P3m1 space group. The structure is two-dimensional and consists of one CdS sheet oriented in the (0, 0, 1) direction. Cd²⁺ is bonded in a trigonal planar geometry to three equivalent S²⁻ atoms. All Cd-S bond lengths are 2.46 Å. S²⁻ is bonded in a trigonal planar geometry to three equivalent Cd²⁺ atoms.',
 'Cs₂Pt crystallizes in the hexagonal P6₃/mmc space group. There are two inequivalent Cs sites. In the first Cs site, Cs is bonded to five equivalent Pt atoms to form distorted CsPt₅ trigonal bipyramids that share corners with twelve equivalent CsPt₆ octahedra, corners with eight equivalent CsPt₅ trigonal bipyramids, edges with six equivalent CsPt₅ trigonal bipyramids, and faces with six equivalent CsPt₆ octahedra. The corner-sharing octahedral tilt angles range from 30-60°. There are three shorter (3.35 Å) and two longer (3.95 Å) Cs-Pt bond lengths. In the second Cs site, Cs is bonded to six equivalent Pt atoms to form distorted CsPt₆ octahedra that share corners with twelve equivalent CsPt₆ octahedra, corners with twelve equivalent CsPt₅ trigonal bipyramids, edges with six equivalent CsPt₆ octahedra, faces with two equivalent CsPt₆ octahedra, and faces with six equivalent CsPt₅ trigonal bipyramids. The corner-sharing octahedral tilt angles are 51°. All Cs-Pt bond lengths are 3.89 Å. Pt is bonded in a 11-coordinate geometry to eleven Cs atoms.',
 'Pd₃In is Uranium Silicide structured and crystallizes in the cubic Pm̅3m space group. Pd is bonded to eight equivalent Pd and four equivalent In atoms to form distorted PdIn₄Pd₈ cuboctahedra that share corners with twelve equivalent PdIn₄Pd₈ cuboctahedra, edges with eight equivalent InPd₁₂ cuboctahedra, edges with sixteen equivalent PdIn₄Pd₈ cuboctahedra, faces with four equivalent InPd₁₂ cuboctahedra, and faces with fourteen equivalent PdIn₄Pd₈ cuboctahedra. All Pd-Pd bond lengths are 2.83 Å. All Pd-In bond lengths are 2.83 Å. In is bonded to twelve equivalent Pd atoms to form InPd₁₂ cuboctahedra that share corners with twelve equivalent InPd₁₂ cuboctahedra, edges with twenty-four equivalent PdIn₄Pd₈ cuboctahedra, faces with six equivalent InPd₁₂ cuboctahedra, and faces with twelve equivalent PdIn₄Pd₈ cuboctahedra.',
 'BaAu₂ is hexagonal omega structure structured and crystallizes in the hexagonal P6/mmm space group. Ba is bonded to twelve equivalent Au atoms to form a mixture of edge and face-sharing BaAu₁₂ cuboctahedra. All Ba-Au bond lengths are 3.51 Å. Au is bonded in a 9-coordinate geometry to six equivalent Ba and three equivalent Au atoms. All Au-Au bond lengths are 2.81 Å.',
 'LiYGe crystallizes in the hexagonal P̅62m space group. Li¹⁺ is bonded to four Ge⁴⁻ atoms to form LiGe₄ tetrahedra that share corners with six equivalent YGe₅ square pyramids, corners with ten equivalent LiGe₄ tetrahedra, edges with six equivalent YGe₅ square pyramids, and edges with two equivalent LiGe₄ tetrahedra. There are two shorter (2.68 Å) and two longer (2.79 Å) Li-Ge bond lengths. Y³⁺ is bonded to five Ge⁴⁻ atoms to form YGe₅ square pyramids that share corners with ten equivalent YGe₅ square pyramids, corners with six equivalent LiGe₄ tetrahedra, edges with six equivalent YGe₅ square pyramids, and edges with six equivalent LiGe₄ tetrahedra. All Y-Ge bond lengths are 2.99 Å. There are two inequivalent Ge⁴⁻ sites. In the first Ge⁴⁻ site, Ge⁴⁻ is bonded in a 9-coordinate geometry to six equivalent Li¹⁺ and three equivalent Y³⁺ atoms. In the second Ge⁴⁻ site, Ge⁴⁻ is bonded in a 9-coordinate geometry to three equivalent Li¹⁺ and six equivalent Y³⁺ atoms.',
 'USnAu₂ is Heusler structured and crystallizes in the cubic Fm̅3m space group. U is bonded in a body-centered cubic geometry to eight equivalent Au atoms. All U-Au bond lengths are 3.06 Å. Au is bonded in a body-centered cubic geometry to four equivalent U and four equivalent Sn atoms. All Au-Sn bond lengths are 3.06 Å. Sn is bonded in a distorted body-centered cubic geometry to eight equivalent Au atoms.',
 'AcN is Wurtzite structured and crystallizes in the hexagonal P6₃mc space group. Ac³⁺ is bonded to four equivalent N³⁻ atoms to form corner-sharing AcN₄ tetrahedra. There are three shorter (2.54 Å) and one longer (2.55 Å) Ac-N bond length. N³⁻ is bonded to four equivalent Ac³⁺ atoms to form corner-sharing NAc₄ tetrahedra.',
 'PdTe₂ is Calaverite structured and crystallizes in the trigonal P̅3m1 space group. The structure is two-dimensional and consists of one PdTe₂ sheet oriented in the (0, 0, 1) direction. Pd⁴⁺ is bonded to six equivalent Te²⁻ atoms to form edge-sharing PdTe₆ octahedra. All Pd-Te bond lengths are 2.72 Å. Te²⁻ is bonded in a 3-coordinate geometry to three equivalent Pd⁴⁺ atoms.',
 'Rb is Copper structured and crystallizes in the cubic Fm̅3m space group. Rb is bonded to twelve equivalent Rb atoms to form a mixture of corner, edge, and face-sharing RbRb₁₂ cuboctahedra. All Rb-Rb bond lengths are 5.15 Å.',
 'Ni₄Ti₃ is delta Molybdenum Boride-like structured and crystallizes in the trigonal R̅3 space group. Ti is bonded in a 9-coordinate geometry to nine Ni atoms. There are a spread of Ti-Ni bond distances ranging from 2.42-2.73 Å. There are three inequivalent Ni sites. In the first Ni site, Ni is bonded in a 8-coordinate geometry to six equivalent Ti and two equivalent Ni atoms. Both Ni-Ni bond lengths are 2.51 Å. In the second Ni site, Ni is bonded in a 7-coordinate geometry to seven equivalent Ti atoms. In the third Ni site, Ni is bonded in a 8-coordinate geometry to six equivalent Ti and two equivalent Ni atoms.',
 'NdGe₂ crystallizes in the tetragonal I4₁/amd space group. Nd is bonded to twelve equivalent Ge atoms to form a mixture of edge and face-sharing NdGe₁₂ cuboctahedra. There are eight shorter (3.19 Å) and four longer (3.44 Å) Nd-Ge bond lengths. Ge is bonded in a 9-coordinate geometry to six equivalent Nd and three equivalent Ge atoms. There are two shorter (2.53 Å) and one longer (2.57 Å) Ge-Ge bond length.',
 'FeSi₄P₄ crystallizes in the triclinic P1 space group. Fe³⁺ is bonded in a distorted T-shaped geometry to three P³⁻ atoms. There are a spread of Fe-P bond distances ranging from 2.28-2.32 Å. Si³⁺ is bonded in a tetrahedral geometry to four P³⁻ atoms. There is one shorter (2.21 Å) and three longer (2.25 Å) Si-P bond lengths. There are three inequivalent Si²⁺ sites. In the first Si²⁺ site, Si²⁺ is bonded in a trigonal non-coplanar geometry to three P³⁻ atoms. There are two shorter (2.31 Å) and one longer (2.32 Å) Si-P bond length. In the second Si²⁺ site, Si²⁺ is bonded in a distorted T-shaped geometry to three P³⁻ atoms. There is one shorter (2.30 Å) and two longer (2.31 Å) Si-P bond lengths. In the third Si²⁺ site, Si²⁺ is bonded in a distorted T-shaped geometry to three P³⁻ atoms. There is one shorter (2.26 Å) and two longer (2.32 Å) Si-P bond lengths. There are four inequivalent P³⁻ sites. In the first P³⁻ site, P³⁻ is bonded to one Fe³⁺, one Si³⁺, and two Si²⁺ atoms to form corner-sharing PFeSi₃ trigonal pyramids. In the second P³⁻ site, P³⁻ is bonded to one Fe³⁺, one Si³⁺, and two Si²⁺ atoms to form corner-sharing PFeSi₃ tetrahedra. In the third P³⁻ site, P³⁻ is bonded to one Si³⁺ and three Si²⁺ atoms to form corner-sharing PSi₄ tetrahedra. In the fourth P³⁻ site, P³⁻ is bonded to one Fe³⁺, one Si³⁺, and two Si²⁺ atoms to form corner-sharing PFeSi₃ tetrahedra.',
 'Mn₂Ge crystallizes in the hexagonal P6₃/mmc space group. There are two inequivalent Mn²⁺ sites. In the first Mn²⁺ site, Mn²⁺ is bonded in a 8-coordinate geometry to two equivalent Mn²⁺ and six equivalent Ge⁴⁻ atoms. Both Mn-Mn bond lengths are 2.55 Å. All Mn-Ge bond lengths are 2.73 Å. In the second Mn²⁺ site, Mn²⁺ is bonded to five equivalent Ge⁴⁻ atoms to form a mixture of corner and edge-sharing MnGe₅ trigonal bipyramids. There are three shorter (2.42 Å) and two longer (2.55 Å) Mn-Ge bond lengths. Ge⁴⁻ is bonded in a 11-coordinate geometry to eleven Mn²⁺ atoms.',
 'TiF₃ is alpha Rhenium trioxide structured and crystallizes in the trigonal R̅3c space group. Ti³⁺ is bonded to six equivalent F¹⁻ atoms to form corner-sharing TiF₆ octahedra. The corner-sharing octahedral tilt angles are 1°. All Ti-F bond lengths are 1.96 Å. F¹⁻ is bonded in a linear geometry to two equivalent Ti³⁺ atoms.',
 'TiS₂ crystallizes in the tetragonal I̅42d space group. Ti⁴⁺ is bonded to four equivalent S²⁻ atoms to form corner-sharing TiS₄ tetrahedra. All Ti-S bond lengths are 2.28 Å. S²⁻ is bonded in a bent 120 degrees geometry to two equivalent Ti⁴⁺ atoms.',
 'LiSi₃ is beta Cu3Ti-like structured and crystallizes in the hexagonal P6₃/mmc space group. Li is bonded to twelve equivalent Si atoms to form a mixture of corner and face-sharing LiSi₁₂ cuboctahedra. There are six shorter (2.71 Å) and six longer (2.83 Å) Li-Si bond lengths. Si is bonded in a 4-coordinate geometry to four equivalent Li atoms.',
 'UP₃ crystallizes in the hexagonal P6₃/mmc space group. U⁶⁺ is bonded to twelve equivalent P²⁻ atoms to form a mixture of distorted corner and face-sharing UP₁₂ cuboctahedra. There are six shorter (2.72 Å) and six longer (2.98 Å) U-P bond lengths. P²⁻ is bonded in a 6-coordinate geometry to four equivalent U⁶⁺ and two equivalent P²⁻ atoms. Both P-P bond lengths are 2.40 Å.',
 'Rb₃Tc is alpha bismuth trifluoride structured and crystallizes in the tetragonal I4/mmm space group. There are two inequivalent Rb sites. In the first Rb site, Rb is bonded in a 4-coordinate geometry to four equivalent Rb and four equivalent Tc atoms. All Rb-Rb bond lengths are 4.17 Å. All Rb-Tc bond lengths are 4.17 Å. In the second Rb site, Rb is bonded to eight equivalent Rb and four equivalent Tc atoms to form RbRb₈Tc₄ cuboctahedra that share corners with four equivalent RbRb₈Tc₄ cuboctahedra, corners with eight equivalent TcRb₁₂ cuboctahedra, edges with eight equivalent RbRb₈Tc₄ cuboctahedra, faces with four equivalent RbRb₈Tc₄ cuboctahedra, and faces with six equivalent TcRb₁₂ cuboctahedra. All Rb-Tc bond lengths are 4.36 Å. Tc is bonded to twelve Rb atoms to form TcRb₁₂ cuboctahedra that share corners with four equivalent TcRb₁₂ cuboctahedra, corners with eight equivalent RbRb₈Tc₄ cuboctahedra, edges with eight equivalent TcRb₁₂ cuboctahedra, faces with four equivalent TcRb₁₂ cuboctahedra, and faces with six equivalent RbRb₈Tc₄ cuboctahedra.',
 'Li₃Ag is Uranium Silicide-like structured and crystallizes in the hexagonal P6₃/mmc space group. Li is bonded in a distorted see-saw-like geometry to four equivalent Ag atoms. There are two shorter (2.80 Å) and two longer (2.88 Å) Li-Ag bond lengths. Ag is bonded to twelve equivalent Li atoms to form a mixture of corner and face-sharing AgLi₁₂ cuboctahedra.',
 'IrTe crystallizes in the hexagonal P6₃/mmc space group. Ir is bonded in a 8-coordinate geometry to two equivalent Ir and six equivalent Te atoms. Both Ir-Ir bond lengths are 2.83 Å. All Ir-Te bond lengths are 2.76 Å. Te is bonded in a 6-coordinate geometry to six equivalent Ir atoms.',
 'ScCd₇ crystallizes in the orthorhombic Cmcm space group. Sc is bonded to fourteen Cd atoms to form a mixture of distorted corner and face-sharing ScCd₁₄ cuboctahedra. There are a spread of Sc-Cd bond distances ranging from 3.12-3.72 Å. There are four inequivalent Cd sites. In the first Cd site, Cd is bonded in a 11-coordinate geometry to two equivalent Sc and nine Cd atoms. There are a spread of Cd-Cd bond distances ranging from 2.96-3.25 Å. In the second Cd site, Cd is bonded in a 11-coordinate geometry to two equivalent Sc and nine Cd atoms. There are a spread of Cd-Cd bond distances ranging from 2.99-3.32 Å. In the third Cd site, Cd is bonded in a 11-coordinate geometry to two equivalent Sc and nine Cd atoms. There are two shorter (2.93 Å) and one longer (3.07 Å) Cd-Cd bond length. In the fourth Cd site, Cd is bonded in a 12-coordinate geometry to two equivalent Sc and ten Cd atoms.',
 'Fe₃Pt is Uranium Silicide structured and crystallizes in the cubic Pm̅3m space group. Fe is bonded to eight equivalent Fe and four equivalent Pt atoms to form distorted FeFe₈Pt₄ cuboctahedra that share corners with twelve equivalent FeFe₈Pt₄ cuboctahedra, edges with eight equivalent PtFe₁₂ cuboctahedra, edges with sixteen equivalent FeFe₈Pt₄ cuboctahedra, faces with four equivalent PtFe₁₂ cuboctahedra, and faces with fourteen equivalent FeFe₈Pt₄ cuboctahedra. All Fe-Fe bond lengths are 2.65 Å. All Fe-Pt bond lengths are 2.65 Å. Pt is bonded to twelve equivalent Fe atoms to form PtFe₁₂ cuboctahedra that share corners with twelve equivalent PtFe₁₂ cuboctahedra, edges with twenty-four equivalent FeFe₈Pt₄ cuboctahedra, faces with six equivalent PtFe₁₂ cuboctahedra, and faces with twelve equivalent FeFe₈Pt₄ cuboctahedra.',
 'PtGa is alpha-derived structured and crystallizes in the cubic P2_13 space group. Pt is bonded in a 7-coordinate geometry to seven equivalent Ga atoms. There are a spread of Pt-Ga bond distances ranging from 2.60-2.67 Å. Ga is bonded in a 7-coordinate geometry to seven equivalent Pt atoms.',
 'Li₆MoN₄ crystallizes in the tetragonal P4₂/nmc space group. There are two inequivalent Li¹⁺ sites. In the first Li¹⁺ site, Li¹⁺ is bonded to four equivalent N³⁻ atoms to form distorted LiN₄ tetrahedra that share corners with two equivalent MoN₄ tetrahedra, corners with twelve LiN₄ tetrahedra,  an edgeedge with one MoN₄ tetrahedra, and edges with four LiN₄ tetrahedra. There are two shorter (2.05 Å) and two longer (2.15 Å) Li-N bond lengths. In the second Li¹⁺ site, Li¹⁺ is bonded to four equivalent N³⁻ atoms to form LiN₄ tetrahedra that share corners with four equivalent MoN₄ tetrahedra, corners with eight equivalent LiN₄ tetrahedra, and edges with six LiN₄ tetrahedra. There are two shorter (2.02 Å) and two longer (2.18 Å) Li-N bond lengths. Mo⁶⁺ is bonded to four equivalent N³⁻ atoms to form MoN₄ tetrahedra that share corners with sixteen LiN₄ tetrahedra and edges with four equivalent LiN₄ tetrahedra. All Mo-N bond lengths are 1.87 Å. N³⁻ is bonded to six Li¹⁺ and one Mo⁶⁺ atom to form a mixture of distorted corner and edge-sharing NLi₆Mo pentagonal bipyramids.',
 'PrSi₂ is hexagonal omega structure-like structured and crystallizes in the tetragonal I4₁/amd space group. Pr²⁺ is bonded to twelve Si¹⁻ atoms to form a mixture of edge and face-sharing PrSi₁₂ cuboctahedra. There are four shorter (3.10 Å) and eight longer (3.28 Å) Pr-Si bond lengths. There are four inequivalent Si¹⁻ sites. In the first Si¹⁻ site, Si¹⁻ is bonded in a 9-coordinate geometry to six equivalent Pr²⁺ and three Si¹⁻ atoms. There are two shorter (3.10 Å) and four longer (3.28 Å) Si-Pr bond lengths. There is one shorter (2.33 Å) and two longer (2.41 Å) Si-Si bond lengths. In the second Si¹⁻ site, Si¹⁻ is bonded in a 9-coordinate geometry to six equivalent Pr²⁺ and three Si¹⁻ atoms. There is one shorter (2.33 Å) and two longer (2.41 Å) Si-Si bond lengths. In the third Si¹⁻ site, Si¹⁻ is bonded in a 9-coordinate geometry to six equivalent Pr²⁺ and three equivalent Si¹⁻ atoms. In the fourth Si¹⁻ site, Si¹⁻ is bonded in a 9-coordinate geometry to six equivalent Pr²⁺ and three equivalent Si¹⁻ atoms. There are two shorter (3.10 Å) and four longer (3.28 Å) Si-Pr bond lengths.',
 'AgI is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Ag¹⁺ is bonded to four equivalent I¹⁻ atoms to form corner-sharing AgI₄ tetrahedra. All Ag-I bond lengths are 2.84 Å. I¹⁻ is bonded to four equivalent Ag¹⁺ atoms to form corner-sharing IAg₄ tetrahedra.',
 'TcAg₃ is alpha bismuth trifluoride structured and crystallizes in the tetragonal I4/mmm space group. Tc is bonded to twelve Ag atoms to form TcAg₁₂ cuboctahedra that share corners with four equivalent TcAg₁₂ cuboctahedra, corners with eight equivalent AgTc₄Ag₈ cuboctahedra, edges with eight equivalent TcAg₁₂ cuboctahedra, edges with sixteen equivalent AgTc₄Ag₈ cuboctahedra, faces with four equivalent TcAg₁₂ cuboctahedra, and faces with fourteen AgTc₄Ag₈ cuboctahedra. There are eight shorter (2.88 Å) and four longer (3.01 Å) Tc-Ag bond lengths. There are two inequivalent Ag sites. In the first Ag site, Ag is bonded to four equivalent Tc and eight Ag atoms to form AgTc₄Ag₈ cuboctahedra that share corners with twelve equivalent AgTc₄Ag₈ cuboctahedra, edges with eight equivalent TcAg₁₂ cuboctahedra, edges with sixteen AgTc₄Ag₈ cuboctahedra, faces with four equivalent TcAg₁₂ cuboctahedra, and faces with fourteen AgTc₄Ag₈ cuboctahedra. There are four shorter (2.88 Å) and four longer (3.01 Å) Ag-Ag bond lengths. In the second Ag site, Ag is bonded to four equivalent Tc and eight equivalent Ag atoms to form AgTc₄Ag₈ cuboctahedra that share corners with four equivalent AgTc₄Ag₈ cuboctahedra, corners with eight equivalent TcAg₁₂ cuboctahedra, edges with twenty-four AgTc₄Ag₈ cuboctahedra, faces with six equivalent TcAg₁₂ cuboctahedra, and faces with twelve AgTc₄Ag₈ cuboctahedra.',
 'TiAs is Molybdenum Carbide MAX Phase-like structured and crystallizes in the hexagonal P6₃/mmc space group. Ti³⁺ is bonded to six equivalent As³⁻ atoms to form a mixture of distorted corner and edge-sharing TiAs₆ pentagonal pyramids. All Ti-As bond lengths are 2.61 Å. As³⁻ is bonded to six equivalent Ti³⁺ atoms to form a mixture of corner, edge, and face-sharing AsTi₆ octahedra. The corner-sharing octahedral tilt angles are 47°.',
 'RbCuO is half-Heusler structured and crystallizes in the cubic F̅43m space group. Rb¹⁺ is bonded to six equivalent Cu¹⁺ and four equivalent O²⁻ atoms to form distorted RbCu₆O₄ tetrahedra that share corners with four equivalent CuRb₆O₄ tetrahedra, corners with six equivalent RbCu₆O₄ tetrahedra, edges with six equivalent CuRb₆O₄ tetrahedra, and faces with twelve equivalent RbCu₆O₄ tetrahedra. All Rb-Cu bond lengths are 3.06 Å. All Rb-O bond lengths are 2.65 Å. Cu¹⁺ is bonded to six equivalent Rb¹⁺ and four equivalent O²⁻ atoms to form distorted CuRb₆O₄ tetrahedra that share corners with four equivalent RbCu₆O₄ tetrahedra, corners with six equivalent CuRb₆O₄ tetrahedra, edges with six equivalent RbCu₆O₄ tetrahedra, and faces with twelve equivalent CuRb₆O₄ tetrahedra. All Cu-O bond lengths are 2.65 Å. O²⁻ is bonded in a body-centered cubic geometry to four equivalent Rb¹⁺ and four equivalent Cu¹⁺ atoms.',
 'B₄C crystallizes in the triclinic P1 space group. There are fifty-two inequivalent B sites. In the first B site, B is bonded in a 6-coordinate geometry to six B atoms. There are a spread of B-B bond distances ranging from 1.69-1.85 Å. In the second B site, B is bonded in a 6-coordinate geometry to six B atoms. There are a spread of B-B bond distances ranging from 1.70-1.85 Å. In the third B site, B is bonded in a 6-coordinate geometry to six B atoms. There is one shorter (1.69 Å) and three longer (1.78 Å) B-B bond lengths. In the fourth B site, B is bonded in a 6-coordinate geometry to six B atoms. There are a spread of B-B bond distances ranging from 1.69-1.87 Å. In the fifth B site, B is bonded in a 6-coordinate geometry to six B atoms. There are a spread of B-B bond distances ranging from 1.69-1.87 Å. In the sixth B site, B is bonded in a single-bond geometry to two B and one C atom. The B-B bond length is 1.79 Å. The B-C bond length is 1.54 Å. In the seventh B site, B is bonded in a single-bond geometry to three B and one C atom. The B-B bond length is 1.77 Å. The B-C bond length is 1.54 Å. In the eighth B site, B is bonded in a distorted single-bond geometry to two B and one C atom. The B-C bond length is 1.67 Å. In the ninth B site, B is bonded in a single-bond geometry to three B and one C atom. The B-B bond length is 1.77 Å. The B-C bond length is 1.54 Å. In the tenth B site, B is bonded in a 6-coordinate geometry to five B and one C atom. There are a spread of B-B bond distances ranging from 1.70-1.83 Å. The B-C bond length is 1.82 Å. In the eleventh B site, B is bonded in a single-bond geometry to two B and one C atom. The B-B bond length is 1.79 Å. The B-C bond length is 1.55 Å. In the twelfth B site, B is bonded in a 2-coordinate geometry to two B and two C atoms. There is one shorter (1.77 Å) and one longer (1.80 Å) B-B bond length. There is one shorter (1.56 Å) and one longer (1.74 Å) B-C bond length. In the thirteenth B site, B is bonded in a single-bond geometry to two B and one C atom. There is one shorter (1.78 Å) and one longer (1.80 Å) B-B bond length. The B-C bond length is 1.55 Å. In the fourteenth B site, B is bonded in a 6-coordinate geometry to five B and one C atom. There are a spread of B-B bond distances ranging from 1.69-1.79 Å. The B-C bond length is 1.81 Å. In the fifteenth B site, B is bonded in a distorted bent 120 degrees geometry to two B and two C atoms. There is one shorter (1.56 Å) and one longer (1.74 Å) B-C bond length. In the sixteenth B site, B is bonded in a single-bond geometry to three B and one C atom. Both B-B bond lengths are 1.80 Å. The B-C bond length is 1.54 Å. In the seventeenth B site, B is bonded in a 2-coordinate geometry to two B and two C atoms. The B-B bond length is 1.78 Å. There is one shorter (1.55 Å) and one longer (1.73 Å) B-C bond length. In the eighteenth B site, B is bonded in a 6-coordinate geometry to five B and one C atom. There are a spread of B-B bond distances ranging from 1.69-1.83 Å. The B-C bond length is 1.80 Å. In the nineteenth B site, B is bonded in a single-bond geometry to three B and one C atom. There is one shorter (1.78 Å) and one longer (1.79 Å) B-B bond length. The B-C bond length is 1.55 Å. In the twentieth B site, B is bonded in a single-bond geometry to two B and one C atom. The B-C bond length is 1.55 Å. In the twenty-first B site, B is bonded in a single-bond geometry to two B and one C atom. The B-B bond length is 1.77 Å. The B-C bond length is 1.55 Å. In the twenty-second B site, B is bonded in a 6-coordinate geometry to five B and one C atom. There are a spread of B-B bond distances ranging from 1.69-1.78 Å. The B-C bond length is 1.80 Å. In the twenty-third B site, B is bonded in a 2-coordinate geometry to two B and two C atoms. The B-B bond length is 1.79 Å. There is one shorter (1.55 Å) and one longer (1.73 Å) B-C bond length. In the twenty-fourth B site, B is bonded in a distorted bent 120 degrees geometry to two B and two C atoms. There is one shorter (1.78 Å) and one longer (1.80 Å) B-B bond length. There is one shorter (1.56 Å) and one longer (1.74 Å) B-C bond length. In the twenty-fifth B site, B is bonded in a 2-coordinate geometry to two B and two C atoms. The B-B bond length is 1.77 Å. There is one shorter (1.56 Å) and one longer (1.74 Å) B-C bond length. In the twenty-sixth B site, B is bonded in a 6-coordinate geometry to five B and one C atom. The B-B bond length is 1.83 Å. The B-C bond length is 1.82 Å. In the twenty-seventh B site, B is bonded in a 6-coordinate geometry to five B and one C atom. The B-C bond length is 1.81 Å. In the twenty-eighth B site, B is bonded in a 6-coordinate geometry to five B and one C atom. There are a spread of B-B bond distances ranging from 1.69-1.83 Å. The B-C bond length is 1.79 Å. In the twenty-ninth B site, B is bonded in a 2-coordinate geometry to two B and two C atoms. The B-B bond length is 1.78 Å. There is one shorter (1.55 Å) and one longer (1.73 Å) B-C bond length. In the thirtieth B site, B is bonded in a single-bond geometry to two equivalent B and one C atom. The B-C bond length is 1.55 Å. In the thirty-first B site, B is bonded in a single-bond geometry to three B and one C atom. There is one shorter (1.78 Å) and one longer (1.80 Å) B-B bond length. The B-C bond length is 1.54 Å. In the thirty-second B site, B is bonded in a 6-coordinate geometry to six B atoms. There are a spread of B-B bond distances ranging from 1.79-1.87 Å. In the thirty-third B site, B is bonded in a 2-coordinate geometry to two B and two C atoms. The B-B bond length is 1.78 Å. There is one shorter (1.55 Å) and one longer (1.73 Å) B-C bond length. In the thirty-fourth B site, B is bonded in a single-bond geometry to three B and one C atom. There is one shorter (1.78 Å) and one longer (1.80 Å) B-B bond length. The B-C bond length is 1.54 Å. In the thirty-fifth B site, B is bonded in a single-bond geometry to two B and one C atom. The B-C bond length is 1.55 Å. In the thirty-sixth B site, B is bonded in a 6-coordinate geometry to six B atoms. The B-B bond length is 1.87 Å. In the thirty-seventh B site, B is bonded in a distorted single-bond geometry to two B and one C atom. The B-C bond length is 1.67 Å. In the thirty-eighth B site, B is bonded in a 2-coordinate geometry to two B and two C atoms. The B-B bond length is 1.78 Å. There is one shorter (1.55 Å) and one longer (1.74 Å) B-C bond length. In the thirty-ninth B site, B is bonded in a single-bond geometry to two B and one C atom. The B-C bond length is 1.53 Å. In the fortieth B site, B is bonded in a single-bond geometry to two B and one C atom. The B-B bond length is 1.78 Å. The B-C bond length is 1.54 Å. In the forty-first B site, B is bonded in a 6-coordinate geometry to six B atoms. There is one shorter (1.77 Å) and two longer (1.84 Å) B-B bond lengths. In the forty-second B site, B is bonded in a distorted bent 120 degrees geometry to two B and two C atoms. The B-B bond length is 1.77 Å. There is one shorter (1.55 Å) and one longer (1.73 Å) B-C bond length. In the forty-third B site, B is bonded in a 6-coordinate geometry to six B atoms. The B-B bond length is 1.84 Å. In the forty-fourth B site, B is bonded in a 6-coordinate geometry to six B atoms. In the forty-fifth B site, B is bonded in a single-bond geometry to two B and one C atom. The B-B bond length is 1.79 Å. The B-C bond length is 1.54 Å. In the forty-sixth B site, B is bonded in a 6-coordinate geometry to six B atoms. There are a spread of B-B bond distances ranging from 1.77-1.86 Å. In the forty-seventh B site, B is bonded in a distorted bent 120 degrees geometry to two equivalent B and two C atoms. There is one shorter (1.55 Å) and one longer (1.71 Å) B-C bond length. In the forty-eighth B site, B is bonded in a single-bond geometry to two B and one C atom. The B-B bond length is 1.79 Å. The B-C bond length is 1.53 Å. In the forty-ninth B site, B is bonded in a single-bond geometry to two equivalent B and one C atom. The B-C bond length is 1.63 Å. In the fiftieth B site, B is bonded in a 6-coordinate geometry to six B atoms. There is one shorter (1.84 Å) and one longer (1.87 Å) B-B bond length. In the fifty-first B site, B is bonded in a 6-coordinate geometry to six B atoms. The B-B bond length is 1.86 Å. In the fifty-second B site, B is bonded in a single-bond geometry to two B and one C atom. The B-C bond length is 1.63 Å. There are thirteen inequivalent C sites. In the first C site, C is bonded in a trigonal planar geometry to three B atoms. In the second C site, C is bonded in a 6-coordinate geometry to six B atoms. In the third C site, C is bonded in a trigonal planar geometry to three B atoms. In the fourth C site, C is bonded in a trigonal planar geometry to three B atoms. In the fifth C site, C is bonded in a 6-coordinate geometry to six B atoms. In the sixth C site, C is bonded in a 6-coordinate geometry to six B atoms. In the seventh C site, C is bonded in a trigonal planar geometry to three B atoms. In the eighth C site, C is bonded in a 6-coordinate geometry to six B atoms. In the ninth C site, C is bonded in a trigonal planar geometry to three B atoms. In the tenth C site, C is bonded in a trigonal planar geometry to three B atoms. In the eleventh C site, C is bonded in a trigonal planar geometry to three B atoms. In the twelfth C site, C is bonded in a trigonal planar geometry to three B atoms. In the thirteenth C site, C is bonded in a trigonal planar geometry to three B atoms.',
 'ErAg is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Er is bonded in a body-centered cubic geometry to eight equivalent Ag atoms. All Er-Ag bond lengths are 3.10 Å. Ag is bonded in a body-centered cubic geometry to eight equivalent Er atoms.',
 'TiOs₃ is Uranium Silicide structured and crystallizes in the cubic Pm̅3m space group. Ti is bonded to twelve equivalent Os atoms to form TiOs₁₂ cuboctahedra that share corners with twelve equivalent TiOs₁₂ cuboctahedra, edges with twenty-four equivalent OsTi₄Os₈ cuboctahedra, faces with six equivalent TiOs₁₂ cuboctahedra, and faces with twelve equivalent OsTi₄Os₈ cuboctahedra. All Ti-Os bond lengths are 2.75 Å. Os is bonded to four equivalent Ti and eight equivalent Os atoms to form distorted OsTi₄Os₈ cuboctahedra that share corners with twelve equivalent OsTi₄Os₈ cuboctahedra, edges with eight equivalent TiOs₁₂ cuboctahedra, edges with sixteen equivalent OsTi₄Os₈ cuboctahedra, faces with four equivalent TiOs₁₂ cuboctahedra, and faces with fourteen equivalent OsTi₄Os₈ cuboctahedra. All Os-Os bond lengths are 2.75 Å.',
 'VSi₂ is Titanium Disilicide-like structured and crystallizes in the hexagonal P6_422 space group. V is bonded in a distorted q6 geometry to ten equivalent Si atoms. There are a spread of V-Si bond distances ranging from 2.49-2.65 Å. Si is bonded in a 10-coordinate geometry to five equivalent V and five equivalent Si atoms. There are a spread of Si-Si bond distances ranging from 2.46-2.65 Å.',
 'PtN is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Pt²⁺ is bonded to four equivalent N²⁻ atoms to form corner-sharing PtN₄ tetrahedra. All Pt-N bond lengths are 2.07 Å. N²⁻ is bonded to four equivalent Pt²⁺ atoms to form corner-sharing NPt₄ tetrahedra.',
 'HoSi₂ is hexagonal omega structure structured and crystallizes in the hexagonal P6/mmm space group. Ho is bonded to twelve equivalent Si atoms to form a mixture of edge and face-sharing HoSi₁₂ cuboctahedra. All Ho-Si bond lengths are 3.07 Å. Si is bonded in a 9-coordinate geometry to six equivalent Ho and three equivalent Si atoms. All Si-Si bond lengths are 2.36 Å.',
 'CuAl is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Cu is bonded in a body-centered cubic geometry to eight equivalent Al atoms. All Cu-Al bond lengths are 2.60 Å. Al is bonded in a body-centered cubic geometry to eight equivalent Cu atoms.',
 'GdB₂ is hexagonal omega structure structured and crystallizes in the hexagonal P6/mmm space group. Gd is bonded to twelve equivalent B atoms to form a mixture of edge and face-sharing GdB₁₂ cuboctahedra. All Gd-B bond lengths are 2.74 Å. B is bonded in a 9-coordinate geometry to six equivalent Gd and three equivalent B atoms. All B-B bond lengths are 1.91 Å.',
 'RhGa is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Rh is bonded in a body-centered cubic geometry to eight equivalent Ga atoms. All Rh-Ga bond lengths are 2.60 Å. Ga is bonded in a body-centered cubic geometry to eight equivalent Rh atoms.',
 'Bi₄Ge₃O₁₂ crystallizes in the cubic I̅43d space group. Ge⁴⁺ is bonded in a tetrahedral geometry to four equivalent O²⁻ atoms. All Ge-O bond lengths are 1.76 Å. Bi³⁺ is bonded in a 6-coordinate geometry to six equivalent O²⁻ atoms. There are three shorter (2.19 Å) and three longer (2.58 Å) Bi-O bond lengths. O²⁻ is bonded in a 3-coordinate geometry to one Ge⁴⁺ and two equivalent Bi³⁺ atoms.',
 'Hg crystallizes in the hexagonal P6/mmm space group. The structure consists of one Hg ribbon oriented in the (0, 0, 1) direction inside a Hg framework. In the Hg ribbon, Hg is bonded in a linear geometry to two equivalent Hg atoms. Both Hg-Hg bond lengths are 3.19 Å. In the Hg framework, Hg is bonded to five equivalent Hg atoms to form a mixture of corner and edge-sharing HgHg₅ trigonal bipyramids. There are three shorter (3.16 Å) and two longer (3.19 Å) Hg-Hg bond lengths.',
 'NiNiGa crystallizes in the hexagonal P6₃/mmc space group. There are two inequivalent Ni sites. In the first Ni site, Ni is bonded in a 14-coordinate geometry to eight Ni and six equivalent Ga atoms. There are two shorter (2.48 Å) and six longer (2.62 Å) Ni-Ni bond lengths. All Ni-Ga bond lengths are 2.62 Å. In the second Ni site, Ni is bonded to six equivalent Ni and five equivalent Ga atoms to form a mixture of distorted corner and face-sharing NiGa₅Ni₆ trigonal bipyramids. There are three shorter (2.31 Å) and two longer (2.48 Å) Ni-Ga bond lengths. Ga is bonded in a 11-coordinate geometry to eleven Ni atoms.',
 'Ni₂In crystallizes in the hexagonal P6₃/mmc space group. There are two inequivalent Ni sites. In the first Ni site, Ni is bonded in a 14-coordinate geometry to eight Ni and six equivalent In atoms. There are two shorter (2.63 Å) and six longer (2.81 Å) Ni-Ni bond lengths. All Ni-In bond lengths are 2.81 Å. In the second Ni site, Ni is bonded to six equivalent Ni and five equivalent In atoms to form a mixture of distorted corner and face-sharing NiIn₅Ni₆ trigonal bipyramids. There are three shorter (2.49 Å) and two longer (2.63 Å) Ni-In bond lengths. In is bonded in a 11-coordinate geometry to eleven Ni atoms.',
 'Mg₃Ca crystallizes in the hexagonal P6₃/mmc space group. Ca is bonded to twelve equivalent Mg atoms to form a mixture of corner and face-sharing CaMg₁₂ cuboctahedra. There are six shorter (3.35 Å) and six longer (3.41 Å) Ca-Mg bond lengths. Mg is bonded in a 10-coordinate geometry to four equivalent Ca and six equivalent Mg atoms. There are four shorter (3.26 Å) and two longer (3.27 Å) Mg-Mg bond lengths.',
 'Fe₂Ge crystallizes in the hexagonal P6/mmm space group. Fe²⁺ is bonded in a 9-coordinate geometry to three equivalent Fe²⁺ and six equivalent Ge⁴⁻ atoms. All Fe-Fe bond lengths are 2.40 Å. All Fe-Ge bond lengths are 2.74 Å. Ge⁴⁻ is bonded in a 14-coordinate geometry to twelve equivalent Fe²⁺ and two equivalent Ge⁴⁻ atoms. Both Ge-Ge bond lengths are 2.63 Å.',
 'CuN₆ crystallizes in the orthorhombic Pnma space group. Cu²⁺ is bonded to six N+0.33- atoms to form distorted edge-sharing CuN₆ octahedra. There are a spread of Cu-N bond distances ranging from 2.01-2.75 Å. There are six inequivalent N+0.33- sites. In the first N+0.33- site, N+0.33- is bonded in a linear geometry to two N+0.33- atoms. There is one shorter (1.16 Å) and one longer (1.23 Å) N-N bond length. In the second N+0.33- site, N+0.33- is bonded in a bent 150 degrees geometry to one Cu²⁺ and one N+0.33- atom. The N-N bond length is 1.16 Å. In the third N+0.33- site, N+0.33- is bonded in a distorted trigonal planar geometry to two equivalent Cu²⁺ and one N+0.33- atom. The N-N bond length is 1.22 Å. In the fourth N+0.33- site, N+0.33- is bonded in a 4-coordinate geometry to three equivalent Cu²⁺ and one N+0.33- atom. In the fifth N+0.33- site, N+0.33- is bonded in a single-bond geometry to one N+0.33- atom. In the sixth N+0.33- site, N+0.33- is bonded in a linear geometry to two N+0.33- atoms.',
 'MgHg₂ crystallizes in the tetragonal I4/mmm space group. Mg is bonded in a distorted q6 geometry to ten equivalent Hg atoms. There are two shorter (2.93 Å) and eight longer (3.10 Å) Mg-Hg bond lengths. Hg is bonded in a 10-coordinate geometry to five equivalent Mg and five equivalent Hg atoms. There is one shorter (2.93 Å) and four longer (3.10 Å) Hg-Hg bond lengths.',
 'BaI₂ crystallizes in the hexagonal P̅62m space group. There are two inequivalent Ba²⁺ sites. In the first Ba²⁺ site, Ba²⁺ is bonded in a 9-coordinate geometry to nine I¹⁻ atoms. There are six shorter (3.54 Å) and three longer (3.76 Å) Ba-I bond lengths. In the second Ba²⁺ site, Ba²⁺ is bonded in a 9-coordinate geometry to nine I¹⁻ atoms. There are three shorter (3.49 Å) and six longer (3.84 Å) Ba-I bond lengths. There are two inequivalent I¹⁻ sites. In the first I¹⁻ site, I¹⁻ is bonded to five Ba²⁺ atoms to form distorted IBa₅ trigonal bipyramids that share corners with six equivalent IBa₄ tetrahedra, corners with ten equivalent IBa₅ trigonal bipyramids, edges with six equivalent IBa₄ tetrahedra, and edges with six equivalent IBa₅ trigonal bipyramids. In the second I¹⁻ site, I¹⁻ is bonded to four Ba²⁺ atoms to form distorted IBa₄ tetrahedra that share corners with ten equivalent IBa₄ tetrahedra, corners with six equivalent IBa₅ trigonal bipyramids, edges with two equivalent IBa₄ tetrahedra, and edges with six equivalent IBa₅ trigonal bipyramids.',
 'Hf₃Bi is Uranium Silicide-like structured and crystallizes in the tetragonal I4/mmm space group. There are two inequivalent Hf sites. In the first Hf site, Hf is bonded to eight Hf and four equivalent Bi atoms to form distorted HfHf₈Bi₄ cuboctahedra that share corners with twelve equivalent HfHf₈Bi₄ cuboctahedra, edges with eight equivalent HfHf₈Bi₄ cuboctahedra, edges with eight equivalent BiHf₁₂ cuboctahedra, faces with four equivalent BiHf₁₂ cuboctahedra, and faces with ten equivalent HfHf₈Bi₄ cuboctahedra. There are four shorter (3.11 Å) and four longer (3.24 Å) Hf-Hf bond lengths. All Hf-Bi bond lengths are 3.24 Å. In the second Hf site, Hf is bonded in a distorted square co-planar geometry to eight equivalent Hf and four equivalent Bi atoms. All Hf-Bi bond lengths are 3.11 Å. Bi is bonded to twelve Hf atoms to form BiHf₁₂ cuboctahedra that share corners with four equivalent BiHf₁₂ cuboctahedra, edges with eight equivalent BiHf₁₂ cuboctahedra, edges with sixteen equivalent HfHf₈Bi₄ cuboctahedra, faces with four equivalent BiHf₁₂ cuboctahedra, and faces with eight equivalent HfHf₈Bi₄ cuboctahedra.',
 'Ca₅Au₄ crystallizes in the monoclinic P2₁/c space group. There are three inequivalent Ca sites. In the first Ca site, Ca is bonded in a 7-coordinate geometry to seven Au atoms. There are a spread of Ca-Au bond distances ranging from 3.13-3.51 Å. In the second Ca site, Ca is bonded to six Au atoms to form a mixture of distorted corner, edge, and face-sharing CaAu₆ pentagonal pyramids. The corner-sharing octahedral tilt angles range from 37-45°. There are a spread of Ca-Au bond distances ranging from 2.96-3.08 Å. In the third Ca site, Ca is bonded to six Au atoms to form a mixture of corner and face-sharing CaAu₆ octahedra. The corner-sharing octahedral tilt angles are 52°. There are a spread of Ca-Au bond distances ranging from 3.08-3.20 Å. There are two inequivalent Au sites. In the first Au site, Au is bonded in a 9-coordinate geometry to eight Ca and one Au atom. The Au-Au bond length is 2.86 Å. In the second Au site, Au is bonded in a 9-coordinate geometry to eight Ca and one Au atom. The Au-Au bond length is 2.97 Å.',
 'CoGa is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Co is bonded in a body-centered cubic geometry to eight equivalent Ga atoms. All Co-Ga bond lengths are 2.47 Å. Ga is bonded in a body-centered cubic geometry to eight equivalent Co atoms.',
 'CeMgNi₄ is Hexagonal Laves-derived structured and crystallizes in the cubic F̅43m space group. Mg is bonded in a 16-coordinate geometry to four equivalent Ce and twelve equivalent Ni atoms. All Mg-Ce bond lengths are 3.01 Å. All Mg-Ni bond lengths are 2.88 Å. Ce is bonded in a 16-coordinate geometry to four equivalent Mg and twelve equivalent Ni atoms. All Ce-Ni bond lengths are 2.89 Å. Ni is bonded to three equivalent Mg, three equivalent Ce, and six equivalent Ni atoms to form a mixture of corner, edge, and face-sharing NiCe₃Mg₃Ni₆ cuboctahedra. There are three shorter (2.43 Å) and three longer (2.49 Å) Ni-Ni bond lengths.',
 'TeO₂ crystallizes in the tetragonal P4_32_12 space group. Te⁴⁺ is bonded in a 4-coordinate geometry to four equivalent O²⁻ atoms. There are two shorter (1.89 Å) and two longer (2.12 Å) Te-O bond lengths. O²⁻ is bonded in a 2-coordinate geometry to two equivalent Te⁴⁺ atoms.',
 'Cu₂Cd is Hexagonal Laves structured and crystallizes in the hexagonal P6₃/mmc space group. There are two inequivalent Cu sites. In the first Cu site, Cu is bonded to six equivalent Cu and six equivalent Cd atoms to form a mixture of corner, edge, and face-sharing CuCd₆Cu₆ cuboctahedra. All Cu-Cu bond lengths are 2.46 Å. All Cu-Cd bond lengths are 2.93 Å. In the second Cu site, Cu is bonded to six Cu and six equivalent Cd atoms to form a mixture of corner, edge, and face-sharing CuCd₆Cu₆ cuboctahedra. There are two shorter (2.47 Å) and two longer (2.54 Å) Cu-Cu bond lengths. There are two shorter (2.85 Å) and four longer (2.92 Å) Cu-Cd bond lengths. Cd is bonded in a 16-coordinate geometry to twelve Cu and four equivalent Cd atoms. There is one shorter (2.99 Å) and three longer (3.05 Å) Cd-Cd bond lengths.',
 'MnPd is Tetraauricupride structured and crystallizes in the tetragonal P4/mmm space group. Mn is bonded in a body-centered cubic geometry to eight equivalent Pd atoms. All Mn-Pd bond lengths are 2.70 Å. Pd is bonded in a body-centered cubic geometry to eight equivalent Mn atoms.',
 'Al₂Te₃ crystallizes in the monoclinic P2₁/c space group. The structure is two-dimensional and consists of two Al₂Te₃ sheets oriented in the (0, 0, 1) direction. There are four inequivalent Al³⁺ sites. In the first Al³⁺ site, Al³⁺ is bonded to four Te²⁻ atoms to form a mixture of corner and edge-sharing AlTe₄ tetrahedra. There are a spread of Al-Te bond distances ranging from 2.55-2.69 Å. In the second Al³⁺ site, Al³⁺ is bonded to four Te²⁻ atoms to form a mixture of corner and edge-sharing AlTe₄ tetrahedra. There are a spread of Al-Te bond distances ranging from 2.57-2.68 Å. In the third Al³⁺ site, Al³⁺ is bonded to four Te²⁻ atoms to form a mixture of corner and edge-sharing AlTe₄ tetrahedra. There are a spread of Al-Te bond distances ranging from 2.56-2.69 Å. In the fourth Al³⁺ site, Al³⁺ is bonded to four Te²⁻ atoms to form a mixture of corner and edge-sharing AlTe₄ tetrahedra. There are a spread of Al-Te bond distances ranging from 2.57-2.68 Å. There are six inequivalent Te²⁻ sites. In the first Te²⁻ site, Te²⁻ is bonded in a trigonal non-coplanar geometry to three Al³⁺ atoms. In the second Te²⁻ site, Te²⁻ is bonded in a 3-coordinate geometry to three Al³⁺ atoms. In the third Te²⁻ site, Te²⁻ is bonded in a distorted trigonal non-coplanar geometry to three Al³⁺ atoms. In the fourth Te²⁻ site, Te²⁻ is bonded in a distorted L-shaped geometry to two Al³⁺ atoms. In the fifth Te²⁻ site, Te²⁻ is bonded in a 3-coordinate geometry to three Al³⁺ atoms. In the sixth Te²⁻ site, Te²⁻ is bonded in a distorted L-shaped geometry to two Al³⁺ atoms.',
 'HgSe is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Hg²⁺ is bonded to four equivalent Se²⁻ atoms to form corner-sharing HgSe₄ tetrahedra. All Hg-Se bond lengths are 2.68 Å. Se²⁻ is bonded to four equivalent Hg²⁺ atoms to form corner-sharing SeHg₄ tetrahedra.',
 'ZnN is Wurtzite structured and crystallizes in the hexagonal P6₃mc space group. Zn²⁺ is bonded to four equivalent N²⁻ atoms to form corner-sharing ZnN₄ tetrahedra. There is one shorter (1.95 Å) and three longer (1.97 Å) Zn-N bond lengths. N²⁻ is bonded to four equivalent Zn²⁺ atoms to form corner-sharing NZn₄ tetrahedra.',
 'ScN is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Sc³⁺ is bonded to four equivalent N³⁻ atoms to form corner-sharing ScN₄ tetrahedra. All Sc-N bond lengths are 2.12 Å. N³⁻ is bonded to four equivalent Sc³⁺ atoms to form corner-sharing NSc₄ tetrahedra.',
 'TiCd crystallizes in the tetragonal P4/nmm space group. Ti is bonded to eight equivalent Ti and four equivalent Cd atoms to form TiTi₈Cd₄ cuboctahedra that share corners with four equivalent TiTi₈Cd₄ cuboctahedra, corners with eight equivalent CdTi₄Cd₈ cuboctahedra, edges with eight equivalent TiTi₈Cd₄ cuboctahedra, edges with sixteen equivalent CdTi₄Cd₈ cuboctahedra, faces with six equivalent CdTi₄Cd₈ cuboctahedra, and faces with twelve equivalent TiTi₈Cd₄ cuboctahedra. There are four shorter (2.83 Å) and four longer (2.89 Å) Ti-Ti bond lengths. All Ti-Cd bond lengths are 3.09 Å. Cd is bonded to four equivalent Ti and eight equivalent Cd atoms to form CdTi₄Cd₈ cuboctahedra that share corners with four equivalent CdTi₄Cd₈ cuboctahedra, corners with eight equivalent TiTi₈Cd₄ cuboctahedra, edges with eight equivalent CdTi₄Cd₈ cuboctahedra, edges with sixteen equivalent TiTi₈Cd₄ cuboctahedra, faces with six equivalent TiTi₈Cd₄ cuboctahedra, and faces with twelve equivalent CdTi₄Cd₈ cuboctahedra. There are four shorter (2.89 Å) and four longer (3.15 Å) Cd-Cd bond lengths.',
 'TiI₃ crystallizes in the orthorhombic Pmmn space group. The structure is one-dimensional and consists of two TiI₃ ribbons oriented in the (1, 0, 0) direction. Ti³⁺ is bonded to six I¹⁻ atoms to form face-sharing TiI₆ octahedra. There are three shorter (2.74 Å) and three longer (2.86 Å) Ti-I bond lengths. There are four inequivalent I¹⁻ sites. In the first I¹⁻ site, I¹⁻ is bonded in a 2-coordinate geometry to two equivalent Ti³⁺ atoms. In the second I¹⁻ site, I¹⁻ is bonded in a distorted L-shaped geometry to two equivalent Ti³⁺ atoms. In the third I¹⁻ site, I¹⁻ is bonded in a 2-coordinate geometry to two equivalent Ti³⁺ atoms. In the fourth I¹⁻ site, I¹⁻ is bonded in a distorted L-shaped geometry to two equivalent Ti³⁺ atoms.',
 'MoSi₃ crystallizes in the hexagonal P6₃/mmc space group. Mo⁴⁺ is bonded to twelve equivalent Si+1.33- atoms to form a mixture of corner and face-sharing MoSi₁₂ cuboctahedra. There are six shorter (2.61 Å) and six longer (2.81 Å) Mo-Si bond lengths. Si+1.33- is bonded in a 12-coordinate geometry to four equivalent Mo⁴⁺ and eight equivalent Si+1.33- atoms. There are a spread of Si-Si bond distances ranging from 2.48-2.89 Å.',
 'CaGe₂ crystallizes in the trigonal R̅3m space group. Ca is bonded in a 7-coordinate geometry to seven Ge atoms. There are a spread of Ca-Ge bond distances ranging from 3.04-3.18 Å. There are two inequivalent Ge sites. In the first Ge site, Ge is bonded to three equivalent Ca and three equivalent Ge atoms to form distorted edge-sharing GeCa₃Ge₃ octahedra. All Ge-Ge bond lengths are 2.56 Å. In the second Ge site, Ge is bonded in a 7-coordinate geometry to four equivalent Ca and three equivalent Ge atoms. All Ge-Ge bond lengths are 2.60 Å.',
 'PdBi is Molybdenum Carbide MAX Phase-like structured and crystallizes in the hexagonal P6₃/mmc space group. Pd³⁺ is bonded to six equivalent Bi³⁻ atoms to form a mixture of corner, edge, and face-sharing PdBi₆ octahedra. The corner-sharing octahedral tilt angles are 51°. All Pd-Bi bond lengths are 2.85 Å. Bi³⁻ is bonded in a 6-coordinate geometry to six equivalent Pd³⁺ atoms.',
 'O₂ is beta oxygen structured and crystallizes in the trigonal R̅3m space group. The structure is zero-dimensional and consists of three O₂ clusters. O is bonded in a single-bond geometry to one O atom. The O-O bond length is 1.22 Å.',
 'Re₂RhPt is Heusler structured and crystallizes in the cubic Fm̅3m space group. Re is bonded in a body-centered cubic geometry to four equivalent Rh and four equivalent Pt atoms. All Re-Rh bond lengths are 2.71 Å. All Re-Pt bond lengths are 2.71 Å. Rh is bonded in a body-centered cubic geometry to eight equivalent Re atoms. Pt is bonded in a body-centered cubic geometry to eight equivalent Re atoms.',
 'YbBr₂ is trigonal omega-like structured and crystallizes in the tetragonal P4₂/mnm space group. Yb²⁺ is bonded to six equivalent Br¹⁻ atoms to form a mixture of corner and edge-sharing YbBr₆ octahedra. The corner-sharing octahedral tilt angles are 39°. There are two shorter (2.82 Å) and four longer (2.84 Å) Yb-Br bond lengths. Br¹⁻ is bonded in a 3-coordinate geometry to three equivalent Yb²⁺ atoms.',
 'Zr₂Co is Khatyrkite structured and crystallizes in the tetragonal I4/mcm space group. Zr is bonded in a 4-coordinate geometry to four equivalent Co atoms. All Zr-Co bond lengths are 2.71 Å. Co is bonded in a 8-coordinate geometry to eight equivalent Zr atoms.',
 'Ta is Copper structured and crystallizes in the cubic Fm̅3m space group. Ta is bonded to twelve equivalent Ta atoms to form a mixture of corner, edge, and face-sharing TaTa₁₂ cuboctahedra. All Ta-Ta bond lengths are 2.95 Å.',
 'TiO₂ is beta Vanadium nitride-like structured and crystallizes in the tetragonal I4₁/amd space group. Ti⁴⁺ is bonded to six equivalent O²⁻ atoms to form a mixture of distorted corner and edge-sharing TiO₆ octahedra. The corner-sharing octahedral tilt angles are 25°. There are four shorter (1.94 Å) and two longer (1.99 Å) Ti-O bond lengths. O²⁻ is bonded in a distorted T-shaped geometry to three equivalent Ti⁴⁺ atoms.',
 'UPb is Tetraauricupride structured and crystallizes in the tetragonal P4/mmm space group. U is bonded in a distorted body-centered cubic geometry to eight equivalent Pb atoms. All U-Pb bond lengths are 3.35 Å. Pb is bonded in a distorted body-centered cubic geometry to eight equivalent U atoms.',
 'RuN is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Ru³⁺ is bonded to four equivalent N³⁻ atoms to form corner-sharing RuN₄ tetrahedra. All Ru-N bond lengths are 1.98 Å. N³⁻ is bonded to four equivalent Ru³⁺ atoms to form corner-sharing NRu₄ tetrahedra.',
 'NpAs is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Np is bonded in a body-centered cubic geometry to eight equivalent As atoms. All Np-As bond lengths are 3.03 Å. As is bonded in a body-centered cubic geometry to eight equivalent Np atoms.',
 'LaBr crystallizes in the trigonal R̅3m space group. The structure is two-dimensional and consists of six LaBr sheets oriented in the (0, 0, 1) direction. La¹⁺ is bonded in a distorted T-shaped geometry to three equivalent Br¹⁻ atoms. All La-Br bond lengths are 3.07 Å. Br¹⁻ is bonded in a distorted T-shaped geometry to three equivalent La¹⁺ atoms.',
 'Rb₃Ca is beta Cu3Ti-like structured and crystallizes in the tetragonal I4/mmm space group. There are two inequivalent Rb sites. In the first Rb site, Rb is bonded to eight Rb and four equivalent Ca atoms to form distorted RbRb₈Ca₄ cuboctahedra that share corners with twelve equivalent RbRb₈Ca₄ cuboctahedra, edges with eight equivalent CaRb₁₂ cuboctahedra, edges with sixteen RbRb₈Ca₄ cuboctahedra, faces with four equivalent CaRb₁₂ cuboctahedra, and faces with fourteen RbRb₈Ca₄ cuboctahedra. There are four shorter (4.66 Å) and four longer (4.74 Å) Rb-Rb bond lengths. All Rb-Ca bond lengths are 4.74 Å. In the second Rb site, Rb is bonded to eight equivalent Rb and four equivalent Ca atoms to form RbRb₈Ca₄ cuboctahedra that share corners with four equivalent RbRb₈Ca₄ cuboctahedra, corners with eight equivalent CaRb₁₂ cuboctahedra, edges with twenty-four RbRb₈Ca₄ cuboctahedra, faces with six equivalent CaRb₁₂ cuboctahedra, and faces with twelve RbRb₈Ca₄ cuboctahedra. All Rb-Ca bond lengths are 4.66 Å. Ca is bonded to twelve Rb atoms to form distorted CaRb₁₂ cuboctahedra that share corners with four equivalent CaRb₁₂ cuboctahedra, corners with eight equivalent RbRb₈Ca₄ cuboctahedra, edges with eight equivalent CaRb₁₂ cuboctahedra, edges with sixteen equivalent RbRb₈Ca₄ cuboctahedra, faces with four equivalent CaRb₁₂ cuboctahedra, and faces with fourteen RbRb₈Ca₄ cuboctahedra.',
 'RhBi is Tungsten Carbide-like structured and crystallizes in the hexagonal P6₃/mmc space group. Rh³⁺ is bonded in a 6-coordinate geometry to six equivalent Bi³⁻ atoms. All Rh-Bi bond lengths are 2.87 Å. Bi³⁻ is bonded in a 6-coordinate geometry to six equivalent Rh³⁺ atoms.',
 'PtCd is Tetraauricupride structured and crystallizes in the tetragonal P4/mmm space group. Pt is bonded in a distorted body-centered cubic geometry to eight equivalent Cd atoms. All Pt-Cd bond lengths are 2.86 Å. Cd is bonded in a distorted body-centered cubic geometry to eight equivalent Pt atoms.',
 'AgI is Wurtzite structured and crystallizes in the hexagonal P6₃mc space group. Ag¹⁺ is bonded to four equivalent I¹⁻ atoms to form corner-sharing AgI₄ tetrahedra. There are three shorter (2.84 Å) and one longer (2.85 Å) Ag-I bond length. I¹⁻ is bonded to four equivalent Ag¹⁺ atoms to form corner-sharing IAg₄ tetrahedra.',
 'BrCl is alpha carbon monoxide-like structured and crystallizes in the hexagonal P6₃mc space group. The structure is zero-dimensional and consists of two BrCl clusters. Br¹⁺ is bonded in a single-bond geometry to one Cl¹⁻ atom. The Br-Cl bond length is 2.17 Å. Cl¹⁻ is bonded in a single-bond geometry to one Br¹⁺ atom.',
 'Tm₃AlN is (Cubic) Perovskite structured and crystallizes in the cubic Pm̅3m space group. Tm is bonded in a linear geometry to four equivalent Al and two equivalent N atoms. All Tm-Al bond lengths are 3.29 Å. Both Tm-N bond lengths are 2.33 Å. Al is bonded to twelve equivalent Tm atoms to form AlTm₁₂ cuboctahedra that share corners with twelve equivalent AlTm₁₂ cuboctahedra, faces with six equivalent AlTm₁₂ cuboctahedra, and faces with eight equivalent NTm₆ octahedra. N is bonded to six equivalent Tm atoms to form NTm₆ octahedra that share corners with six equivalent NTm₆ octahedra and faces with eight equivalent AlTm₁₂ cuboctahedra. The corner-sharing octahedra are not tilted.',
 'SmRh is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Sm is bonded in a body-centered cubic geometry to eight equivalent Rh atoms. All Sm-Rh bond lengths are 3.01 Å. Rh is bonded in a body-centered cubic geometry to eight equivalent Sm atoms.',
 'CdC is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Cd²⁺ is bonded to four equivalent C²⁻ atoms to form corner-sharing CdC₄ tetrahedra. All Cd-C bond lengths are 2.25 Å. C²⁻ is bonded to four equivalent Cd²⁺ atoms to form corner-sharing CCd₄ tetrahedra.',
 'GeSe₂ crystallizes in the tetragonal I̅42d space group. Ge⁴⁺ is bonded to four equivalent Se²⁻ atoms to form corner-sharing GeSe₄ tetrahedra. All Ge-Se bond lengths are 2.37 Å. Se²⁻ is bonded in a water-like geometry to two equivalent Ge⁴⁺ atoms.',
 'LuAu is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Lu is bonded in a body-centered cubic geometry to eight equivalent Au atoms. All Lu-Au bond lengths are 3.03 Å. Au is bonded in a body-centered cubic geometry to eight equivalent Lu atoms.',
 'TiP crystallizes in the hexagonal P6₃/mmc space group. Ti³⁺ is bonded to five equivalent P³⁻ atoms to form a mixture of corner and edge-sharing TiP₅ trigonal bipyramids. All Ti-P bond lengths are 2.44 Å. P³⁻ is bonded to five equivalent Ti³⁺ atoms to form a mixture of corner and edge-sharing PTi₅ trigonal bipyramids.',
 'Cr₃Ni is Uranium Silicide structured and crystallizes in the cubic Pm̅3m space group. Cr is bonded to eight equivalent Cr and four equivalent Ni atoms to form CrCr₈Ni₄ cuboctahedra that share corners with twelve equivalent CrCr₈Ni₄ cuboctahedra, edges with eight equivalent NiCr₁₂ cuboctahedra, edges with sixteen equivalent CrCr₈Ni₄ cuboctahedra, faces with four equivalent NiCr₁₂ cuboctahedra, and faces with fourteen equivalent CrCr₈Ni₄ cuboctahedra. All Cr-Cr bond lengths are 2.52 Å. All Cr-Ni bond lengths are 2.52 Å. Ni is bonded to twelve equivalent Cr atoms to form NiCr₁₂ cuboctahedra that share corners with twelve equivalent NiCr₁₂ cuboctahedra, edges with twenty-four equivalent CrCr₈Ni₄ cuboctahedra, faces with six equivalent NiCr₁₂ cuboctahedra, and faces with twelve equivalent CrCr₈Ni₄ cuboctahedra.',
 'Rb₃Ru is alpha bismuth trifluoride structured and crystallizes in the cubic Fm̅3m space group. There are two inequivalent Rb sites. In the first Rb site, Rb is bonded to four equivalent Rb and four equivalent Ru atoms to form a mixture of distorted corner, edge, and face-sharing RbRb₄Ru₄ tetrahedra. All Rb-Rb bond lengths are 3.91 Å. All Rb-Ru bond lengths are 3.91 Å. In the second Rb site, Rb is bonded in a body-centered cubic geometry to eight equivalent Rb atoms. Ru is bonded in a body-centered cubic geometry to eight equivalent Rb atoms.',
 'LiH is Halite, Rock Salt structured and crystallizes in the cubic Fm̅3m space group. Li¹⁺ is bonded to six equivalent H¹⁻ atoms to form a mixture of corner and edge-sharing LiH₆ octahedra. The corner-sharing octahedra are not tilted. All Li-H bond lengths are 2.01 Å. H¹⁻ is bonded to six equivalent Li¹⁺ atoms to form a mixture of corner and edge-sharing HLi₆ octahedra. The corner-sharing octahedra are not tilted.',
 'EuAg₅ crystallizes in the hexagonal P6/mmm space group. Eu is bonded in a distorted hexagonal planar geometry to eighteen Ag atoms. There are six shorter (3.25 Å) and twelve longer (3.65 Å) Eu-Ag bond lengths. There are two inequivalent Ag sites. In the first Ag site, Ag is bonded in a 12-coordinate geometry to three equivalent Eu and nine Ag atoms. There are six shorter (2.83 Å) and three longer (3.25 Å) Ag-Ag bond lengths. In the second Ag site, Ag is bonded to four equivalent Eu and eight Ag atoms to form a mixture of corner, edge, and face-sharing AgEu₄Ag₈ cuboctahedra. All Ag-Ag bond lengths are 2.82 Å.',
 'LuSi₂ is hexagonal omega structure structured and crystallizes in the hexagonal P6/mmm space group. Lu is bonded to twelve equivalent Si atoms to form a mixture of edge and face-sharing LuSi₁₂ cuboctahedra. All Lu-Si bond lengths are 3.03 Å. Si is bonded in a 9-coordinate geometry to six equivalent Lu and three equivalent Si atoms. All Si-Si bond lengths are 2.36 Å.',
 'LiVS₂ is Caswellsilverite-like structured and crystallizes in the trigonal P̅3m1 space group. Li¹⁺ is bonded to six equivalent S²⁻ atoms to form LiS₆ octahedra that share corners with twelve equivalent VS₆ octahedra, edges with six equivalent LiS₆ octahedra, and faces with two equivalent VS₆ octahedra. The corner-sharing octahedral tilt angles are 47°. All Li-S bond lengths are 2.59 Å. V³⁺ is bonded to six equivalent S²⁻ atoms to form VS₆ octahedra that share corners with twelve equivalent LiS₆ octahedra, edges with six equivalent VS₆ octahedra, and faces with two equivalent LiS₆ octahedra. The corner-sharing octahedral tilt angles are 47°. All V-S bond lengths are 2.44 Å. S²⁻ is bonded to three equivalent Li¹⁺ and three equivalent V³⁺ atoms to form a mixture of distorted corner and edge-sharing SLi₃V₃ pentagonal pyramids.',
 'Re₄Si₇ crystallizes in the monoclinic Cm space group. There are eight inequivalent Re³⁺ sites. In the first Re³⁺ site, Re³⁺ is bonded in a 9-coordinate geometry to nine Si+1.71- atoms. There are a spread of Re-Si bond distances ranging from 2.51-2.67 Å. In the second Re³⁺ site, Re³⁺ is bonded in a 8-coordinate geometry to eight Si+1.71- atoms. There are a spread of Re-Si bond distances ranging from 2.42-2.54 Å. In the third Re³⁺ site, Re³⁺ is bonded in a 8-coordinate geometry to eight Si+1.71- atoms. There are a spread of Re-Si bond distances ranging from 2.49-2.65 Å. In the fourth Re³⁺ site, Re³⁺ is bonded in a distorted q6 geometry to nine Si+1.71- atoms. There are a spread of Re-Si bond distances ranging from 2.50-2.70 Å. In the fifth Re³⁺ site, Re³⁺ is bonded in a 8-coordinate geometry to eight Si+1.71- atoms. There are a spread of Re-Si bond distances ranging from 2.50-2.68 Å. In the sixth Re³⁺ site, Re³⁺ is bonded in a 8-coordinate geometry to eight Si+1.71- atoms. There are a spread of Re-Si bond distances ranging from 2.41-2.53 Å. In the seventh Re³⁺ site, Re³⁺ is bonded in a 9-coordinate geometry to nine Si+1.71- atoms. There are a spread of Re-Si bond distances ranging from 2.50-3.08 Å. In the eighth Re³⁺ site, Re³⁺ is bonded in a 8-coordinate geometry to eight Si+1.71- atoms. There are a spread of Re-Si bond distances ranging from 2.49-2.60 Å. There are fourteen inequivalent Si+1.71- sites. In the first Si+1.71- site, Si+1.71- is bonded in a 9-coordinate geometry to five Re³⁺ and four Si+1.71- atoms. There are a spread of Si-Si bond distances ranging from 2.40-2.75 Å. In the second Si+1.71- site, Si+1.71- is bonded in a 9-coordinate geometry to five Re³⁺ and four Si+1.71- atoms. There are two shorter (2.65 Å) and two longer (2.67 Å) Si-Si bond lengths. In the third Si+1.71- site, Si+1.71- is bonded in a distorted q6 geometry to five Re³⁺ and four Si+1.71- atoms. There are two shorter (2.65 Å) and two longer (2.68 Å) Si-Si bond lengths. In the fourth Si+1.71- site, Si+1.71- is bonded in a 9-coordinate geometry to five Re³⁺ and four Si+1.71- atoms. Both Si-Si bond lengths are 2.63 Å. In the fifth Si+1.71- site, Si+1.71- is bonded in a 4-coordinate geometry to four Re³⁺ and three Si+1.71- atoms. There is one shorter (2.39 Å) and two longer (2.87 Å) Si-Si bond lengths. In the sixth Si+1.71- site, Si+1.71- is bonded in a distorted q6 geometry to five Re³⁺ and five Si+1.71- atoms. There is one shorter (2.55 Å) and two longer (2.64 Å) Si-Si bond lengths. In the seventh Si+1.71- site, Si+1.71- is bonded in a distorted q6 geometry to five Re³⁺ and four Si+1.71- atoms. There are two shorter (2.55 Å) and one longer (2.61 Å) Si-Si bond length. In the eighth Si+1.71- site, Si+1.71- is bonded in a distorted q6 geometry to five Re³⁺ and five Si+1.71- atoms. There are a spread of Si-Si bond distances ranging from 2.62-2.65 Å. In the ninth Si+1.71- site, Si+1.71- is bonded in a distorted q6 geometry to five Re³⁺ and five Si+1.71- atoms. In the tenth Si+1.71- site, Si+1.71- is bonded in a 10-coordinate geometry to five Re³⁺ and five Si+1.71- atoms. In the eleventh Si+1.71- site, Si+1.71- is bonded in a distorted q6 geometry to five Re³⁺ and four Si+1.71- atoms. In the twelfth Si+1.71- site, Si+1.71- is bonded in a 4-coordinate geometry to three Re³⁺ and three Si+1.71- atoms. Both Si-Si bond lengths are 2.84 Å. In the thirteenth Si+1.71- site, Si+1.71- is bonded in a distorted q6 geometry to five Re³⁺ and five Si+1.71- atoms. In the fourteenth Si+1.71- site, Si+1.71- is bonded in a 10-coordinate geometry to five Re³⁺ and five Si+1.71- atoms.',
 'HfB₂ is hexagonal omega structure structured and crystallizes in the hexagonal P6/mmm space group. Hf⁴⁺ is bonded to twelve equivalent B²⁻ atoms to form a mixture of edge and face-sharing HfB₁₂ cuboctahedra. All Hf-B bond lengths are 2.51 Å. B²⁻ is bonded in a 9-coordinate geometry to six equivalent Hf⁴⁺ and three equivalent B²⁻ atoms. All B-B bond lengths are 1.81 Å.',
 'Ti₃AlC₂ is MAX Phase-like structured and crystallizes in the hexagonal P6₃/mmc space group. There are two inequivalent Ti sites. In the first Ti site, Ti is bonded in a 3-coordinate geometry to three equivalent Al and three equivalent C atoms. All Ti-Al bond lengths are 2.89 Å. All Ti-C bond lengths are 2.08 Å. In the second Ti site, Ti is bonded to six equivalent C atoms to form edge-sharing TiC₆ octahedra. All Ti-C bond lengths are 2.20 Å. Al is bonded to six equivalent Ti and six equivalent Al atoms to form distorted AlTi₆Al₆ cuboctahedra that share corners with six equivalent AlTi₆Al₆ cuboctahedra, corners with six equivalent CTi₆ octahedra, edges with six equivalent AlTi₆Al₆ cuboctahedra, edges with six equivalent CTi₆ octahedra, and faces with six equivalent AlTi₆Al₆ cuboctahedra. The corner-sharing octahedral tilt angles are 21°. All Al-Al bond lengths are 3.07 Å. C is bonded to six Ti atoms to form CTi₆ octahedra that share corners with three equivalent AlTi₆Al₆ cuboctahedra, corners with three equivalent CTi₆ octahedra, edges with three equivalent AlTi₆Al₆ cuboctahedra, and edges with nine equivalent CTi₆ octahedra. The corner-sharing octahedra are not tilted.',
 'Te₃Sn is alpha bismuth trifluoride structured and crystallizes in the cubic Fm̅3m space group. Sn³⁺ is bonded in a body-centered cubic geometry to eight equivalent Te¹⁻ atoms. All Sn-Te bond lengths are 3.34 Å. There are two inequivalent Te¹⁻ sites. In the first Te¹⁻ site, Te¹⁻ is bonded in a distorted body-centered cubic geometry to four equivalent Sn³⁺ and four equivalent Te¹⁻ atoms. All Te-Te bond lengths are 3.34 Å. In the second Te¹⁻ site, Te¹⁻ is bonded in a body-centered cubic geometry to eight equivalent Te¹⁻ atoms.',
 'TiS₂ is trigonal omega structured and crystallizes in the trigonal P3m1 space group. The structure is two-dimensional and consists of three TiS₂ sheets oriented in the (0, 0, 1) direction. Ti⁴⁺ is bonded to six equivalent S²⁻ atoms to form edge-sharing TiS₆ octahedra. All Ti-S bond lengths are 2.42 Å. S²⁻ is bonded in a distorted T-shaped geometry to three equivalent Ti⁴⁺ atoms.',
 'HfRu₃ is Uranium Silicide structured and crystallizes in the cubic Pm̅3m space group. Hf is bonded to twelve equivalent Ru atoms to form a mixture of corner and face-sharing HfRu₁₂ cuboctahedra. All Hf-Ru bond lengths are 2.80 Å. Ru is bonded in a distorted square co-planar geometry to four equivalent Hf atoms.',
 'CrH is Wurtzite structured and crystallizes in the hexagonal P6₃mc space group. Cr is bonded to four equivalent H atoms to form corner-sharing CrH₄ tetrahedra. There is one shorter (1.68 Å) and three longer (1.69 Å) Cr-H bond lengths. H is bonded to four equivalent Cr atoms to form corner-sharing HCr₄ tetrahedra.',
 'CeHSe crystallizes in the hexagonal P6₃/mmc space group. Ce³⁺ is bonded in a distorted trigonal planar geometry to three equivalent H¹⁻ and six equivalent Se²⁻ atoms. All Ce-H bond lengths are 2.33 Å. All Ce-Se bond lengths are 3.08 Å. H¹⁻ is bonded in a trigonal planar geometry to three equivalent Ce³⁺ atoms. Se²⁻ is bonded to six equivalent Ce³⁺ atoms to form a mixture of corner, edge, and face-sharing SeCe₆ octahedra. The corner-sharing octahedral tilt angles are 45°.',
 'ZnN is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Zn²⁺ is bonded to four equivalent N²⁻ atoms to form corner-sharing ZnN₄ tetrahedra. All Zn-N bond lengths are 1.97 Å. N²⁻ is bonded to four equivalent Zn²⁺ atoms to form corner-sharing NZn₄ tetrahedra.',
 'Pt₃Al crystallizes in the tetragonal P4/mbm space group. There are three inequivalent Pt sites. In the first Pt site, Pt is bonded in a distorted rectangular see-saw-like geometry to four equivalent Al atoms. There are two shorter (2.61 Å) and two longer (2.76 Å) Pt-Al bond lengths. In the second Pt site, Pt is bonded in a distorted rectangular see-saw-like geometry to four equivalent Al atoms. All Pt-Al bond lengths are 2.74 Å. In the third Pt site, Pt is bonded in a distorted water-like geometry to two equivalent Al atoms. Both Pt-Al bond lengths are 2.54 Å. Al is bonded in a 10-coordinate geometry to ten Pt atoms.',
 'HfH₂ is Fluorite-like structured and crystallizes in the tetragonal I4/mmm space group. Hf²⁺ is bonded in a body-centered cubic geometry to eight equivalent H¹⁻ atoms. All Hf-H bond lengths are 2.04 Å. H¹⁻ is bonded to four equivalent Hf²⁺ atoms to form a mixture of corner and edge-sharing HHf₄ tetrahedra.',
 'TiCrB₄ is hexagonal omega structure-derived structured and crystallizes in the hexagonal P6/mmm space group. Ti⁴⁺ is bonded to twelve equivalent B²⁻ atoms to form TiB₁₂ cuboctahedra that share edges with twelve equivalent CrB₁₂ cuboctahedra, faces with two equivalent CrB₁₂ cuboctahedra, and faces with six equivalent TiB₁₂ cuboctahedra. All Ti-B bond lengths are 2.36 Å. Cr⁴⁺ is bonded to twelve equivalent B²⁻ atoms to form CrB₁₂ cuboctahedra that share edges with twelve equivalent TiB₁₂ cuboctahedra, faces with two equivalent TiB₁₂ cuboctahedra, and faces with six equivalent CrB₁₂ cuboctahedra. All Cr-B bond lengths are 2.28 Å. B²⁻ is bonded in a 9-coordinate geometry to three equivalent Ti⁴⁺, three equivalent Cr⁴⁺, and three equivalent B²⁻ atoms. All B-B bond lengths are 1.74 Å.',
 'HfRh is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Hf is bonded in a body-centered cubic geometry to eight equivalent Rh atoms. All Hf-Rh bond lengths are 2.81 Å. Rh is bonded in a body-centered cubic geometry to eight equivalent Hf atoms.',
 'Mn(NbS₂)₄ crystallizes in the hexagonal P6₃/mmc space group. Nb³⁺ is bonded to six S²⁻ atoms to form distorted NbS₆ pentagonal pyramids that share corners with four equivalent MnS₆ octahedra, edges with two equivalent NbS₆ pentagonal pyramids, and edges with four equivalent NbS₆ pentagonal pyramids. The corner-sharing octahedral tilt angles are 48°. There are two shorter (2.46 Å) and four longer (2.52 Å) Nb-S bond lengths. Nb⁴⁺ is bonded to six equivalent S²⁻ atoms to form distorted NbS₆ pentagonal pyramids that share edges with six equivalent NbS₆ pentagonal pyramids and faces with two equivalent MnS₆ octahedra. All Nb-S bond lengths are 2.50 Å. Mn³⁺ is bonded to six equivalent S²⁻ atoms to form MnS₆ octahedra that share corners with twelve equivalent NbS₆ pentagonal pyramids and faces with two equivalent NbS₆ pentagonal pyramids. All Mn-S bond lengths are 2.51 Å. There are two inequivalent S²⁻ sites. In the first S²⁻ site, S²⁻ is bonded in a 3-coordinate geometry to three equivalent Nb³⁺ atoms. In the second S²⁻ site, S²⁻ is bonded in a rectangular see-saw-like geometry to one Nb⁴⁺, two equivalent Nb³⁺, and one Mn³⁺ atom.',
 'Mn₂As crystallizes in the tetragonal P4/nmm space group. There are two inequivalent Mn sites. In the first Mn site, Mn is bonded in a 5-coordinate geometry to five equivalent As atoms. There is one shorter (2.44 Å) and four longer (2.62 Å) Mn-As bond lengths. In the second Mn site, Mn is bonded in a 4-coordinate geometry to four equivalent As atoms. All Mn-As bond lengths are 2.45 Å. As is bonded in a 9-coordinate geometry to nine Mn atoms.',
 'KNa₂ is Hexagonal Laves structured and crystallizes in the hexagonal P6₃/mmc space group. K is bonded in a 16-coordinate geometry to four equivalent K and twelve Na atoms. There are three shorter (4.54 Å) and one longer (4.67 Å) K-K bond length. There are a spread of K-Na bond distances ranging from 4.34-4.42 Å. There are two inequivalent Na sites. In the first Na site, Na is bonded to six equivalent K and six equivalent Na atoms to form a mixture of corner, edge, and face-sharing NaK₆Na₆ cuboctahedra. All Na-Na bond lengths are 3.77 Å. In the second Na site, Na is bonded to six equivalent K and six Na atoms to form a mixture of distorted corner, edge, and face-sharing NaK₆Na₆ cuboctahedra. There are two shorter (3.68 Å) and two longer (3.70 Å) Na-Na bond lengths.',
 'IrGa is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Ir is bonded in a body-centered cubic geometry to eight equivalent Ga atoms. All Ir-Ga bond lengths are 2.62 Å. Ga is bonded in a body-centered cubic geometry to eight equivalent Ir atoms.',
 'VN is Tungsten Carbide structured and crystallizes in the hexagonal P̅6m2 space group. V³⁺ is bonded to six equivalent N³⁻ atoms to form a mixture of distorted corner, edge, and face-sharing VN₆ pentagonal pyramids. All V-N bond lengths are 2.06 Å. N³⁻ is bonded to six equivalent V³⁺ atoms to form a mixture of distorted corner, edge, and face-sharing NV₆ pentagonal pyramids.',
 'Mn₂Sc is Hexagonal Laves structured and crystallizes in the hexagonal P6₃/mmc space group. Sc is bonded in a 16-coordinate geometry to four equivalent Sc and twelve Mn atoms. There is one shorter (3.03 Å) and three longer (3.27 Å) Sc-Sc bond lengths. There are a spread of Sc-Mn bond distances ranging from 3.00-3.13 Å. There are two inequivalent Mn sites. In the first Mn site, Mn is bonded to six equivalent Sc and six equivalent Mn atoms to form a mixture of corner, edge, and face-sharing MnSc₆Mn₆ cuboctahedra. All Mn-Mn bond lengths are 2.60 Å. In the second Mn site, Mn is bonded to six equivalent Sc and six Mn atoms to form a mixture of corner, edge, and face-sharing MnSc₆Mn₆ cuboctahedra. There are two shorter (2.60 Å) and two longer (2.74 Å) Mn-Mn bond lengths.',
 'NdHg is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Nd is bonded in a body-centered cubic geometry to eight equivalent Hg atoms. All Nd-Hg bond lengths are 3.31 Å. Hg is bonded in a body-centered cubic geometry to eight equivalent Nd atoms.',
 'Mo₂Zr is Cubic Laves structured and crystallizes in the cubic Fd̅3m space group. Zr is bonded in a 12-coordinate geometry to twelve equivalent Mo atoms. All Zr-Mo bond lengths are 3.15 Å. Mo is bonded to six equivalent Zr and six equivalent Mo atoms to form a mixture of corner, edge, and face-sharing MoZr₆Mo₆ cuboctahedra. All Mo-Mo bond lengths are 2.69 Å.',
 'PaPd₃ is beta Cu3Ti-like structured and crystallizes in the hexagonal P6₃/mmc space group. Pa is bonded to twelve Pd atoms to form PaPd₁₂ cuboctahedra that share corners with six equivalent PaPd₁₂ cuboctahedra, corners with twelve PdPa₄Pd₈ cuboctahedra, edges with eighteen PdPa₄Pd₈ cuboctahedra, faces with eight equivalent PaPd₁₂ cuboctahedra, and faces with twelve PdPa₄Pd₈ cuboctahedra. There are six shorter (2.84 Å) and six longer (3.02 Å) Pa-Pd bond lengths. There are two inequivalent Pd sites. In the first Pd site, Pd is bonded to four equivalent Pa and eight Pd atoms to form distorted PdPa₄Pd₈ cuboctahedra that share corners with four equivalent PaPd₁₂ cuboctahedra, corners with fourteen PdPa₄Pd₈ cuboctahedra, edges with six equivalent PaPd₁₂ cuboctahedra, edges with twelve PdPa₄Pd₈ cuboctahedra, faces with four equivalent PaPd₁₂ cuboctahedra, and faces with sixteen PdPa₄Pd₈ cuboctahedra. There are a spread of Pd-Pd bond distances ranging from 2.81-3.00 Å. In the second Pd site, Pd is bonded to four equivalent Pa and eight Pd atoms to form distorted PdPa₄Pd₈ cuboctahedra that share corners with four equivalent PaPd₁₂ cuboctahedra, corners with fourteen PdPa₄Pd₈ cuboctahedra, edges with six equivalent PaPd₁₂ cuboctahedra, edges with twelve PdPa₄Pd₈ cuboctahedra, faces with four equivalent PaPd₁₂ cuboctahedra, and faces with sixteen PdPa₄Pd₈ cuboctahedra. Both Pd-Pd bond lengths are 3.00 Å.',
 'UBe₁₃ crystallizes in the cubic Fm̅3c space group. There are two inequivalent Be sites. In the first Be site, Be is bonded to ten Be and two equivalent U atoms to form a mixture of distorted corner, edge, and face-sharing BeU₂Be₁₀ cuboctahedra. There are a spread of Be-Be bond distances ranging from 2.15-2.34 Å. Both Be-U bond lengths are 3.01 Å. In the second Be site, Be is bonded to twelve equivalent Be atoms to form a mixture of corner and face-sharing BeBe₁₂ cuboctahedra. U is bonded in a 24-coordinate geometry to twenty-four equivalent Be atoms.',
 'CaTe is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Ca²⁺ is bonded in a body-centered cubic geometry to eight equivalent Te²⁻ atoms. All Ca-Te bond lengths are 3.40 Å. Te²⁻ is bonded in a body-centered cubic geometry to eight equivalent Ca²⁺ atoms.',
 'Y₃SnC is (Cubic) Perovskite structured and crystallizes in the cubic Pm̅3m space group. Y is bonded in a linear geometry to four equivalent Sn and two equivalent C atoms. All Y-Sn bond lengths are 3.46 Å. Both Y-C bond lengths are 2.44 Å. Sn is bonded to twelve equivalent Y atoms to form SnY₁₂ cuboctahedra that share corners with twelve equivalent SnY₁₂ cuboctahedra, faces with six equivalent SnY₁₂ cuboctahedra, and faces with eight equivalent CY₆ octahedra. C is bonded to six equivalent Y atoms to form CY₆ octahedra that share corners with six equivalent CY₆ octahedra and faces with eight equivalent SnY₁₂ cuboctahedra. The corner-sharing octahedra are not tilted.',
 'Y₂MgGe₂ crystallizes in the tetragonal P4/mbm space group. Mg²⁺ is bonded in a square co-planar geometry to four equivalent Ge⁴⁻ atoms. All Mg-Ge bond lengths are 2.88 Å. Y³⁺ is bonded in a distorted hexagonal planar geometry to six equivalent Ge⁴⁻ atoms. There are two shorter (2.95 Å) and four longer (3.07 Å) Y-Ge bond lengths. Ge⁴⁻ is bonded in a 9-coordinate geometry to two equivalent Mg²⁺, six equivalent Y³⁺, and one Ge⁴⁻ atom. The Ge-Ge bond length is 2.52 Å.',
 'Ce is Magnesium structured and crystallizes in the hexagonal P6₃/mmc space group. Ce is bonded to twelve equivalent Ce atoms to form a mixture of corner, edge, and face-sharing CeCe₁₂ cuboctahedra. There are six shorter (3.16 Å) and six longer (3.46 Å) Ce-Ce bond lengths.',
 'Be₁₇Nb₂ crystallizes in the trigonal R̅3m space group. There are four inequivalent Be sites. In the first Be site, Be is bonded in a 14-coordinate geometry to thirteen Be and one Nb atom. There are a spread of Be-Be bond distances ranging from 2.12-2.42 Å. The Be-Nb bond length is 2.59 Å. In the second Be site, Be is bonded in a 10-coordinate geometry to ten Be and two equivalent Nb atoms. There are four shorter (2.11 Å) and four longer (2.15 Å) Be-Be bond lengths. Both Be-Nb bond lengths are 2.83 Å. In the third Be site, Be is bonded to ten Be and two equivalent Nb atoms to form distorted BeBe₁₀Nb₂ cuboctahedra that share corners with twenty BeBe₉Nb₃ cuboctahedra, edges with three BeBe₉Nb₃ cuboctahedra, and faces with seventeen BeBe₁₀Nb₂ cuboctahedra. There are a spread of Be-Be bond distances ranging from 2.18-2.26 Å. Both Be-Nb bond lengths are 2.62 Å. In the fourth Be site, Be is bonded to nine Be and three equivalent Nb atoms to form distorted BeBe₉Nb₃ cuboctahedra that share corners with eighteen BeBe₉Nb₃ cuboctahedra, edges with seven BeBe₉Nb₃ cuboctahedra, and faces with seventeen BeBe₁₀Nb₂ cuboctahedra. Both Be-Be bond lengths are 2.11 Å. There are a spread of Be-Nb bond distances ranging from 2.64-2.79 Å. Nb is bonded in a 19-coordinate geometry to nineteen Be atoms.',
 'MgIr₂ is Hexagonal Laves structured and crystallizes in the hexagonal P6₃/mmc space group. Mg is bonded in a 12-coordinate geometry to twelve Ir atoms. There are a spread of Mg-Ir bond distances ranging from 3.00-3.05 Å. There are two inequivalent Ir sites. In the first Ir site, Ir is bonded to six equivalent Mg and six equivalent Ir atoms to form a mixture of corner, edge, and face-sharing IrMg₆Ir₆ cuboctahedra. All Ir-Ir bond lengths are 2.61 Å. In the second Ir site, Ir is bonded to six equivalent Mg and six Ir atoms to form a mixture of corner, edge, and face-sharing IrMg₆Ir₆ cuboctahedra. There are two shorter (2.50 Å) and two longer (2.67 Å) Ir-Ir bond lengths.',
 'YC is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Y³⁺ is bonded to four equivalent C³⁻ atoms to form corner-sharing YC₄ tetrahedra. All Y-C bond lengths are 2.37 Å. C³⁻ is bonded to four equivalent Y³⁺ atoms to form corner-sharing CY₄ tetrahedra.',
 'Li₇SbN₄ is alpha bismuth trifluoride-derived structured and crystallizes in the cubic P̅43n space group. There are five inequivalent Li¹⁺ sites. In the first Li¹⁺ site, Li¹⁺ is bonded to four N³⁻ atoms to form LiN₄ tetrahedra that share corners with sixteen LiN₄ tetrahedra, edges with two equivalent SbN₄ tetrahedra, and edges with four LiN₄ tetrahedra. All Li-N bond lengths are 2.14 Å. In the second Li¹⁺ site, Li¹⁺ is bonded to four N³⁻ atoms to form distorted LiN₄ tetrahedra that share corners with two SbN₄ tetrahedra, corners with fourteen LiN₄ tetrahedra,  an edgeedge with one SbN₄ tetrahedra, and edges with five LiN₄ tetrahedra. There are a spread of Li-N bond distances ranging from 2.10-2.26 Å. In the third Li¹⁺ site, Li¹⁺ is bonded to four N³⁻ atoms to form LiN₄ tetrahedra that share corners with four equivalent SbN₄ tetrahedra, corners with twelve LiN₄ tetrahedra, and edges with six LiN₄ tetrahedra. All Li-N bond lengths are 2.18 Å. In the fourth Li¹⁺ site, Li¹⁺ is bonded to four N³⁻ atoms to form LiN₄ tetrahedra that share corners with four SbN₄ tetrahedra, corners with twelve LiN₄ tetrahedra, and edges with six equivalent LiN₄ tetrahedra. There is one shorter (2.10 Å) and three longer (2.23 Å) Li-N bond lengths. In the fifth Li¹⁺ site, Li¹⁺ is bonded to four N³⁻ atoms to form LiN₄ tetrahedra that share corners with two equivalent SbN₄ tetrahedra, corners with fourteen LiN₄ tetrahedra,  an edgeedge with one SbN₄ tetrahedra, and edges with five LiN₄ tetrahedra. There are two shorter (2.12 Å) and two longer (2.24 Å) Li-N bond lengths. There are two inequivalent Sb⁵⁺ sites. In the first Sb⁵⁺ site, Sb⁵⁺ is bonded to four N³⁻ atoms to form SbN₄ tetrahedra that share corners with sixteen LiN₄ tetrahedra and edges with six equivalent LiN₄ tetrahedra. All Sb-N bond lengths are 2.03 Å. In the second Sb⁵⁺ site, Sb⁵⁺ is bonded to four N³⁻ atoms to form SbN₄ tetrahedra that share corners with sixteen LiN₄ tetrahedra and edges with six LiN₄ tetrahedra. All Sb-N bond lengths are 2.04 Å. There are five inequivalent N³⁻ sites. In the first N³⁻ site, N³⁻ is bonded in a body-centered cubic geometry to seven Li¹⁺ and one Sb⁵⁺ atom. In the second N³⁻ site, N³⁻ is bonded in a body-centered cubic geometry to seven Li¹⁺ and one Sb⁵⁺ atom. There are a spread of N-Li bond distances ranging from 2.10-2.24 Å. In the third N³⁻ site, N³⁻ is bonded in a body-centered cubic geometry to seven Li¹⁺ and one Sb⁵⁺ atom. In the fourth N³⁻ site, N³⁻ is bonded in a body-centered cubic geometry to seven Li¹⁺ and one Sb⁵⁺ atom. There is one shorter (2.12 Å) and one longer (2.23 Å) N-Li bond length. In the fifth N³⁻ site, N³⁻ is bonded in a body-centered cubic geometry to seven Li¹⁺ and one Sb⁵⁺ atom. There are a spread of N-Li bond distances ranging from 2.14-2.23 Å. The N-Sb bond length is 2.04 Å.',
 'NaAlF₄ crystallizes in the tetragonal P4/mmm space group. Na¹⁺ is bonded in a body-centered cubic geometry to eight equivalent F¹⁻ atoms. All Na-F bond lengths are 2.86 Å. Al³⁺ is bonded to six F¹⁻ atoms to form corner-sharing AlF₆ octahedra. The corner-sharing octahedra are not tilted. There are two shorter (1.80 Å) and four longer (1.81 Å) Al-F bond lengths. There are two inequivalent F¹⁻ sites. In the first F¹⁻ site, F¹⁻ is bonded in a linear geometry to two equivalent Al³⁺ atoms. In the second F¹⁻ site, F¹⁻ is bonded in a distorted single-bond geometry to four equivalent Na¹⁺ and one Al³⁺ atom.',
 'Ni₃Sb crystallizes in the hexagonal P6₃/mmc space group. Ni¹⁺ is bonded in a see-saw-like geometry to four equivalent Sb³⁻ atoms. There are two shorter (2.61 Å) and two longer (2.63 Å) Ni-Sb bond lengths. Sb³⁻ is bonded to twelve equivalent Ni¹⁺ atoms to form a mixture of corner and face-sharing SbNi₁₂ cuboctahedra.',
 'Al₁₂Re crystallizes in the cubic Im̅3 space group. Re is bonded in a cuboctahedral geometry to twelve equivalent Al atoms. All Re-Al bond lengths are 2.70 Å. Al is bonded in a 11-coordinate geometry to one Re and ten equivalent Al atoms. There are a spread of Al-Al bond distances ranging from 2.74-2.87 Å.',
 'Lu₃AlN is (Cubic) Perovskite structured and crystallizes in the cubic Pm̅3m space group. Lu is bonded in a linear geometry to four equivalent Al and two equivalent N atoms. All Lu-Al bond lengths are 3.25 Å. Both Lu-N bond lengths are 2.30 Å. Al is bonded to twelve equivalent Lu atoms to form AlLu₁₂ cuboctahedra that share corners with twelve equivalent AlLu₁₂ cuboctahedra, faces with six equivalent AlLu₁₂ cuboctahedra, and faces with eight equivalent NLu₆ octahedra. N is bonded to six equivalent Lu atoms to form NLu₆ octahedra that share corners with six equivalent NLu₆ octahedra and faces with eight equivalent AlLu₁₂ cuboctahedra. The corner-sharing octahedra are not tilted.',
 'ZnF₂ is Rutile structured and crystallizes in the tetragonal P4₂/mnm space group. Zn²⁺ is bonded to six equivalent F¹⁻ atoms to form a mixture of corner and edge-sharing ZnF₆ octahedra. The corner-sharing octahedral tilt angles are 50°. There are two shorter (2.02 Å) and four longer (2.04 Å) Zn-F bond lengths. F¹⁻ is bonded in a distorted trigonal planar geometry to three equivalent Zn²⁺ atoms.',
 'BeI₂ is Silicon Disuphide structured and crystallizes in the orthorhombic Ibam space group. The structure is one-dimensional and consists of two BeI₂ ribbons oriented in the (1, 0, 0) direction. Be²⁺ is bonded to four equivalent I¹⁻ atoms to form edge-sharing BeI₄ tetrahedra. All Be-I bond lengths are 2.43 Å. I¹⁻ is bonded in a distorted L-shaped geometry to two equivalent Be²⁺ atoms.',
 'BC₅ crystallizes in the trigonal P3m1 space group. B³⁺ is bonded to four C+0.60- atoms to form BC₄ tetrahedra that share corners with six equivalent BC₄ tetrahedra and corners with six CC₄ tetrahedra. There is one shorter (1.61 Å) and three longer (1.63 Å) B-C bond lengths. There are five inequivalent C+0.60- sites. In the first C+0.60- site, C+0.60- is bonded to one B³⁺ and three equivalent C+0.60- atoms to form corner-sharing CBC₃ tetrahedra. All C-C bond lengths are 1.56 Å. In the second C+0.60- site, C+0.60- is bonded to four C+0.60- atoms to form CC₄ tetrahedra that share corners with three equivalent BC₄ tetrahedra and corners with nine CC₄ tetrahedra. The C-C bond length is 1.55 Å. In the third C+0.60- site, C+0.60- is bonded to four C+0.60- atoms to form CC₄ tetrahedra that share corners with three equivalent BC₄ tetrahedra and corners with nine CC₄ tetrahedra. There is one shorter (1.50 Å) and three longer (1.57 Å) C-C bond lengths. In the fourth C+0.60- site, C+0.60- is bonded to four C+0.60- atoms to form corner-sharing CC₄ tetrahedra. In the fifth C+0.60- site, C+0.60- is bonded to three equivalent B³⁺ and one C+0.60- atom to form distorted corner-sharing CB₃C tetrahedra.',
 'CH₄ is Silicon tetrafluoride structured and crystallizes in the cubic I̅43m space group. The structure is zero-dimensional and consists of two CH₄ clusters. C⁴⁻ is bonded in a tetrahedral geometry to four equivalent H¹⁺ atoms. All C-H bond lengths are 1.09 Å. H¹⁺ is bonded in a single-bond geometry to one C⁴⁻ atom.',
 'ScSn₃ is Uranium Silicide-like structured and crystallizes in the tetragonal I4/mmm space group. Sc is bonded to twelve Sn atoms to form ScSn₁₂ cuboctahedra that share corners with four equivalent ScSn₁₂ cuboctahedra, edges with eight equivalent ScSn₁₂ cuboctahedra, edges with sixteen equivalent SnSc₄Sn₈ cuboctahedra, faces with four equivalent ScSn₁₂ cuboctahedra, and faces with eight equivalent SnSc₄Sn₈ cuboctahedra. There are four shorter (3.14 Å) and eight longer (3.36 Å) Sc-Sn bond lengths. There are two inequivalent Sn sites. In the first Sn site, Sn is bonded to four equivalent Sc and eight Sn atoms to form distorted SnSc₄Sn₈ cuboctahedra that share corners with twelve equivalent SnSc₄Sn₈ cuboctahedra, edges with eight equivalent ScSn₁₂ cuboctahedra, edges with eight equivalent SnSc₄Sn₈ cuboctahedra, faces with four equivalent ScSn₁₂ cuboctahedra, and faces with ten equivalent SnSc₄Sn₈ cuboctahedra. There are four shorter (3.14 Å) and four longer (3.36 Å) Sn-Sn bond lengths. In the second Sn site, Sn is bonded in a square co-planar geometry to four equivalent Sc and eight equivalent Sn atoms.',
 'DyB₂ is hexagonal omega structure structured and crystallizes in the hexagonal P6/mmm space group. Dy is bonded to twelve equivalent B atoms to form a mixture of edge and face-sharing DyB₁₂ cuboctahedra. All Dy-B bond lengths are 2.71 Å. B is bonded in a 9-coordinate geometry to six equivalent Dy and three equivalent B atoms. All B-B bond lengths are 1.90 Å.',
 'Pt₃Pb is Uranium Silicide structured and crystallizes in the cubic Pm̅3m space group. Pt is bonded to eight equivalent Pt and four equivalent Pb atoms to form PtPt₈Pb₄ cuboctahedra that share corners with twelve equivalent PtPt₈Pb₄ cuboctahedra, edges with eight equivalent PbPt₁₂ cuboctahedra, edges with sixteen equivalent PtPt₈Pb₄ cuboctahedra, faces with four equivalent PbPt₁₂ cuboctahedra, and faces with fourteen equivalent PtPt₈Pb₄ cuboctahedra. All Pt-Pt bond lengths are 2.90 Å. All Pt-Pb bond lengths are 2.90 Å. Pb is bonded to twelve equivalent Pt atoms to form PbPt₁₂ cuboctahedra that share corners with twelve equivalent PbPt₁₂ cuboctahedra, edges with twenty-four equivalent PtPt₈Pb₄ cuboctahedra, faces with six equivalent PbPt₁₂ cuboctahedra, and faces with twelve equivalent PtPt₈Pb₄ cuboctahedra.',
 'PuBi is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Pu is bonded in a body-centered cubic geometry to eight equivalent Bi atoms. All Pu-Bi bond lengths are 3.36 Å. Bi is bonded in a body-centered cubic geometry to eight equivalent Pu atoms.',
 'Bi₂O₃ crystallizes in the trigonal P31c space group. There are two inequivalent Bi³⁺ sites. In the first Bi³⁺ site, Bi³⁺ is bonded in a 5-coordinate geometry to five O²⁻ atoms. There are a spread of Bi-O bond distances ranging from 2.14-2.41 Å. In the second Bi³⁺ site, Bi³⁺ is bonded in a 3-coordinate geometry to three equivalent O²⁻ atoms. All Bi-O bond lengths are 2.15 Å. There are two inequivalent O²⁻ sites. In the first O²⁻ site, O²⁻ is bonded in a distorted trigonal non-coplanar geometry to three Bi³⁺ atoms. In the second O²⁻ site, O²⁻ is bonded in a 3-coordinate geometry to three equivalent Bi³⁺ atoms.',
 'Li(TiSe₂)₂TiSe₂ crystallizes in the trigonal P̅3m1 space group. The structure is two-dimensional and consists of one Li(TiSe₂)₂ sheet oriented in the (0, 0, 1) direction and one TiSe₂ sheet oriented in the (0, 0, 1) direction. In the Li(TiSe₂)₂ sheet, Li¹⁺ is bonded to six equivalent Se²⁻ atoms to form LiSe₆ octahedra that share corners with twelve equivalent TiSe₆ octahedra, edges with six equivalent LiSe₆ octahedra, and faces with two equivalent TiSe₆ octahedra. The corner-sharing octahedral tilt angles are 46°. All Li-Se bond lengths are 2.68 Å. Ti⁴⁺ is bonded to six Se²⁻ atoms to form TiSe₆ octahedra that share corners with six equivalent LiSe₆ octahedra, edges with six equivalent TiSe₆ octahedra, and  a faceface with one LiSe₆ octahedra. The corner-sharing octahedral tilt angles are 46°. There are three shorter (2.54 Å) and three longer (2.61 Å) Ti-Se bond lengths. There are two inequivalent Se²⁻ sites. In the first Se²⁻ site, Se²⁻ is bonded to three equivalent Li¹⁺ and three equivalent Ti⁴⁺ atoms to form a mixture of distorted corner and edge-sharing SeLi₃Ti₃ pentagonal pyramids. In the second Se²⁻ site, Se²⁻ is bonded in a distorted T-shaped geometry to three equivalent Ti⁴⁺ atoms. In the TiSe₂ sheet, Ti³⁺ is bonded to six equivalent Se²⁻ atoms to form edge-sharing TiSe₆ octahedra. All Ti-Se bond lengths are 2.57 Å. Se²⁻ is bonded in a distorted T-shaped geometry to three equivalent Ti³⁺ atoms.',
 'Pb₂Rh is Khatyrkite structured and crystallizes in the tetragonal I4/mcm space group. Rh is bonded in a 10-coordinate geometry to two equivalent Rh and eight equivalent Pb atoms. Both Rh-Rh bond lengths are 2.94 Å. All Rh-Pb bond lengths are 2.91 Å. Pb is bonded in a 5-coordinate geometry to four equivalent Rh and one Pb atom. The Pb-Pb bond length is 3.15 Å.',
 'PdTa is gamma CuTi structured and crystallizes in the tetragonal P4/nmm space group. The structure is two-dimensional and consists of two PdTa sheets oriented in the (0, 0, 1) direction. Ta is bonded in a 4-coordinate geometry to four equivalent Pd atoms. All Ta-Pd bond lengths are 2.69 Å. Pd is bonded in a 4-coordinate geometry to four equivalent Ta atoms.',
 'Ca₃SnO is (Cubic) Perovskite structured and crystallizes in the cubic Pm̅3m space group. Ca is bonded in a linear geometry to four equivalent Sn and two equivalent O atoms. All Ca-Sn bond lengths are 3.41 Å. Both Ca-O bond lengths are 2.41 Å. Sn is bonded to twelve equivalent Ca atoms to form SnCa₁₂ cuboctahedra that share corners with twelve equivalent SnCa₁₂ cuboctahedra, faces with six equivalent SnCa₁₂ cuboctahedra, and faces with eight equivalent OCa₆ octahedra. O is bonded to six equivalent Ca atoms to form OCa₆ octahedra that share corners with six equivalent OCa₆ octahedra and faces with eight equivalent SnCa₁₂ cuboctahedra. The corner-sharing octahedra are not tilted.',
 'Be(OH)₂ crystallizes in the orthorhombic P2_12_12₁ space group. Be²⁺ is bonded to four O²⁻ atoms to form corner-sharing BeO₄ tetrahedra. There are a spread of Be-O bond distances ranging from 1.62-1.65 Å. There are two inequivalent H¹⁺ sites. In the first H¹⁺ site, H¹⁺ is bonded in a single-bond geometry to one O²⁻ atom. The H-O bond length is 0.98 Å. In the second H¹⁺ site, H¹⁺ is bonded in a single-bond geometry to one O²⁻ atom. The H-O bond length is 0.98 Å. There are two inequivalent O²⁻ sites. In the first O²⁻ site, O²⁻ is bonded in a distorted trigonal planar geometry to two equivalent Be²⁺ and one H¹⁺ atom. In the second O²⁻ site, O²⁻ is bonded in a distorted trigonal non-coplanar geometry to two equivalent Be²⁺ and one H¹⁺ atom.',
 'Cu₃Se₂ is Hazelwoodite-derived structured and crystallizes in the tetragonal P̅42₁m space group. Cu¹⁺ is bonded to four equivalent Se²⁻ atoms to form distorted CuSe₄ tetrahedra that share corners with four equivalent CuSe₄ tetrahedra, corners with ten equivalent CuSe₄ tetrahedra,  an edgeedge with one CuSe₄ tetrahedra, and edges with two equivalent CuSe₄ tetrahedra. There are a spread of Cu-Se bond distances ranging from 2.41-2.45 Å. Cu²⁺ is bonded to four equivalent Se²⁻ atoms to form distorted CuSe₄ tetrahedra that share corners with four equivalent CuSe₄ tetrahedra, corners with eight equivalent CuSe₄ tetrahedra, and edges with four equivalent CuSe₄ tetrahedra. All Cu-Se bond lengths are 2.52 Å. Se²⁻ is bonded in a 6-coordinate geometry to two equivalent Cu²⁺ and four equivalent Cu¹⁺ atoms.',
 'BeAu₂ crystallizes in the tetragonal I4/mmm space group. The structure is two-dimensional and consists of two BeAu₂ sheets oriented in the (0, 0, 1) direction. Be is bonded in a body-centered cubic geometry to eight equivalent Au atoms. All Be-Au bond lengths are 2.51 Å. Au is bonded in a 4-coordinate geometry to four equivalent Be atoms.',
 'Al₂Y is Cubic Laves structured and crystallizes in the cubic Fd̅3m space group. Y is bonded in a 16-coordinate geometry to four equivalent Y and twelve equivalent Al atoms. All Y-Y bond lengths are 3.40 Å. All Y-Al bond lengths are 3.25 Å. Al is bonded to six equivalent Y and six equivalent Al atoms to form a mixture of corner, edge, and face-sharing AlY₆Al₆ cuboctahedra. All Al-Al bond lengths are 2.77 Å.',
 'CuSe is Covellite-derived structured and crystallizes in the hexagonal P6₃/mmc space group. Cu¹⁺ is bonded to one Se²⁻ and three equivalent Se¹⁻ atoms to form CuSe₄ tetrahedra that share corners with three equivalent SeCu₃Se tetrahedra and corners with seven equivalent CuSe₄ tetrahedra. The Cu-Se bond length is 2.49 Å. All Cu-Se bond lengths are 2.42 Å. Cu²⁺ is bonded in a trigonal planar geometry to three equivalent Se²⁻ atoms. All Cu-Se bond lengths are 2.32 Å. Se¹⁻ is bonded to three equivalent Cu¹⁺ and one Se¹⁻ atom to form distorted SeCu₃Se tetrahedra that share corners with three equivalent CuSe₄ tetrahedra, corners with six equivalent SeCu₃Se tetrahedra, and corners with three equivalent SeCu₅ trigonal bipyramids. The Se-Se bond length is 2.43 Å. Se²⁻ is bonded to two equivalent Cu¹⁺ and three equivalent Cu²⁺ atoms to form SeCu₅ trigonal bipyramids that share corners with six equivalent SeCu₃Se tetrahedra and corners with six equivalent SeCu₅ trigonal bipyramids.',
 'Ag₂Ga is zeta silver zinc structured and crystallizes in the hexagonal P̅62m space group. There are two inequivalent Ag sites. In the first Ag site, Ag is bonded to eight Ag and four Ga atoms to form a mixture of distorted corner, edge, and face-sharing AgGa₄Ag₈ cuboctahedra. There are a spread of Ag-Ag bond distances ranging from 2.91-2.99 Å. There are two shorter (2.69 Å) and two longer (2.77 Å) Ag-Ga bond lengths. In the second Ag site, Ag is bonded in a 13-coordinate geometry to eight Ag and five Ga atoms. Both Ag-Ag bond lengths are 2.91 Å. There is one shorter (2.80 Å) and four longer (2.87 Å) Ag-Ga bond lengths. There are two inequivalent Ga sites. In the first Ga site, Ga is bonded in a 11-coordinate geometry to nine Ag and two equivalent Ga atoms. Both Ga-Ga bond lengths are 2.91 Å. In the second Ga site, Ga is bonded in a 11-coordinate geometry to nine Ag and two equivalent Ga atoms. Both Ga-Ga bond lengths are 2.91 Å.',
 'Pm₂O₃ crystallizes in the trigonal P̅3m1 space group. Pm³⁺ is bonded in a 7-coordinate geometry to seven O²⁻ atoms. There are a spread of Pm-O bond distances ranging from 2.29-2.65 Å. There are two inequivalent O²⁻ sites. In the first O²⁻ site, O²⁻ is bonded to four equivalent Pm³⁺ atoms to form OPm₄ tetrahedra that share corners with six equivalent OPm₆ octahedra, corners with six equivalent OPm₄ tetrahedra, edges with three equivalent OPm₆ octahedra, and edges with three equivalent OPm₄ tetrahedra. The corner-sharing octahedral tilt angles range from 18-56°. In the second O²⁻ site, O²⁻ is bonded to six equivalent Pm³⁺ atoms to form OPm₆ octahedra that share corners with twelve equivalent OPm₄ tetrahedra, edges with six equivalent OPm₆ octahedra, and edges with six equivalent OPm₄ tetrahedra.',
 'PbI₂ is trigonal omega-like structured and crystallizes in the trigonal R̅3m space group. The structure is two-dimensional and consists of three PbI₂ sheets oriented in the (0, 0, 1) direction. Pb²⁺ is bonded to six equivalent I¹⁻ atoms to form edge-sharing PbI₆ octahedra. All Pb-I bond lengths are 3.26 Å. I¹⁻ is bonded in a 3-coordinate geometry to three equivalent Pb²⁺ atoms.',
 'Pd₃Zn is Uranium Silicide-like structured and crystallizes in the hexagonal P6₃/mmc space group. Pd is bonded to eight equivalent Pd and four equivalent Zn atoms to form distorted PdZn₄Pd₈ cuboctahedra that share corners with four equivalent ZnPd₁₂ cuboctahedra, corners with fourteen equivalent PdZn₄Pd₈ cuboctahedra, edges with six equivalent ZnPd₁₂ cuboctahedra, edges with twelve equivalent PdZn₄Pd₈ cuboctahedra, faces with four equivalent ZnPd₁₂ cuboctahedra, and faces with sixteen equivalent PdZn₄Pd₈ cuboctahedra. There are a spread of Pd-Pd bond distances ranging from 2.72-2.78 Å. There are two shorter (2.73 Å) and two longer (2.75 Å) Pd-Zn bond lengths. Zn is bonded to twelve equivalent Pd atoms to form ZnPd₁₂ cuboctahedra that share corners with six equivalent ZnPd₁₂ cuboctahedra, corners with twelve equivalent PdZn₄Pd₈ cuboctahedra, edges with eighteen equivalent PdZn₄Pd₈ cuboctahedra, faces with eight equivalent ZnPd₁₂ cuboctahedra, and faces with twelve equivalent PdZn₄Pd₈ cuboctahedra.',
 'SnBSc₃ is (Cubic) Perovskite structured and crystallizes in the cubic Pm̅3m space group. Sc is bonded in a distorted linear geometry to two equivalent B and four equivalent Sn atoms. Both Sc-B bond lengths are 2.33 Å. All Sc-Sn bond lengths are 3.29 Å. B is bonded to six equivalent Sc atoms to form BSc₆ octahedra that share corners with six equivalent BSc₆ octahedra and faces with eight equivalent SnSc₁₂ cuboctahedra. The corner-sharing octahedra are not tilted. Sn is bonded to twelve equivalent Sc atoms to form SnSc₁₂ cuboctahedra that share corners with twelve equivalent SnSc₁₂ cuboctahedra, faces with six equivalent SnSc₁₂ cuboctahedra, and faces with eight equivalent BSc₆ octahedra.',
 'F₂ is Cubic alpha N2-like structured and crystallizes in the monoclinic C2/m space group. The structure is zero-dimensional and consists of four F₂ clusters. F is bonded in a single-bond geometry to one F atom. The F-F bond length is 1.41 Å.',
 'Ag₃Zn is Uranium Silicide structured and crystallizes in the cubic Pm̅3m space group. Ag is bonded to eight equivalent Ag and four equivalent Zn atoms to form AgZn₄Ag₈ cuboctahedra that share corners with twelve equivalent AgZn₄Ag₈ cuboctahedra, edges with eight equivalent ZnAg₁₂ cuboctahedra, edges with sixteen equivalent AgZn₄Ag₈ cuboctahedra, faces with four equivalent ZnAg₁₂ cuboctahedra, and faces with fourteen equivalent AgZn₄Ag₈ cuboctahedra. All Ag-Ag bond lengths are 2.86 Å. All Ag-Zn bond lengths are 2.86 Å. Zn is bonded to twelve equivalent Ag atoms to form ZnAg₁₂ cuboctahedra that share corners with twelve equivalent ZnAg₁₂ cuboctahedra, edges with twenty-four equivalent AgZn₄Ag₈ cuboctahedra, faces with six equivalent ZnAg₁₂ cuboctahedra, and faces with twelve equivalent AgZn₄Ag₈ cuboctahedra.',
 'KMgH₃ is (Cubic) Perovskite structured and crystallizes in the cubic Pm̅3m space group. K¹⁺ is bonded to twelve equivalent H¹⁻ atoms to form KH₁₂ cuboctahedra that share corners with twelve equivalent KH₁₂ cuboctahedra, faces with six equivalent KH₁₂ cuboctahedra, and faces with eight equivalent MgH₆ octahedra. All K-H bond lengths are 2.82 Å. Mg²⁺ is bonded to six equivalent H¹⁻ atoms to form MgH₆ octahedra that share corners with six equivalent MgH₆ octahedra and faces with eight equivalent KH₁₂ cuboctahedra. The corner-sharing octahedra are not tilted. All Mg-H bond lengths are 1.99 Å. H¹⁻ is bonded to four equivalent K¹⁺ and two equivalent Mg²⁺ atoms to form a mixture of distorted corner, edge, and face-sharing HK₄Mg₂ octahedra. The corner-sharing octahedral tilt angles range from 0-60°.',
 'BaSiC is Copper structured and crystallizes in the tetragonal I4mm space group. The structure is zero-dimensional and consists of two BaSiC clusters. Ba²⁺ is bonded in a single-bond geometry to one C⁴⁻ atom. The Ba-C bond length is 2.87 Å. Si²⁺ is bonded in a single-bond geometry to one C⁴⁻ atom. The Si-C bond length is 1.71 Å. C⁴⁻ is bonded in a distorted linear geometry to one Ba²⁺ and one Si²⁺ atom.',
 'Ir₃Si crystallizes in the tetragonal I4/mcm space group. There are two inequivalent Ir sites. In the first Ir site, Ir is bonded in a distorted square co-planar geometry to four equivalent Si atoms. All Ir-Si bond lengths are 2.63 Å. In the second Ir site, Ir is bonded in a distorted water-like geometry to two equivalent Si atoms. Both Ir-Si bond lengths are 2.42 Å. Si is bonded in a 8-coordinate geometry to eight Ir atoms.',
 'HoAg is Tetraauricupride structured and crystallizes in the cubic Pm̅3m space group. Ho is bonded in a body-centered cubic geometry to eight equivalent Ag atoms. All Ho-Ag bond lengths are 3.11 Å. Ag is bonded in a body-centered cubic geometry to eight equivalent Ho atoms.',
 'MnTe is Zincblende, Sphalerite structured and crystallizes in the cubic F̅43m space group. Mn²⁺ is bonded to four equivalent Te²⁻ atoms to form corner-sharing MnTe₄ tetrahedra. All Mn-Te bond lengths are 2.76 Å. Te²⁻ is bonded to four equivalent Mn²⁺ atoms to form corner-sharing TeMn₄ tetrahedra.',
 'Al₃Hf is Uranium Silicide-like structured and crystallizes in the tetragonal I4/mmm space group. Hf is bonded to twelve Al atoms to form HfAl₁₂ cuboctahedra that share corners with four equivalent AlHf₄Al₈ cuboctahedra, corners with eight equivalent HfAl₁₂ cuboctahedra, edges with four equivalent HfAl₁₂ cuboctahedra, edges with twenty AlHf₄Al₈ cuboctahedra, faces with five equivalent HfAl₁₂ cuboctahedra, and faces with thirteen AlHf₄Al₈ cuboctahedra. There are a spread of Hf-Al bond distances ranging from 2.81-2.99 Å. There are three inequivalent Al sites. In the first Al site, Al is bonded to four equivalent Hf and eight Al atoms to form distorted AlHf₄Al₈ cuboctahedra that share corners with four equivalent HfAl₁₂ cuboctahedra, corners with eight equivalent AlHf₄Al₈ cuboctahedra, edges with four equivalent HfAl₁₂ cuboctahedra, edges with twenty AlHf₄Al₈ cuboctahedra, faces with five equivalent HfAl₁₂ cuboctahedra, and faces with thirteen AlHf₄Al₈ cuboctahedra. There are four shorter (2.91 Å) and four longer (2.92 Å) Al-Al bond lengths. In the second Al site, Al is bonded to four equivalent Hf and eight Al atoms to form distorted AlHf₄Al₈ cuboctahedra that share corners with twelve AlHf₄Al₈ cuboctahedra, edges with eight equivalent HfAl₁₂ cuboctahedra, edges with sixteen AlHf₄Al₈ cuboctahedra, faces with four equivalent HfAl₁₂ cuboctahedra, and faces with fourteen AlHf₄Al₈ cuboctahedra. All Al-Al bond lengths are 2.81 Å. In the third Al site, Al is bonded to four equivalent Hf and eight Al atoms to form AlHf₄Al₈ cuboctahedra that share corners with twelve AlHf₄Al₈ cuboctahedra, edges with eight equivalent HfAl₁₂ cuboctahedra, edges with sixteen AlHf₄Al₈ cuboctahedra, faces with four equivalent HfAl₁₂ cuboctahedra, and faces with fourteen AlHf₄Al₈ cuboctahedra. All Al-Al bond lengths are 2.81 Å.',
 'Ca₂Zn is zeta iron carbide structured and crystallizes in the tetragonal P4₂/mnm space group. Ca is bonded in a 3-coordinate geometry to three equivalent Zn atoms. There is one shorter (2.98 Å) and two longer (3.03 Å) Ca-Zn bond lengths. Zn is bonded to six equivalent Ca atoms to form a mixture of corner and edge-sharing ZnCa₆ octahedra. The corner-sharing octahedral tilt angles are 41°.']
  # Provide actual text inputs

# True labels for the above texts
true_labels = [0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1]
# Lists to store predictions and results
predicted_labels = []
inference_results = []

# Run inference
for i, text_input in enumerate(texts):
    predicted_class, probabilities = predict(text_input)
    predicted_labels.append(predicted_class)
    true_label = true_labels[i]
    is_correct = predicted_class == true_label

    result = f"Instance {i+1}:"

    result += f"True Label: {true_label}, Predicted Label: {predicted_class}\n"
    result += f"Probabilities: {probabilities.tolist()}\n"
    result += f"Correct Prediction: {'True' if is_correct else 'False'}\n"
    result += "-" * 50 + "\n"
    inference_results.append(result)

# Compute evaluation metrics
f1 = f1_score(true_labels[:len(predicted_labels)], predicted_labels, average='weighted')
mcc = matthews_corrcoef(true_labels[:len(predicted_labels)], predicted_labels)

# Save inference results to a text file
output_file = "inference_results_project.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.writelines(inference_results)
    f.write(f"F1 Score: {f1:.4f}\n")
    f.write(f"Matthews Correlation Coefficient: {mcc:.4f}\n")

# Print evaluation metrics
print(f"F1 Score: {f1:.4f}")
print(f"Matthews Correlation Coefficient: {mcc:.4f}")
print(f"Inference results saved to {output_file}")


F1 Score: 0.8225
Matthews Correlation Coefficient: 0.6333
Inference results saved to inference_results_project.txt
